# Time Series Data Downloader 
### by Ilia Parshakov

This script downloads time series data for a user-specified MGRS tile.
<br />• Raw DN 60m Landsat 1 - 5 MSS.
<br />• 30m surface reflectance or raw Landsat 4 - 9 data and 15m raw panchromatic band for Landsat 7 - 9. Each image takes 137 MB (+95 MB pan band) of disk space as an uncompressed tif + 141 MB or less as numpy arrays.
<br />• TOA reflectance or surface reflectance 10m and 20m Sentinel-2 data. 8-band 20m image (VNIR, RE, and SWIR) = 475MB (uncompressed tif) and 4-band 10m VNIR image (optional) = 844 MB. Processing time: ~50s/image.
<br />• Harmonized Landsat Sentinel-2 (HLS) data downloading from earthdata.nasa.gov (enter your Earthdata username and password in the Parameters cell).
<br />• GEDI canopy height and biomass data resampled from 25m to the resolution of any of the optical sensors. Processing time (previous version, no biomass): 43 months of data in 15m45s.
<br />• Yearly Google Satellite Embedding (2017 onward) data resampled from 10m to the resolution of any of the optical sensors. 64 "bands" - expect very large files (e.g., 1.5 GB per year at 30m).
<br />• Auxiliary data used in preprocessing: 30m NASA DEM (or any other DEM available on GEE) for topo correction; water and agriculture mask based on 30m AAFC and/or 10m ESA WorldCover map. MODIS MCD12Q2 is used to determine the growing season.
<br />• Preprocessing: topographic correction, image coregistration in GEE (optional), MSS and Sentinel-2 cloud and cloud-shadow masking, snow and fire smoke masking, same-date same-sensor Landsat image mosaicking, coregistration (MSS to Landsat 4 TM or 5 TM)
<br />• Incorrectly georeferenced MSS images are filtered out using a water mask (location of water bodies in MSS image vs Landsat 5 TM). Low-quality MSS images are filtered based on interband correlation.

Possible issues that have not been assessed or addressed yet:
<br />• All data are resampled to the UTM zone of the MGRS tile. This means that no reprojection is applied to Sentinel-2 and most of Landsat data. However, some Landsat images are reprojected from adjacent UTM projections. The effect of this resampling on the quality of data hasn't been assessed.
<br />• Image coregistration in GEE (optional) may blur images. Bicubic convolution should work better than bilinear (at least it does when it comes to reprojection), but not in this case for some reason.
<br />• Some S2 images (only 2016?) on GEE have two copies with different 'PRODUCT_ID' (probably because the naming convention changed in Dec 2016). This is not a problem if skip_existing is set to True.
<br />• TSD makes only two attempts to download each HLS image. This may not be sufficient.
<br />• Landsat QA band sometime does not cover image edges. In this case, TSD does not mask clouds in those areas. There is no easy solution here. The QA band is 30m and, unfortunately, it includes SLC-off stripes, which are sometimes wider/longer than in 30m and 15m pan bands, so masking all pixels outside the QA band coverage will remove useful data. Using own algorithms to mask clouds outside the QA coverage might be the only solution.

Future work:
<br />• Landsat QA band sometimes does not cover the entire image, resulting in unmasked clouds. Should mask out pixels outside QA band coverage.
<br />• GEDI data are currently downloaded as raster files (it would make more sense to download as vector files).
<br />• Use yearly AROSICS (Automated and Robust Open-Source Image Co-Registration Software) client-side coregistration instead of GEE coregistration?
<br />• Use a different value for invalid/masked pixels instead of zero. Topographic correction can produce negative values, which are masked out in the current version.
<br />• Download narrower tiles where two UTM zones overlap.
<br />• Add GUI.

Notes:
<br />• The pixel grids of Landsat 30m, Landsat panchromatic, Landsat MSS, and Sentinel-2 images do not align: https://www.usgs.gov/media/images/differences-between-pixel-area-and-pixel-point-designations. This means that downloaded images do not cover exactly the same area on the ground (e.g., there is 15m shift between 30m Landsat and Sentinel-2). This script (TSD) does not align the pixel grids of different sensors because that would require bicubic resampling, resulting in blurry images. Take this into account in your multisensor data processing.
<br />• There is no surface reflectance Sentinel-2 for 2016 on GEE but there is on ESA Copernicus Data Space. Set download_ESA_S2 to True to download Sentinel-2 data from the Copernicus Hub instead of GEE.

This is an original script except for some of the code used for topographic correction and cloud-shadow masking. Please cite "Time Series Data Downloader: automatic preprocessing and downloading of Landsat 1 – 9, Sentinel-2, and other remote sensing data for multi-resolution time-series image analysis" (working title of the technical note). Contact emails: parshakov@uleth.alumni.ca or ilya85parshakov@gmail.com

If you use TSD in your research, please cite: 
<br />Parshakov, I., & Chen, D., 2025. Time Series Data Downloader: automatic preprocessing and downloading of Landsat 1–9, Sentinel-2, and other remote sensing data for multi-resolution time-series image analysis. Submitted to Big Earth Data.

### Instructions:
• This is a python script in the form of a Jupyter notebook. You can open and run it using one of many free code editors, like Microsoft’s Visual Studio Code.
<br />• Before running the script, create a Cloud Project on GEE: https://developers.google.com/earth-engine/cloud/earthengine_cloud_project_setup
<br />• Use this link to select a tile: https://mappingsupport.com/p2/gissurfer.php?center=18TUP&zoom=4 (e.g., tile for Ottawa is 18TVR). Alternatively, use the "Sentinel-2 tiling grid kml" file available on https://sentiwiki.copernicus.eu/web/s2-products 
<br />• Enter the tile number in the Parameters cell and run the script.

When you run the script,
1. accounts.google.com will open in your default web browser automatically.
2. Click "Generate token", then select your Google Earth Engine account.
3. A warning message will pop up saying "Google hasn’t verified this app". Click "Continue".
4. Make sure to check both "View and manage your Google Earth Engine data" and "Manage your data and permissions in Cloud Storage and see the email address for your Google Account."
5. Click "Continue" and copy the generated authorization code in the "Enter verification code" pop-up in your code editor.

Images will be downloaded directly into RAM and saved as numpy arrays (two files per image: one containing pixel values and the other containing pixel locations) and geotif images (tif + ENVI header file containing band names and wavelengths). 

### Changelog 
2025-08-16
- Added option to download the newly released Google Satellite Embedding data
- Bug fixes and optimizations.

2025-04-28
- Thermal band downloading (band 6 of Landsat 4-5, only B6_VCID_2 (high gain) for Landsat 7, and only Band 10 for L8 and L9) - enable the parameter download_L_thermal
- Improved Landsat 7 pan masking. Multispectral cloud masks sometime do not cover all pan pixels. As before, the multispectral "datapixel" mask is used to mask areas outside the cloud mask coverage. However, 2px erosion and 1px dilation is now applied to the datapixel mask to preserve pixels near SLC-off stripes.
- Sentinel-2 downloading script now tries to ensure 95% coverage of the study area per year (this was only applied to Landsat 1 - 9 previously), resulting in improved coverage for the years 2016 and 2017, which have very low availability of Sentinel-2 data. Need to rewrite it so it's downloads data yearly???.
- SWIR1 is now used instead of SWIR2 for snow and smoke masking in Landsat images.
- Smoke masking based on B1 and SWIR1 is added for S2 - not tested properly yet! Cloud Score+ is already good for smoke masking, maybe band thresholds are not required.

2024-10-14
- Option to download Sentinel-2 data from the copernicus website instead of GEE. This is the new default setting. Provide your username_esa and password_esa to use this function. Unlike the Copernicus website, GEE does not replace old S2 images when ESA updates their preprocessing algorithms, so older S2 images (pre 2024) have serious geometric accuracy issues on GEE. GEE's Cloud Score+ cloud masks are applied to Sentinel-2 data downloaded from the copernicus website.
- Python version of the Sun Canopy Sensor + C Correction (SCS + C) algorithm for the topographic correction of Level 1C Sentinel-2 data downloaded from the copernicus website.
- Code updated to work with the latest version of Python (3.12.6).

2024-07-20
- Replaced the Sentinel-2 cloud masking algorithm with "Cloud Score+", resulting in more accurate cloud masks and much faster downloading.
- Harmonized Landsat Sentenel-2 (HLS) data are now downloaded as multiband geotifs with ENVI header files and given proper georeferencing information.
- Downloading normal (Collection 2, as opposed to HLS) Landsat and Sentinel-2 data is now optional.
- Added option not to perform MSS coregistration. Added option to perform S2 coregistration.
- Better snow masking in Sentinel-2 and Landsat. A buffer is added to snow masks.
- Option to download old UC-Change maps for British Columbia (produced in 2020) and phenology maps based on MCD12Q2 MODIS data.
- Option to use multiyear Landsat 8&9 pan image composites for Sentinel-2 coregistration and multiyear Landsat 4&5 image composites for MSS coregistration (see the parameters use_TM_composite and composite_pan).
- Fixed bug when images are provided with slightly incorrect georeferencing information.
- Stability improvements but GEE can still crash if it's not in the mood.

2023-10-24
- Bug fixes
- Option to download existing forest change and canopy height maps: C2C 1984 - 2020 harvest and forest fire map (Canada only); GFC 2000 - 2022 forest change map; Potapov's 30m canopy height map; and Lang's 10m canopy height map. Search for "download_other_maps" in this notebook.

2023-08-26
- Bug fixes

2023-07-31
- Wildfire smoke masking in Landsat data using red and SWIR2 thresholds.
- Landsat 7 - 9 15m panchromatic data downloading. Note: atmospherically corrected Landsat images do not have a pan band. If you want SR multispectral bands and raw pan data, run this script with sr = True, then again with sr = False and skip_existing = True. This way, only the pan band will be downloaded during the second run.
- Faster Landsat 4 - 9 downloading.

2023-06-21
- Proper snow masking in Landsat and Sentinel-2 images. Topographic correction is highly sensitive to unmasked snow on north-facing slope, so this can greatly improve the topo correction of May and Sep images.
- ENVI header files .hdr containing band names and wavelengths are generated for every image. ENVI will pull this information automatically from these header files when opening downloaded geotifs.
- Additional filter for MSS for removing images with inaccurate georeferencing

2023-06-10
- Images are now saved as geotifs (previous versions saved images as regular tifs, without any georeferencing information).
- Bug fixes

2023-06-05 
- Added HLS, GEDI biomass, and 10m Sentinel-2 download
- Automatic start and end of growing season using MCD12Q2 MODIS dataset
- Co-registration of Sentinel-2 with Landsat panchromatic and MSS with Landsat 5 TM. Translation (linear alignment = x and y shift, but no rotation) only in case of Sentinel-2.

### This cell automatically installs required packages

In [ ]:
import importlib
import subprocess
import sys

# List of packages to install (package_name, import_name)
packages = [
    ('earthengine-api', 'ee'),
    ('pyproj', 'pyproj'),
    ('rasterio', 'rasterio'),
    ('folium', 'folium'),
    ('scipy', 'scipy')
]

for package_name, import_name in packages:
    try:
        importlib.import_module(import_name)
        print(f"{package_name} is already installed.")
    except ImportError:
        print(f"{package_name} is not installed. Installing now...")
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_name])
            print(f"{package_name} has been installed.")
        except subprocess.CalledProcessError as e:
            print(f"⚠ Failed to install {package_name}: {e}")

In [ ]:
# if you are getting an error like "ModuleNotFoundError: No module named 'rasterio'", make sure to select the right environment (and before that, make sure you've installed all the modules from the cell above, e.g., 'pip install rasterio'). In VS Code, press Ctrl+Shift+P → type 'Notebook: Select Notebook Kernel', and select the environment

import io
import requests
import numpy as np
import math
import os
import time
import rasterio
import pandas as pd
from scipy.ndimage import minimum_filter
from scipy.ndimage import zoom
from pyproj import Transformer, CRS
from datetime import datetime
from joblib import parallel_backend, Parallel, delayed
from http import cookiejar
from urllib import request
from io import BytesIO
from scipy.ndimage import binary_erosion
from scipy.ndimage import binary_dilation

### When you run the next cell, follow this steps to connect to your Google Earth Engine account:
1. accounts.google.com will open in your default browser automatically.
2. Click "Generate token", then select your Google Earth Engine account.
3. A warning message will pop up saying "Google hasn’t verified this app". Make sure there is no malicious code embedded in the script, then click "Continue".
4. Make sure to check both "View and manage your Google Earth Engine data" and "Manage your data and permissions in Cloud Storage and see the email address for your Google Account."
5. Click "Continue" and copy the generated authorization code in the "Enter verification code" pop-up in your code editor (top of the screen in Visual Studio).

In [ ]:
for attempt in range(5):
    try:
        import ee
        ee.Authenticate()
        ee.Initialize()
        break
    except:
        print('Retrying to connect to GEE...')

In [ ]:
# # Run this cell to fully disconnect from GEE. Do it when you keep getting errors like "EEException: Earth Engine capacity exceeded." After that, comment out this cell again and re-run the script.

# ee.Reset()
# if 'EARTHENGINE_TOKEN' in os.environ:
#     del os.environ['EARTHENGINE_TOKEN']

# credentials_path = os.path.expanduser("~/.config/earthengine/credentials")
# if os.path.exists(credentials_path):
#     os.remove(credentials_path)

## Parameters

'buffer' is set relative to the full-size Sentinel-2 tile. Sentinel-2 tiles are based on MGRS tiles. MGRS tiles are 100km x 100km in size; Sentinel-2 tiles are 109800m x 109800m because there is a 9800m overlap between adjacent Sentinel-2 tiles south and east of the corresponding MGRS tiles.

Data are saved into your default python directory. If 11ULR is the tile you want to download, two folders are created automatically: 'your_default_python_folder/11ULR/' for numpy arrays and 'your_default_python_folder/11ULR_tif/' for optional tif images.

In [ ]:
# Your earthdata.nasa.gov username and password - enter only if you need Harmonized Landsat Sentinel-2 (HLS) 30m data.
username_nasa = ""
password_nasa = ""

# Your Copernicus Data Space Ecosystem (https://browser.dataspace.copernicus.eu/) username and password - enter if you need more consistent Sentinel-2 data (v5.00 preprocessing using 30m DEM instead of 90m DEM). GEE does not replace old S2 images when ESA reprocesses them using newer image preprocessing algorithms.
username_esa = ""
password_esa = ""

MGRS_tile = '10UCV' # 10UCV = Vancouver Island # 11ULR = Kelowna # 18TUQ = Kingston # 18TXR = Montreal # 47WMQ = Siberia # Use this link to select a tile: https://mappingsupport.com/p2/gissurfer.php?center=18TUP&zoom=4

start_year = 1972 # 1972
end_year = datetime.now().year # datetime.now().year # datetime.now().year = current year, but you can enter any number (e.g., 2023)
start_DOY = 0 # 0 # 152 = June 1 # if set to 0, the algorithm will automatically determine this using MODIS MCD12Q2 MidGreenup_1 data
end_DOY = 0 # 0 # 262 = Sep 19 # if set to 0, the algorithm will automatically determine this using MODIS MCD12Q2 MidGreendown_1 data

sr = False # download surface reflectance data if true, raw DN if false. Note: there is no sr data for MSS, DN will be downloaded regardless.
do_topo = False # topographic correction (except SR Sentinel-2, which is already topo corrected)

mask_agriculture = True # mask out agriculture in addition to water using ESA WorldCover 10m (https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v200) and AAFC 30m map (https://developers.google.com/earth-engine/datasets/catalog/AAFC_ACI - Canada only) .
use_AAFC = True # for Canada only: use AAFC map in addition to the ESA WorldCover water map to mask out agricultural land and water. Disabling this parameter may speed up download of images but note that the AAFC map is more accurate than the ESA crop map.
AAFC_year = 2017

skip_existing_agri = True # set this to False after changing the use_AAFC parameter (default: False). If True, the script will use already downloaded AAFC and water masks (which take ~2 minutes to redownload) instead of downloading them again (this is useful when testing other parameters).
water_agri_mask_HLS = True # apply water or water+agri mask to HLS images too

skip_existing = True # don't re-download images if true
min_coverage = 0.95 # download enough images to cover 95% of the tile per year
min_pan_coverage = 0.9 # download enough Landsat 7 pan images to cover 90% of the tile per year for the years 1999 to 2011 (see the parameter pan_only for more info)
mid_summer_only = False # download only midsummer data (use MODIS MCD12Q2 Maturity_1 to Senescence_1 instead of MidGreenup_1 to MidGreendown_1)

do_L = True # select and download Landsat data
download_normal_L = True # download Landsat Collection 2 data (SR or raw DN depending on the value of sr)
download_L_pan = True # download the 15m panchromatic band for raw DN (not available for SR Landsat) Landsat 7 and newer Landsat sensors
download_L_thermal = False # download the thermal band of L4-L9
download_HLS_L = False # only L8 and L9; observed issues: much blurrier than normal L due to double resampling (15m shift); no topographic correction (and TSD does not apply topo correction to HLS data); issues with atmospheric correction (negative values in the blue band)
do_pan_topo = True # perform topographic illumination correction on panchromatic image, but only if it's also applied to multispectral images. If do_topo = False, do_pan_topo is set to False automatically.
if not do_topo:
   do_pan_topo = False

do_S2 = True # select and download Sentinel-2 data
cloudiness_s2_max = 80 # maximum percentage of cloudy pixels in S2 images.
cloudiness_s2_min = 40 # if cloudiness_s2_min = 40%, start by checking whether images with cloud cover <40% ensure 95% coverage of the tile (or whatever the value of min_coverage is). If not, incremetally increase this threshold until min_coverage target or cloudiness_s2_max is reached. 
download_ESA_S2 = True # download S2 data from Copernicus Hub instead of GEE for better quality (default: True).
download_GEE_S2 = False # mutually exclusive with download_ESA_S2
download_S2_10m = True # if False, resample 10m bands to 20m before downloading
s2_cloud_mask_type = 'cs' # Cloud Score+ GEE S2 cloud masks: use 'cs' or 'cs_cdf'. The cs band tends to be more sensitive to haze and cloud edges (which is great if you need only the absolute clearest pixels) while cs_cdf tends to be less sensitive to these low-magnitude spectral changes as well as terrain shadows (potentially giving you more “usable” pixels to work with).
s2_cloud_score_threshold = 0.80 # "values between 0.50 and 0.65 generally work well. Higher values will remove thin clouds, haze & cirrus shadows."
mask_esa_s2_with_gee = True # Default: True = use Cloud Score+ GEE masks to mask S2 images downloaded from ESA Copernicus Hub. If False, the script will use the masks provided by ESA (which are not as good as GEE masks).
DEM_forESA_S2 = 'ESA' # DEM for client-side topographic correction of Sentinel-2 data downloaded from Copernicus Hub. Possible values: ESA NASA CSA ALOS
download_HLS_S2 = False # available in 30m resolution only. Note: the do_topo parameter does not apply to HLS data; HLS data is not topographically corrected and currently TSD does not apply topo correction to HLS data.
do_coregister_S2 = False # this enables GEE coregistration with Landsat pan. Not necessary if download_ESA_S2 = True.
align_S2 = False # Client-side simple global coregistration with L pan. Not necessary if download_ESA_S2 = True. 
composite_pan = False # use a multiyear (2018 - present) mid-summer L8 or L9 pan image composite instead of a single mid-summer L8 or L9 pan for S2 coregistration. Enable this if the tile is in between two Landsat paths but GEE might crash.
if download_ESA_S2:
   download_GEE_S2 = False

do_MSS = True # select and download MSS data
coregister_MSS = False # this enables GEE coregistration with Landsat TM
cloud_height_m = 3000 # Cloud shadows are projected on the ground based on this height (since it cannot be determined automatically), sun angle, and pixel values (darker pixels = shadow). 
cloud_shadow_distance = ee.Number(20) # maximum distance in pixels to search for cloud shadows from cloud edges in MSS images
align_MSS = False # simple global coregistration: shift MSS image until it aligns with the reference L image (client-side operation; doesn't work well when a lot of unmasked clouds)
use_TM_composite = True # use a multiyear (1982 - 1993) mid-summer L4 or L5 image composite instead of a single mid-summer L4 or L5 image for MSS coregistration. Enable this if the tile is in between two Landsat paths.

do_GEDI = True # download monthly GEDI composites as 20m and 30m raster images
GEDI_resolutions = ['L','S2'] # resample original 25m GEDI raster images to to the pixel grids of these sensors. Possible values: 'M','L','L_pan','S2', 'S2_10m', 'HLS'.

do_GEE_embedding = True # download the Google Satellite Embedding dataset https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_SATELLITE_EMBEDDING_V1_ANNUAL Requires a lot of storage (16-bit 10m).
embedding_resolutions = ['L'] # For original resolution, set this to ['S2_10m']. ['L', 'S2'] would download Google Embedding data in 30m Landsat and 20m Sentinel-2 pixel grids, resampling from 10m using Pixel Aggregation. 

pan_only = False # if pan_only = True, download enough L7 images to ensure min_pan_coverage yearly coverage of the study area based on L7 alone, for the years 1999 to 2011. This is important if you want to "pan-sharpen" UC-Change maps. To download all Landsat data from 1982 to present, set pan_only to False first and then to True.
if pan_only: # do not change these
   start_year = 1999
   end_year = 2011
   min_coverage = min_pan_coverage
   do_S2 = False
   do_MSS = False
   do_GEDI = False 

bands = {} # bands to download
bands['M'] = ['G', 'R', 'NIR1', 'NIR2']
bands['L'] = ['B','G', 'R', 'NIR', 'SWIR1', 'SWIR2'] # do not add thermal or pan bands here! instead, set the download_L_thermal or download_pan parameters to True 
bands['S2_10m'] = ['B','G','R','NIR_wide']
bands['S2_20m'] = ['RE1','RE2','RE3','NIR', 'SWIR1', 'SWIR2'] # only one of the NIR bands (the 20m one) is used in the 20m stack (10m and 20m bands are combined into a single 20m S2 stack)

# OTHER 
clearsky_threshold_global = 0.7 # this is automatically reduced for Landsat 30m and MSS data to ensure min_coverage of the area for each year. No need to change this in most cases.
download_other_maps = False # download c2c, gfc, ccdc, phenology, canopy_height and other maps

# Original S2 tiles are always extended by 9.8 km towards south and east. "The S-2 tiles’ upper-left corner not always match perfectly the MGRS tiles’ corner and thus the overlap varies between 9.78 km and 9.84 km." (from "2023 Wasting petabytes - A survey of the Sentinel-2 UTM tiling grid and its spatial overhead")
tile_overlap_m = 2960 # images will expand beyond the MGRS tile to the east and south by this number of meters. # Future work: a different overlap should be set for the east direction dynamically based on the overlap of UTM zones! tile_overlap_m is adjusted automatically so that (100km+tile_overlap_m) is a multiple of 240 (the resolution of reducers)

reducer_resolution_client = 240
reducer_resolution = ee.Number(reducer_resolution_client)
cloud_buffer_m = 240.0 # would it make sense to use different buffer sizes for different sensors?

max_shift_s2_10m = 5 # in pixels. Shift image vertically and horizontally one pixel at a time until it aligns with L_pan
max_shift_s2_10m = 5
max_shift_MSS = 5
max_shift_MSS = 5

# == cloud (MSS), snow and smoke thresholds

NIR2_MSS_dark_threshold = 10 # TOA reflectance threshold for cloud-shadow detection in MSS data

snow_thres_green_toa = 15 # TOA in % for snow masking in MSS and S2 # changed from 15% to 20% on 2025-06-25
snow_thres_swir1_toa = 15 # TOA in % for snow masking in S2 only. Used SWIR2=10% before 2025-04-29
ndsi_thres_s2 = 0.0 # NDSI threshold for snow masking in SR S2

# smoke has high visible and low SWIR reflectance, just like snow.
smoke_thres_aerosol_toa = 15 # 15
smoke_thres_blue_toa = 15 # TOA in % # Works better than the green band for a July 2 Landsat 5 image, but need more testing
smoke_thres_swir1_toa = 15 # 10 # TOA in % # Used SWIR2=10% before 2025-04-29

smoke_thres_aerosol_sr = 8 # SR in %
smoke_thres_blue_sr = 10 # SR in %
smoke_thres_swir1_sr = 10 # SR in %. Used SWIR2=10% before 2025-04-29

# =============

smoke_thres_aerosol_L_sr = smoke_thres_aerosol_sr * 363.63635 + 7272.727
smoke_thres_blue_L_sr = smoke_thres_blue_sr * 363.63635 + 7272.727
smoke_thres_swir1_L_sr = smoke_thres_swir1_sr * 363.63635 + 7272.727 # scaled TOA
smoke_thres_aerosol_L_sr = ee.Number(smoke_thres_aerosol_L_sr)
smoke_thres_blue_L_sr = ee.Number(smoke_thres_blue_L_sr)
smoke_thres_swir1_L_sr = ee.Number(smoke_thres_swir1_L_sr)

snow_thres_green_s2 = snow_thres_green_toa*100 # scaled TOA
snow_thres_swir1_s2 = snow_thres_swir1_toa*100 # scaled TOA

s2_toa_b1_threshold = smoke_thres_aerosol_toa*100
s2_toa_swir1_threshold = smoke_thres_swir1_toa*100
s2_sr_swir1_threshold = smoke_thres_swir1_sr*100
s2_sr_b1_threshold = smoke_thres_aerosol_sr*100
ndsi_thres_s2 = ee.Number(ndsi_thres_s2) # no need to do this

NIR2_MSS_dark_threshold = NIR2_MSS_dark_threshold/100.0
green_MSS_bright_threshold = snow_thres_green_toa/100.0 # TOA: 0 to 1.0

smoke_thres_aerosol_Lraw = ee.Number(smoke_thres_aerosol_toa/100.0)
smoke_thres_blue_Lraw = ee.Number(smoke_thres_blue_toa/100.0)
smoke_thres_swir1_Lraw = ee.Number(smoke_thres_swir1_toa/100.0)

save_tif = True
save_np = True
compress_tif = False # compress tif files using LZW lossless compression (note: ENVI takes a long of time to open compressed tifs, so you might want to set this to False if you are using ENVI)
transpose_np = True # Transpose images if required for Python classifier compatibility (from shape [bands x valid_pixels] to shape [valid_pixels x bands])

In [ ]:
# Mid-summer Landsat 8 - 9 panchromatic (2016 - present) and Landsat 5 TM images (1984 - 1988) are used for Sentinel-2 and MSS coregistration, respectively, as reference images:
start_DOY_reference = 0 # 180 # if set to 0, the algorithm will automatically determine this using MODIS data MCD12Q2 Maturity_1 data
end_DOY_reference = 0 # 240 # if set to 0, the algorithm will automatically determine this using MODIS data MCD12Q2 Senescence_1 data

# =============

S2_total_tile_width_m = 109800 # meters

h_meters = 100000 + tile_overlap_m
w_meters = 100000 + tile_overlap_m

if not (h_meters/240).is_integer(): # if the user-defined tile width is not a multiple of 240m, adjust it. 
    tile_overlap_m = 240*int(h_meters/240) - 100000 # here! the overlap will be reduced. This is better than increasing it.
    h_meters = 100000 + tile_overlap_m
    w_meters = 100000 + tile_overlap_m
    print('adjusted overlap:', tile_overlap_m)
  
print('Output image height and width in meters (all sensors):', h_meters,w_meters)

# create a directory
if os.path.exists(MGRS_tile) == False:
  print('No data found on this computer for tile ',MGRS_tile,'. Downloading from GEE...')
  usegee = True
  os.makedirs(MGRS_tile)
if os.path.exists(MGRS_tile+'_tif') == False:
  os.makedirs(MGRS_tile+'_tif')

np.save(MGRS_tile+'/height_and_width', [h_meters,w_meters])

if not (len(password_nasa) >= 8 and any(char.isdigit() for char in password_nasa)): # check if the password is >= 8 characters and contains a number, as required by earthdata
  print('earthdata.nasa.gov username and password were not provided. HLS data will not be downloaded.')
  download_HLS_L = False

In [ ]:
sensors = ['M','L','S2']
subsensors = ['M','L','L_pan','S2', 'S2_10m', 'HLS']
n_images = {}
dates_str = {}
sensor_numbers = {}
n_pixels = {}
resolution = {'M': 60, 'L': 30, 'S2': 20, 'L_pan': 15, 'S2_10m': 10, 'HLS': 30} # , 'PL': 5

h = {}
w = {}

if 'NIR_wide' in bands['S2_10m'] and 'NIR' in bands['S2_20m']:
    bands['S2'] = [band for band in bands['S2_10m'] if band != 'NIR_wide'] + bands['S2_20m'] # when creating a 20m stack of all S2 bands, use only one of the NIR bands (the 20m one)
else:
    bands['S2'] = bands['S2_10m'] + bands['S2_20m']

wavelengths_all = {}
wavelengths_all['_M1'] = {'G': 0.55,'R': 0.65, 'NIR1':0.75, 'NIR2': 0.95}
wavelengths_all['_M2'] = {'G': 0.55,'R': 0.65, 'NIR1':0.75, 'NIR2': 0.95}
wavelengths_all['_M3'] = {'G': 0.55,'R': 0.65, 'NIR1':0.75, 'NIR2': 0.95}
wavelengths_all['_M4'] = {'G': 0.55,'R': 0.65, 'NIR1':0.75, 'NIR2': 0.95}
wavelengths_all['_M5'] = {'G': 0.55,'R': 0.65, 'NIR1':0.75, 'NIR2': 0.95}
wavelengths_all['_L4'] = {'B': 0.485,'G': 0.56,'R': 0.66,'NIR':0.83,'SWIR1':1.65,'SWIR2':2.215, 'T': 11.45}
wavelengths_all['_L5'] = {'B': 0.485,'G': 0.56,'R': 0.66,'NIR':0.83,'SWIR1':1.65,'SWIR2':2.215, 'T': 11.45}
wavelengths_all['_L7'] = {'B': 0.485,'G': 0.56,'R': 0.66,'NIR':0.835,'SWIR1':1.65,'SWIR2':2.22, 'T': 11.45}
wavelengths_all['_L8'] = {'B': 0.48,'G': 0.56,'R': 0.655,'NIR':0.865,'SWIR1':1.61,'SWIR2':2.2, 'T': 10.895} # 'T1': 10.895, 'T2': 12.005, 'T': 11.45
wavelengths_all['_L9'] = {'B': 0.48,'G': 0.56,'R': 0.655,'NIR':0.865,'SWIR1':1.61,'SWIR2':2.2, 'T': 10.895}
wavelengths_all['_S2'] = {'B': 0.49,'G': 0.56,'R': 0.665,'NIR_wide':0.842,'RE1':0.705,'RE2':0.74,'RE3':0.783,'NIR':0.865,'SWIR1':1.61,'SWIR2':2.19}

wavelengths_pan = {}
wavelengths_pan['_L7'] = {'pan': 0.71} # red + NIR
wavelengths_pan['_L8'] = {'pan': 0.59} # green + red
wavelengths_pan['_L9'] = {'pan': 0.59} # green + red

n_bands = {}
n_bands['M'] = len(bands['M'])
n_bands['L'] = len(bands['L']) 
n_bands['S2'] = len(bands['S2'])

water_AAFC_mask = {}

UTM_zone = int(MGRS_tile[0:2])

## Get MGRS_tile info and measure misalignment between the pixel grids of different sensors and the Sentinel-2 pixel grid

In [ ]:
MGRS_tile = MGRS_tile.upper()
mgrs_coords = MGRS_tile.encode('utf-8')

# 1. Get the Sentinel-2 tile footprint in WGS84
B12 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterMetadata('MGRS_TILE', 'equals', MGRS_tile).first().select('B12')

# 2. Get tile coordinates in WGS84 lat/lon
coords = B12.geometry().bounds().coordinates().getInfo()[0]  # [[lon, lat], ...] in order, for each image corner
# For top-left (NW corner)
lon_W = coords[0][0]
lat_N = coords[2][1]

utm_zone = int(MGRS_tile[-5:-3])
is_northern = True if MGRS_tile[-3:-2] > 'M' else False  # Determine if the tile is in the southern hemisphere
epsg_number = 32600 + utm_zone if is_northern else 32700 + utm_zone
crs_UTM_str = f"EPSG:{epsg_number}"
S2_projection = ee.Projection(crs_UTM_str)
crs = CRS.from_dict({'proj': 'utm', 'zone': utm_zone, 'south': False if is_northern else True})
print("UTM EPSG:", crs_UTM_str)
print("UTM zone:", utm_zone)
print("MGRS_tile:", MGRS_tile)

# Setup transformer
transformer = Transformer.from_crs("EPSG:4326", crs_UTM_str, always_xy=True)

# 4. Compute NW corner in UTM
w_easting, n_northing = transformer.transform(lon_W, lat_N) # lon, lat coordinates do not translate precisely to UTM coordinates, so we get the actual UTM coordinates directly from the pixel in the NW corner of a Sentinel-2 image (see the code below)

print('S2 NW corner coordinates (from lat/lon): northing =', n_northing, '; easting =', w_easting)

# 5. Create geometry rectangle for downloading a tiny chip of Sentinel-2 image in the NW corner. This chip constains the NW-most pixel in the image with true UTM coordinates.
geometry_coords = ee.Geometry.Rectangle(**{
    'coords': [w_easting-2000, n_northing-2000, w_easting+2000, n_northing+2000], 
    'proj': crs_UTM_str,
    'geodesic': False,
    'evenOdd': True
})

# 6. Pixel coordinates image
xy = B12.pixelCoordinates(ee.Projection(crs_UTM_str)).updateMask(B12) # 2-band image containing pixel coordinates as values

xy_np = ''
for attempt in range(50):
    url = xy.getDownloadUrl({'bands': ['y','x'], 'region': geometry_coords, 'scale': 20, 'format': 'NPY'}) # download the tiny image chip
    response = requests.get(url)
    try:
        xy_np = np.load(io.BytesIO(response.content))
    except:
        xy_np = ''
    if isinstance(xy_np, np.ndarray):
        break

xy_np = np.dstack((xy_np['x'], xy_np['y'])).transpose(2, 0, 1) # convert to 2-band np array (band 1 for easting and band 2 for northing)

# 7. Compute final extents (based on Sentinel-2 tile with added overlap)
n_northing = xy_np[1].max() + 10 # +10 because GEE returns the coordinates of pixel center
w_easting = xy_np[0][xy_np[0] != 0].min() - 10 # -10 because GEE returns the coordinates of pixel center
s_northing = n_northing - h_meters
e_easting = w_easting + w_meters

print('S2 NW corner coordinates (actual): northing =', n_northing, '; easting =', w_easting)

for r in resolution.values():
    h[r] = round(h_meters / r)
    w[r] = round(w_meters / r)
    n_pixels[r] = h[r] * w[r]

for s in subsensors:
    h[s] = h[resolution[s]]
    w[s] = w[resolution[s]]
    n_pixels[s] = h[s] * w[s]

h['HLS'] = h[30]
w['HLS'] = w[30]
n_pixels['HLS'] = h[30]*w[30]

# Tile center
S2_tile_center = ee.Geometry.Point(**{
        'coords': [w_easting + w_meters/2, n_northing - h_meters/2],
        'proj': crs_UTM_str,
        'geodesic': False,
        'evenOdd': True
})

S2_tile = {}
tiles = {} # produce polygons for subtiles. Each subtile must contain no more than 48MB of data, so use more subtiles for higher-bit, higher spatial resultion data. Or just use the maximum number of subtiles.

shift_test_collections = {'M':'LANDSAT/LM01/C02/T2', 'L':'LANDSAT/LC09/C02/T1','S2':'COPERNICUS/S2_HARMONIZED'}
shift_test_bands = {'M':['B4'], 'L':['B2', 'B8'],'S2':['B12', 'B3']}
shift_test_suffix = {'M':['M'], 'L':['L', 'L_pan'],'S2':['S2', 'S2_10m']}
shift_test_suffix = {'M':['M'], 'L':['L', 'L_pan'],'S2':['S2', 'S2_10m']}

# The ee.Image.pixelCoordinates function in Google Earth Engine assumes pixel coordinates = the coordinates of pixel centers. This is not true for Sentinel-2, so we correct for this here.
shift_test_correction_x = {'M':0, 'L':0, 'L_pan':0,'S2':-resolution['S2']/2.0, 'S2_10m':-resolution['S2_10m']/2.0}
shift_test_correction_y = {'M':0, 'L':0, 'L_pan':0,'S2':resolution['S2']/2.0, 'S2_10m':resolution['S2_10m']/2.0}

# download a tiny image chip (180m x 180m) to get pixel coordinates
test_tile = ee.Geometry.Rectangle(**{
    'coords': [w_easting, n_northing-180,w_easting+180,n_northing],
    'proj': crs_UTM_str,
    'geodesic': False,
    'evenOdd': True
    })

shift_y = {} # positive values mean downloaded images are y meters north of the tile
shift_x = {} # positive values mean downloaded images are x meters east of the tile
metadata = {}
transform = {}
nw_northing_and_easting = {}

shift_y['S2'] = 0
shift_x['S2'] = 0
shift_y['S2_10m'] = 0
shift_x['S2_10m'] = 0

pixel_shift = {}
for s in subsensors:
    pixel_shift[s] = 0 if s == 'S2' or s == 'S2_10m' else resolution[s]/2.0

for s in sensors:
  for index, band in enumerate(shift_test_bands[s]):
    subsensor = shift_test_suffix[s][index]

    if s != 'S2' or s != 'HLS':
        # print('Determining pixel-grid shift for',subsensor,'...')
        xy = ee.ImageCollection(shift_test_collections[s]).filterMetadata('UTM_ZONE', 'equals', UTM_zone).filterBounds(test_tile).first().select(band).pixelCoordinates(S2_projection)
        xy_np = ''
        for attempt in range(50):
            url = xy.getDownloadUrl({'bands': ['y','x'], 'region': test_tile, 'scale':resolution[subsensor], 'format': 'NPY'})
            response = requests.get(url)
            try:
                xy_np = np.load(io.BytesIO(response.content))
            except:
                print('pickles!')
                xy_np = ''
            if isinstance(xy_np, np.ndarray):
                break

        print(subsensor)
        first_pixel_northing, first_pixel_easting = xy_np[-1,0] # not sure why the last row of pixels instead of the first one, but that's how it is
        print('Sentinel-2 NW northing', n_northing, 'vs',subsensor,first_pixel_northing)
        print('Sentinel-2 NW easting', w_easting, 'vs',subsensor, first_pixel_easting)
        shift_y[subsensor] = first_pixel_northing - n_northing + shift_test_correction_y[subsensor]
        shift_x[subsensor] = first_pixel_easting - w_easting + shift_test_correction_x[subsensor]
        np.save(MGRS_tile+'/'+subsensor+'_shift',np.array([shift_y[subsensor],shift_x[subsensor]]))
        print('pixel-grid shift y', shift_y[subsensor],'; x', shift_x[subsensor],'\n')   

    n_northing_subsensor = n_northing + shift_y[subsensor]
    w_easting_subsensor = w_easting + shift_x[subsensor]
    s_northing_subsensor = n_northing_subsensor - h[subsensor]*resolution[subsensor]
    e_easting_subsensor = w_easting_subsensor + w[subsensor]*resolution[subsensor]

    nw_northing_and_easting[subsensor] = [n_northing_subsensor,w_easting_subsensor]
    np.save(MGRS_tile+'/northing_and_easting_'+subsensor,np.array([n_northing_subsensor,w_easting_subsensor]))

    transform[subsensor] = [resolution[subsensor], 0, w_easting_subsensor, 0, -1*resolution[subsensor], n_northing_subsensor]

    metadata[subsensor] = {
        'driver': 'GTiff',
        'dtype': np.uint16,
        'count': 1,  # Number of bands
        'nodata': 0,
        'height': h[subsensor], # sadly, rasterio cannot determine this and dtype automatically...
        'width': w[subsensor],
        'transform': rasterio.Affine(resolution[subsensor], 0, w_easting_subsensor, 0, -1*resolution[subsensor], n_northing_subsensor),
        'crs': crs
    }

    tiles[subsensor] = {}

    S2_tile[subsensor] = ee.Geometry.Rectangle(**{
            'coords': [w_easting_subsensor,s_northing_subsensor,e_easting_subsensor,n_northing_subsensor],
            'proj': crs_UTM_str,
            'geodesic': False,
            'evenOdd': True
        })

    for n_tiles in [1,4,16]:
        tiles_this = []
        n_tiles_sqrt = int(math.sqrt(n_tiles))
        tile_h = h_meters/n_tiles_sqrt
        tile_w = w_meters/n_tiles_sqrt

        for qv in range(n_tiles_sqrt):
            for qh in range(n_tiles_sqrt):
                new_tile = ee.Geometry.Rectangle(**{
                    'coords': [w_easting_subsensor+qh*tile_w,s_northing_subsensor+qv*tile_h,w_easting_subsensor+(qh+1)*tile_w,s_northing_subsensor+(qv+1)*tile_h], # [w_easting_subsensor+qh*tile_w,s_northing_subsensor+qv*tile_h,w_easting_subsensor+(qh+1)*tile_w,s_northing_subsensor+(qv+1)*tile_h], # 'coords': [w_easting_subsensor+qh*tile_w,s_northing_subsensor+qv*tile_h + resolution[s],w_easting_subsensor+(qh+1)*tile_w - resolution[s],s_northing_subsensor+(qv+1)*tile_h],
                    'proj': crs_UTM_str,
                    'geodesic': False,
                    'evenOdd': True
                    })
                tiles_this.append(new_tile)

        tiles[subsensor][n_tiles] = tiles_this

# use a larger geometry for image coregistration to avoid no-data pixels due to image shift.
S2_tile['GEE'] = ee.Geometry.Rectangle(**{ 
        'coords': [w_easting_subsensor-180,s_northing_subsensor-180,e_easting_subsensor+180,n_northing_subsensor+180],
        'proj': crs_UTM_str,
        'geodesic': False,
        'evenOdd': True
    })

shift_y['HLS'] = 0
shift_x['HLS'] = 0
metadata['HLS'] = metadata['L'].copy()
metadata['HLS']['transform'] = rasterio.Affine(30, 0, w_easting, 0, -30, n_northing)
transform['HLS'] = [30, 0, w_easting, 0, -30, n_northing]

tiles['HLS'] = {}
tiles['HLS'][16] = tiles['S2'][16] # for downloading AAFC

S2_tile_Centroid = S2_tile['S2'].centroid(**{'maxError': 1}).coordinates().getInfo()

# 'M_30m' is L5 image used for MSS coregistration
w['M_30m'] = w['M']*2
h['M_30m'] = h['M']*2
resolution['M_30m'] = 30
shift_y['M_30m'] = shift_y['M']
shift_x['M_30m'] = shift_x['M']
metadata['M_30m'] = metadata['L'].copy()
metadata['M_30m']['transform'] = rasterio.Affine(30, 0, w_easting + shift_x['M'], 0, -30, n_northing + shift_y['M'])
transform['M_30m'] = [30, 0, w_easting + shift_x['M'], 0, -30, n_northing + shift_y['M']]
tiles['M_30m'] = {}
tiles['M_30m'][16] = tiles['M'][16] # for downloading AAFC

hd_sensors = {'M':'','L':'L_pan','S2':'S2_10m'}

shift_to_hd_y = {}
shift_to_hd_x = {}
for s in hd_sensors.keys():
    if hd_sensors[s] != '':
        shift_to_hd_y[s] = nw_northing_and_easting[hd_sensors[s]][0] - nw_northing_and_easting[s][0]
        shift_to_hd_x[s] = nw_northing_and_easting[s][1] - nw_northing_and_easting[hd_sensors[s]][1]

linearFitGeometry = ee.Geometry(S2_tile['S2'].buffer(-5000)) # forgot why...

np.save(MGRS_tile+'/northing_and_easting', [n_northing,w_easting])

with open(MGRS_tile+'/crs.txt', 'w') as file:
    file.write(crs_UTM_str)

## Universal functions

In [ ]:
# add buffer to clouds
def cloud_bufferer(mask, cloud_buffer):
  notCloud_buffered = np.full((mask.shape[0] + 2*cloud_buffer, mask.shape[1] + 2*cloud_buffer), True)
  notCloud_buffered[cloud_buffer: mask.shape[0] + cloud_buffer, cloud_buffer: mask.shape[1] + cloud_buffer] = mask
  notCloud_buffered = minimum_filter(notCloud_buffered, size=2*cloud_buffer+1, mode='constant', cval=1)
  return notCloud_buffered[cloud_buffer:mask.shape[0] + cloud_buffer, cloud_buffer:mask.shape[1] + cloud_buffer]

def np2tif(image_np, filename, subsensor, band_names_this = '', wavelengths = ''):
  if image_np.ndim != 3: # if only one band
    image_np = image_np[np.newaxis, :, :] # reshape to make compatible with rasterio

    if not isinstance(band_names_this, list):
      band_names_this = [band_names_this]

  n_bands_this = image_np.shape[0]
  height_this = image_np.shape[1]
  width_this = image_np.shape[2]

  # Create a rasterio dataset and write the array as a GeoTIFF file
  meta = metadata[subsensor].copy()
  meta['dtype'] = image_np.dtype
  meta['count'] = n_bands_this

  if compress_tif:
    # Lossless GeoTIFF image compression. This is not recognized by ENVI
    # meta['compress'] = 'deflate'           # lossless compression
    # meta['predictor'] = 2                  # horizontal differencing predictor
    # meta['tiled'] = True                   # improves performance in some viewers
    meta['compress'] = 'lzw' # 'deflate' is not supported by ENVI, 'lzw' can half storage requirements, but takes a long time to open in ENVI
    # # optional: remove predictor or leave it unset
    # meta.pop('predictor', None)
    meta['tiled'] = True

  with rasterio.open(MGRS_tile+'_tif/'+filename+'.tif', 'w', **meta) as dst:
    if n_bands_this == 1:
      dst.write(image_np)
    else:
      for band in range(n_bands_this):
          dst.write(image_np[band], band + 1)

  bandname_str = '{'+','.join(band_names_this)+'}'
  wavelength_str = '{'+', '.join([str(wavelengths[band_name]) for band_name in band_names_this])+'}' # if isinstance(wavelengths, list) else '{'+str(wavelengths)+'}'

  content = f'''ENVI
  samples = {width_this}
  lines   = {height_this}
  bands   = {n_bands_this}
  file type = TIFF
  interleave = bsq
  wavelength units = Micrometers
  band names = {bandname_str}
  wavelength = {wavelength_str}'''

  with open(MGRS_tile+'_tif/'+filename+'.hdr', 'w') as file: # create an ENVI header file containing band names and wavelengths. Georeferencing infromation is contained in the geotiff. Open geotiff in ENVI and it will automatically pull info from the header file
      file.write(content) 

# lossless compression of numpy files: save pixel values and pixel locationas separately to save disk space
def save_data_and_mask(image_np, short_name, subsensor, band_names_this = '', wavelengths = ''): 
  if save_tif:
    np2tif(image_np, short_name, subsensor, band_names_this, wavelengths)
  if save_np:
    # Identify valid pixels (non-zero in first band)
    data_where = image_np[0] > 0 # Shape: (height, width)
    np.save(MGRS_tile+'/'+short_name+'_mask', data_where)
    
    image_np = image_np[:,data_where] # Shape (bands, height, width) becomes (bands, valid_pixels)
    if transpose_np: # Transpose is required for classifier compatibility (to shape: valid_pixels x bands)
      image_np = np.transpose(image_np)
      
    np.save(MGRS_tile+'/'+short_name+'_data', image_np)
    print('n_notmasked',np.count_nonzero(data_where),'/',n_pixels[subsensor])  

max_shift = int(max(resolution.values()) / 2.5)

def gcd_float(x, y):
    factor = 1000  # Adjust this factor based on your precision requirements
    x_int = int(x * factor)
    y_int = int(y * factor)
    gcd = math.gcd(x_int, y_int)
    return gcd / factor # int(gcd / factor)

def resize_mixed2zero(img, from_resolution, to_resolution):
  intermediate_resolution = gcd_float(from_resolution, to_resolution) # find the greatest common divisor / denominator of the two integers 
  if intermediate_resolution != from_resolution:

    resize_factor = from_resolution/intermediate_resolution
    intermediate_height = int(img.shape[0]*resize_factor)
    intermediate_width = int(img.shape[1]*resize_factor)
    
    intermediate_img = np.zeros((intermediate_height, intermediate_width), img.dtype)
    ss = int(from_resolution/intermediate_resolution)

    for sv in range(ss):
      for sh in range(ss):
        intermediate_img[sv::ss,sh::ss] = img

    from_resolution = intermediate_resolution
    img = intermediate_img
  
  if from_resolution == to_resolution:
    return img
  else:
    ss = int(to_resolution/from_resolution)
    deleted_rows = img.shape[0]%ss
    deleted_columns = img.shape[1]%ss
    if deleted_rows or deleted_columns: # this should never happen if the downloaded image subsets are properly sized
      print('Rows and columns deleted during resizing:', deleted_rows, deleted_columns)
      img = img[:(img.shape[0]-deleted_rows),:(img.shape[0]-deleted_columns)]

    resized_img = img[0::ss,0::ss]
    for sv in range(ss):
      for sh in range(ss):
        if sv == 0 and sh == 0:
          continue
        resized_img[resized_img != img[sv::ss,sh::ss]] = 0 # False # if other subpixels don't all have the same value as the first subpixels (e.g., the fisrt (upper-left) subpixels of 9 subpixels)

    return resized_img

def resize_avg(img, from_resolution, to_resolution): # here! pixels containing subpixels with a value of zero should be zero!
  intermediate_resolution = gcd_float(from_resolution, to_resolution) # find the greatest common divisor / denominator of the two integers 
  if intermediate_resolution != from_resolution:
    resize_factor = from_resolution/intermediate_resolution
    intermediate_height = int(img.shape[0]*resize_factor)
    intermediate_width = int(img.shape[1]*resize_factor)
    
    intermediate_img = np.zeros((intermediate_height, intermediate_width), img.dtype)
    ss = int(from_resolution/intermediate_resolution)
    print('ss',ss)
    for sv in range(ss):
      for sh in range(ss):
        intermediate_img[sv::ss,sh::ss] = img

    from_resolution = intermediate_resolution
    img = intermediate_img
  
  if from_resolution == to_resolution:
    return img
  else:
    ss = int(to_resolution/from_resolution)
    deleted_rows = img.shape[0]%ss
    deleted_columns = img.shape[1]%ss
    if deleted_rows or deleted_columns: # this should never happen if the downloaded image subsets are properly sized
      print('Rows and columns deleted during resizing:', deleted_rows, deleted_columns)
      img = img[:(img.shape[0]-deleted_rows),:(img.shape[0]-deleted_columns)]

    # Reshape the image and create the boolean mask
    reshaped_img = img.reshape((img.shape[0] // ss, ss, img.shape[1] // ss, ss))

    # Sum the values within each sub-sampling block
    resized_img = reshaped_img.sum(axis=(1, 3))

    # Detect zero values
    where_zero = (reshaped_img == 0).any(axis=(1, 3))

    # Normalize the values by the number of pixels summed
    resized_img = resized_img / (ss * ss)

    # Set locations with any zero values to zero
    resized_img[where_zero] = 0
  
    return resized_img if np.issubdtype(img.dtype, np.floating) else np.round(resized_img).astype(img.dtype)

def upscale_nx(img,n=2,shift_y=0,shift_x=0): # shift_y and shift_x is in PIXELS in this case
  upscaled_height = img.shape[0]*n
  upscaled_width = img.shape[1]*n

  if n != 1:
    upscaled_img = np.zeros((upscaled_height, upscaled_width), img.dtype) 
    for sv in range(n):
      for sh in range(n):
        upscaled_img[sv::n,sh::n] = img
    img = upscaled_img

  if shift_y != 0 or shift_x != 0:
    img = np.pad(img, ((max_shift, max_shift), (max_shift, max_shift)), mode='constant', constant_values=0) # add some zero-valued pixels on each side of the image to prevent rollover when doing np.roll
    img = np.roll(img, (shift_y, shift_x), axis=(0, 1))
    img = img[max_shift:max_shift+upscaled_height, max_shift:max_shift+upscaled_width]  # Crop back to original shape
  
  return img

def shift_and_resize(img,from_resolution = 30.0, to_resolution = 15.0, shift_y=-7.5, shift_x=7.5, method = 1): #  # shift_y and shift_x is in METERS in this case
  if shift_y != 0 or shift_x != 0:
    shift = min(abs(shift_y),abs(shift_x)) if shift_y != 0 and shift_x != 0 else max(abs(shift_y),abs(shift_x)) # robust?
    scaling_factor = from_resolution/shift
    intermediate_resolution = gcd_float(from_resolution,shift)
    scaling_factor = int(from_resolution/intermediate_resolution) 

    shift_y = int(shift_y/intermediate_resolution) # *-1 because original shift_y is northing and converted shift_y is image row
    shift_x = int(shift_x/intermediate_resolution)
    img = upscale_nx(img, scaling_factor, shift_y, shift_x)
    from_resolution = intermediate_resolution

  if from_resolution == to_resolution:
    return img
  elif method == 0:
    return resize_mixed2zero(img, from_resolution, to_resolution)
  elif method == 1:
    return zoom(img, from_resolution/to_resolution, order=0) # nearest neighbour # IMPORTANT! nearest neighbour will shift images if the misalignment is 0.5 pixels, resulting in a positional error of 0.5 pixels!! Propably should not use it to resample 30m Landsat to 15m pan pixel grid for this reason.
  else:
    return resize_avg(img, from_resolution, to_resolution)

## Functions for downloading images from GEE

In [ ]:
# export Google Earth image as a numpy array: https://developers.google.com/earth-engine/apidocs/ee-image-getdownloadurl#colab-python
crs_global = ''
transform_global = ''

def resample_dem_10m(img):
  return img.resample('bicubic').reproject(S2_projection, crsTransform = transform['S2_10m']).clip(S2_tile['S2'])
def resample_dem_20m(img):
  return img.resample('bicubic').reproject(S2_projection, crsTransform = transform['S2']).clip(S2_tile['S2'])

transform['DEM120'] = transform['S2'].copy()
transform['DEM120'][0] = 120
transform['DEM120'][4] = -120
def resample_dem_120m(img):
  return img.resample('bicubic').reproject(S2_projection, crsTransform = transform['DEM120']).clip(S2_tile['S2'])

def gee2np_1band_1tile(gee_image,band_name,resolution, tile_region = S2_tile): 
  gee2np_tile = ''
  for attempt in range(50):
    # getDownloadUrl can export maximum 48 MB (doesn't matter NPY or GEO_TIFF or ZIPPED_GEO_TIFF), e.g. 8000 x 6000 pixels (48 MP) at uint8 or half as many (24 MP) at 16bit
    url = gee_image.getDownloadUrl({'bands': band_name, 'region': tile_region, 'scale': resolution, 'format': 'NPY'}) # https://developers.google.com/earth-engine/apidocs/ee-image-getdownloadurl # , 'crs': crs_global, 'crs_transform': transform_global 
    response = requests.get(url)
    try:
      gee2np_tile = np.load(io.BytesIO(response.content)) # io.BytesIO() writes data to an in memory buffer
    except:
      print('pickles!')
      gee2np_tile = ''
    if isinstance(gee2np_tile, np.ndarray):
      break
  return gee2np_tile

def gee2np_1band(gee_image,band_name,subsensor,n_tiles=4): 
  tiles_np = []
  tiles_list = []

  with parallel_backend('threading'): # parallel downloading of all subtiles
    tiles_list.extend(Parallel()(delayed(gee2np_1band_1tile)(gee_image,band_name,resolution[subsensor],t) for t in tiles[subsensor][n_tiles]))

  for q in range(n_tiles):
    tiles_np.append(np.array(tiles_list[q]))

  np2d = np.zeros((h[subsensor], w[subsensor]), tiles_np[0].dtype)

  n_tiles_sqrt = int(math.sqrt(n_tiles))
  tile_h_px = round(h[subsensor]/n_tiles_sqrt)
  tile_w_px = round(w[subsensor]/n_tiles_sqrt)
  print('band',band_name,'downloaded')
  
  t = 0
  for qv in range(n_tiles_sqrt):
    for qh in range(n_tiles_sqrt):
      np2d[h[subsensor]-(qv+1)*tile_h_px:h[subsensor]-qv*tile_h_px,qh*tile_w_px:(qh+1)*tile_w_px] = tiles_np[t]
      t = t+1

  return np2d

def gee2np(gee_image,band_names,subsensor,filename='',n_tiles = 1):
  image_np = ''
  if resolution[subsensor] < 30 and n_tiles == 1: # here! this is not robust
    n_tiles = 4
    print('n_tiles automatically increased to enable high-spatial resolution data export')

  if isinstance(band_names, list) and len(band_names) > 1: # if the image has more than one band
    image_list = []
    with parallel_backend('threading'): # parallel downloading of all bands
      image_list.extend(Parallel()(delayed(gee2np_1band)(gee_image,b,subsensor,n_tiles) for b in band_names)) 

    image_np = np.asarray(image_list, dtype=image_list[0][0][0][0].dtype)
  else:
    image_list = gee2np_1band(gee_image,band_names,subsensor,n_tiles)
    image_np = np.asarray(image_list, dtype=image_list[0][0][0].dtype) # this assumes that all bands have the same datatype

  print('image_np.shape',image_np.shape)  
    
  if filename: # delete. This is legacy code.
    if save_np:
      np.save(MGRS_tile+'/'+filename,image_np) # don't save entire images, save only non-masked portion
    if save_tif:
      os.path.abspath(os.path.join(os.path.dirname(MGRS_tile+'/'+filename), '..', 'templates'))
      
      meta = metadata[subsensor].copy()
      meta['dtype'] = image_np.dtype

      if isinstance(band_names, list) and len(band_names) > 1: # ugly
        meta['count'] = image_np.shape[0]

        with rasterio.open(MGRS_tile+'_tif/'+filename+'.tif', 'w', **meta) as dst:
          dst.write(image_np)
      else:
        meta['count'] = 1
        with rasterio.open(MGRS_tile+'_tif/'+filename+'.tif', 'w', **meta) as dst:
          dst.write(image_np, 1)    

  return image_np

## Determine the greenup and greendown day-of-year using the MODIS Land Cover Dynamics (MCD12Q2) Product 

In [ ]:
modis_year = 2001
# if start_year > 2001:
#     modis_year = start_year # growing seasons gets longer due to global warming
# consider using MODIS Snow Cover product (MOD10CM V6) in addition or instead of MCD12Q2, though it's just based on NDSI. Already using NDSI when masking SR Sentinel-2.

print(modis_year)
modis = ee.ImageCollection('MODIS/061/MCD12Q2').filter(ee.Filter.date(str(modis_year)+'-01-01', str(modis_year+1)+'-01-01')).first() # There is one MCD12Q2 image per year (global mosaic). For more information, see "User Guide to Collection 6 MODIS Land Cover Dynamics (MCD12Q2) Product"

date1 = datetime(1970, 1, 1) # MCD12Q2 pixels values are number of days since Jan 1, 1970.
date2 = datetime(modis_year, 1, 1)
n_days_bn1970and_modis_year = (date2 - date1).days

if start_DOY == 0:
    Greenup = modis.select('MidGreenup_1').reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': S2_tile['S2'],
        'scale': 500,
        'bestEffort': True,
        'maxPixels': 1e7
        }).get('MidGreenup_1').getInfo() - n_days_bn1970and_modis_year # this gives day of the year for greenup (e.g., 182 ~ June 2) - here! not sure if averaging over the entire tile is a good idea, especially in mountainous areas
    Greenup = round(Greenup)
    print('Greenup',Greenup)
    start_DOY = Greenup
    print('Greenup date will be used as start_DOY')

if end_DOY == 0:
    Greendown = modis.select('MidGreendown_1').reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': S2_tile['S2'],
        'scale': 500,
        'bestEffort': True,
        'maxPixels': 1e7
        }).get('MidGreendown_1').getInfo() - n_days_bn1970and_modis_year
    Greendown = round(Greendown)
    print('Greendown',Greendown)

    end_DOY = Greendown
    print('Greendown date will be used as end_DOY')

if start_DOY_reference == 0:
    Maturity = modis.select('Maturity_1').reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': S2_tile['S2'],
        'scale': 500,
        'bestEffort': True,
        'maxPixels': 1e7
        }).get('Maturity_1').getInfo() - n_days_bn1970and_modis_year
    Maturity = round(Maturity)
    print('Maturity',Maturity)

    start_DOY_reference = Maturity
    print('Maturity date will be used as start_DOY_reference')

if end_DOY_reference == 0:
    Senescence = modis.select('Senescence_1').reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': S2_tile['S2'],
        'scale': 500,
        'bestEffort': True,
        'maxPixels': 1e7
        }).get('Senescence_1').getInfo() - n_days_bn1970and_modis_year
    Senescence = round(Senescence)
    print('Senescence',Senescence)
    end_DOY_reference = Senescence
    print('Senescence date will be used as end_DOY_reference')

if mid_summer_only:
    start_DOY = start_DOY_reference
    end_DOY = end_DOY_reference

# Function to convert MidGreenup_1 from days since Jan 1, 1970, to days since Jan 1 of the year of the image.
def convert_midgreenup_and_midgreendown(image):
    # Get the year of the image.
    year = ee.Date(image.get('system:time_start')).get('year').subtract(1970)
    # Calculate the number of days since Jan 1 of the year of the image.
    MidGreenup_1 = image.select('MidGreenup_1').subtract(year.multiply(365.25)).toUint16().rename("start_day") # .toUint16()
    MidGreendown_1 = image.select('MidGreendown_1').subtract(year.multiply(365.25)).toUint16().rename("end_day")
    return MidGreenup_1.addBands(MidGreendown_1)

# Function to convert Maturity_1 from days since Jan 1, 1970, to days since Jan 1 of the year of the image.
def convert_maturity_and_senescence(image): # not sure if need a separate function because GEE only performs operations required for the final results - not true. 7.6 seconds vs 5 seconds
    # Get the year of the image.
    year = ee.Date(image.get('system:time_start')).get('year').subtract(1970)
    # Calculate the number of days since Jan 1 of the year of the image.
    Maturity_1 = image.select('Maturity_1').subtract(year.multiply(365.25)).toUint16().rename("start_day")
    Senescence_1 = image.select('Senescence_1').subtract(year.multiply(365.25)).toUint16().rename("end_day")
    return Maturity_1.addBands(Senescence_1)

np.save(MGRS_tile+'/phenology_DOY', np.array([start_DOY,start_DOY_reference,end_DOY_reference,end_DOY],np.uint16))

### DEMO: The pixel grids of different sensors do not align

In [ ]:
# Landsat pixel coordinated = coordinates of pixel centres. Sentinel-2 pixel coordinates = coordinates of the NW corners of pixels.
# Landsat 30m does not align with S2
# Landsat 30m does not align with Landsat 30m pan
# MSS does not align with 20m S2, 30m Landsat, and 15m Landsat pan
# 10m S2 aligns perfectly with 20m S2

show_demo = False
if show_demo:
    # Display images using folium https://colab.research.google.com/github/giswqs/qgis-earthengine-examples/blob/master/Folium/ee-api-folium-setup.ipynb
    import folium
    def add_ee_layer(self, ee_object, vis_params, name):
        try:
            if isinstance(ee_object, ee.Image):
                map_id_dict = ee_object.getMapId(vis_params)
            elif isinstance(ee_object, ee.ImageCollection):
                ee_object_new = ee_object.mosaic()
                map_id_dict = ee_object_new.getMapId(vis_params)
            elif isinstance(ee_object, ee.Geometry) or isinstance(ee_object, ee.FeatureCollection):
                map_id_dict = ee.Image().paint(ee_object, 0, 2).getMapId(vis_params)
            elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
                ee_object_new = ee.Image().paint(ee_object, 0, 2)
                map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
                folium.raster_layers.TileLayer(
                tiles = map_id_dict['tile_fetcher'].url_format,
                attr = 'Google Earth Engine',
                name = name,
                overlay = True,
                control = True
            ).add_to(self)
            else:
                raise Exception("Unsupported EE object type: {}".format(type(ee_object)))

            folium.TileLayer(
                tiles=map_id_dict['tile_fetcher'].url_format,
                attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
                name=name,
                overlay=True,
                control=True
            ).add_to(self)
        except Exception as e:
            print(f"Could not display {name}: {e}")

    # Add Earth Engine drawing method to folium.
    folium.Map.add_ee_layer = add_ee_layer

    L9_best = ee.ImageCollection('LANDSAT/LC08/C02/T1').filterBounds(test_tile).filter(ee.Filter.calendarRange(2020, 2024,'year')).filter(ee.Filter.calendarRange(start_DOY_reference,end_DOY_reference,'day_of_year')).filter(ee.Filter.lt('CLOUD_COVER', 30)).sort('CLOUD_COVER').first()
    L9_pan = L9_best.select('B8').clip(S2_tile['L_pan'])
    L9_nir = L9_best.select('B3').clip(S2_tile['L'])

    lc_vis_params = {
        'min': 1,'max': 15000
    }

    S2_tile_Centroid = S2_tile['S2'].centroid(**{'maxError': 1}).coordinates().getInfo()
    my_map = folium.Map(location=[S2_tile_Centroid[1],S2_tile_Centroid[0]], zoom_start=10) # folium.Map(location=[39.712183, -104.998424], zoom_start=5)

    my_map.add_ee_layer(ee.Geometry(S2_tile['S2']), {}, 'S2_tile')

    my_map.add_ee_layer(L9_pan, lc_vis_params, 'L9_pan')
    my_map.add_ee_layer(L9_nir, {'min': 1,'max': 20000}, 'L9_green')

    S2_best = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filter(ee.Filter.calendarRange(2020,2024,'year')).filter(ee.Filter.calendarRange(start_DOY_reference,end_DOY_reference,'day_of_year')).filterMetadata('MGRS_TILE', 'equals', MGRS_tile).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)).filter(ee.Filter.lt('NODATA_PIXEL_PERCENTAGE', 20)).sort('NODATA_PIXEL_PERCENTAGE').first().select(['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12','QA60'],['B','G','R','RE1','RE2','RE3','NIR_wide','NIR', 'SWIR1', 'SWIR2','QA60'])

    s2_green = ee.Image(S2_best.select('R').clip(S2_tile['S2'])) # greed # NIRwide is B8
    my_map.add_ee_layer(s2_green, {'min': 1,'max': 5000}, 's2_red')

    s2_re1 = ee.Image(S2_best.select('RE1').clip(S2_tile['S2'])) # greed # NIRwide is B8
    my_map.add_ee_layer(s2_re1, {'min': 1,'max': 5000}, 's2_re1')

    MSS_best = ee.ImageCollection('LANDSAT/LM05/C02/T2').filterBounds(test_tile).filter(ee.Filter.calendarRange(1984, 1985,'year')).filter(ee.Filter.calendarRange(start_DOY_reference,end_DOY_reference,'day_of_year')).filter(ee.Filter.lt('CLOUD_COVER', 30)).sort('CLOUD_COVER').first().select(['B1', 'B2', 'B3', 'B4','QA_PIXEL','QA_RADSAT'],['G', 'R', 'NIR1','NIR2','QA_PIXEL','QA_RADSAT'])
    MSS_green = ee.Image(MSS_best.select('G').clip(S2_tile['M'])) # greed # NIRwide is B8
    my_map.add_ee_layer(MSS_green, {'min': 1,'max': 150}, 'MSS_green')

    # c2c = ee.Image('projects/ee-ilya85parshakov/assets/C2C2020_harvest').clip(S2_tile['L'])
    # my_map.add_ee_layer(c2c, {'min': 1,'max': 150}, 'c2c_temp')

    fc = ee.FeatureCollection(S2_tile['S2']).style(**{'color': '#000000', 'width': 1, 'fillColor': '#00000000'})
    my_map.add_ee_layer(fc, {}, "S2_tile") 

    my_map.add_child(folium.LayerControl())
    display(my_map)

## Get agriculture and water mask

In [ ]:
# "The WorldCover 2020 v100 product reaches an overall accuracy of 74.4%. For more details please see the Product Validation Report V1.0.""
# "The WorldCover 2021 v200 reaches an overall accuracy of 76.7%. For more details please see the Product Validation Report V2.0."

# ESA WorldCover (global)
WorldCover10m_water = ee.ImageCollection("ESA/WorldCover/v200").first()
WorldCover10m_water = WorldCover10m_water.neq(80).And(WorldCover10m_water.neq(40)) if mask_agriculture else WorldCover10m_water.neq(80)
water_and_agri_mask = WorldCover10m_water.updateMask(WorldCover10m_water).rename("water_and_agri_mask") # https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100 # here! change variable name

if mask_agriculture and use_AAFC:
    # Create a global constant 200 image (forest) for areas outside the AAFC map coverage
    global_fill = ee.Image.constant(200).rename('class')

    # AAFC image for Canada
    AAFC_2022 = ee.ImageCollection('AAFC/ACI') \
        .filter(ee.Filter.calendarRange(AAFC_year, AAFC_year, 'year')) \
        .first() \
        .unmask(200)

    # Blend AAFC on top of global fill: inside Canada = AAFC value, outside = 200
    AAFC_2022 = global_fill.blend(AAFC_2022)

    # List of non-agricultural class values to keep (i.e., to mask out cropland)
    non_agri_classes = ee.List([
        10,   # Cloud
        20,   # Water
        30,   # Exposed Land and Barren
        34,   # Urban and Developed
        35,   # Greenhouses
        50,   # Shrubland
        80,   # Wetland  
        85,   # Peatland
        110,  # Grassland
        200,  # Forest (undifferentiated)
        210,  # Coniferous
        220,  # Broadleaf
        230   # Mixedwood
    ])

    # Build mask: 1 = non-agri, 0 = cropland
    AAFC_noagri = ee.ImageCollection(
        non_agri_classes.map(lambda v: AAFC_2022.eq(ee.Number(v)))
    ).max().rename('class') # if pixel value is one of the good classes (non_agri_classes), return 1.

    AAFC_noagri = AAFC_noagri.neq(0).rename("AAFC_mask")

    # Combine ESA and AAFC → a pixel is non-agri if both agree it’s non-agri
    water_and_agri_mask = water_and_agri_mask.And(AAFC_noagri).rename("water_and_agri_mask")

# ==== download AAFC + water masks as numpy arrays or load them if they have already been downloaded - this is needed for S2 cloud masking on PC (instead of GEE)
water_and_agri_mask_np = {}
water_and_agri_mask_n_pixels = {}

for s in subsensors:
  mask_filename = 'water_and_agri_mask_'+s if mask_agriculture else 'water_mask_'+s
  if os.path.exists(MGRS_tile+'/'+mask_filename+'.npy') and skip_existing_agri:
    water_and_agri_mask_np[s] = np.load(MGRS_tile+'/'+mask_filename+'.npy') 
  else: 
    water_and_agri_mask_np[s] = gee2np(water_and_agri_mask.reproject(crs=S2_projection, crsTransform = transform[s]),'water_and_agri_mask',s,mask_filename,16)
  water_and_agri_mask_n_pixels[s] = np.count_nonzero(water_and_agri_mask_np[s]) 

n_not_water_max_client = np.count_nonzero(water_and_agri_mask_np['S2_10m']) # here! calculate for each sensor!
print("percent non-agri land pixels in the scene: ", 100.0* n_not_water_max_client/n_pixels['S2_10m'])

# number of non-water and non-agri pixels at reducer resolution (default: 120m)
n_not_water_max_client = n_not_water_max_client*(10.0/reducer_resolution_client)*(10.0/reducer_resolution_client)
print('n_not_water_max_client',n_not_water_max_client)
n_not_water_max = ee.Number(n_not_water_max_client)

## Other maps (optional)

In [ ]:
# download_other_maps = True
if download_other_maps:
    MODIS_phenology_collection = ee.ImageCollection('MODIS/061/MCD12Q2').map(convert_maturity_and_senescence) if mid_summer_only else ee.ImageCollection('MODIS/061/MCD12Q2').map(convert_midgreenup_and_midgreendown) # convert days since Jan 1, 1970 to days since Jan 1 of the year
    # Composite the images to get the median value for all years of MODIS.
    MODIS_phenology = MODIS_phenology_collection.median().clip(S2_tile['M']) #.toUint16() # do not use mosaic()
    MODIS_phenology = MODIS_phenology.addBands(MODIS_phenology.select("end_day").unmask(365), overwrite=True)
    MODIS_phenology = MODIS_phenology.reproject(crs=MODIS_phenology.projection(), scale=1000).focal_median(radius=3, units='pixels') # apply a low-pass filter to remove non-clustered no-data pixels
    MODIS_phenology_np = gee2np(MODIS_phenology.reproject(crs=S2_projection, crsTransform = transform['M']),['start_day', 'end_day'],"M",'',16)
    if mid_summer_only:
        np2tif(MODIS_phenology_np, 'MODIS_MCD12Q2_phenology_composite', 'M', ['Maturity_1','Senescence_1'],{'Maturity_1':0.55,'Senescence_1':0.65})
    else:
        np2tif(MODIS_phenology_np, 'MODIS_MCD12Q2_phenology_composite', 'M', ['MidGreenup_1','MidGreendown_1'],{'MidGreenup_1':0.55,'MidGreendown_1':0.65})   
    del MODIS_phenology_np 

    canopy_height_nico_lang = ee.Image('users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1').reproject(crs=S2_projection, crsTransform = transform['S2_10m']).clip(S2_tile['S2_10m']).rename("lang_canopy_height")
    canopy_height_nico_lang = gee2np(canopy_height_nico_lang,'lang_canopy_height','S2_10m','',16) 
    np2tif(canopy_height_nico_lang, 'canopy_height_nico_lang_2020_10m', 'S2_10m', ['lang_canopy_height'],{'lang_canopy_height':0.5})
    del canopy_height_nico_lang
    
    # https://glad.umd.edu/dataset/gedi/ # this is a collection containing 7 maps, one for each geographical region. This code does not mosaic
    potapov_canopy_height = ee.ImageCollection('users/potapovpeter/GEDI_V27').filterBounds(S2_tile['L']).first().reproject(crs=S2_projection, crsTransform = transform['L']).clip(S2_tile['L']).rename("potapov") # Global Forest Canopy Height, 2019; 
    potapov_canopy_height = gee2np(potapov_canopy_height,'potapov','L')
    np2tif(potapov_canopy_height, 'canopy_height_potapov_2019_30m', 'L', ['potapov_canopy_height'],{'potapov_canopy_height':0.5}) # Pixel values: 0-60 Forest canopy height, meters; 101 Water; 102 Snow/ice; 103 No data
    del potapov_canopy_height

    # 1m resolution canopy height map for 2018 - 2020 (note: resampling to 10m may produce low canopy height values compared to the other maps): https://gee-community-catalog.org/projects/meta_trees/ 
    canopyHeight1m_Tolan = ee.ImageCollection('projects/sat-io/open-datasets/facebook/meta-canopy-height').filterBounds(S2_tile['S2']).mosaic().reproject(crs=S2_projection, crsTransform = transform['S2_10m']).clip(S2_tile['S2_10m']).rename("canopy_height_talon") # previously 'projects/meta-forest-monitoring-okw37/assets/CanopyHeight'
    canopyHeight1m_Tolan = gee2np(canopyHeight1m_Tolan,'canopy_height_talon','S2_10m','',16)
    np2tif(canopyHeight1m_Tolan, 'canopy_height_talon_2018to2020_1m_rs10m', 'S2_10m', ['talon_canopy_height'],{'talon_canopy_height':0.5})

    change_map_grid = 'L' # 'L_pan'

    # potapov's global forest change map for the years 2000 - 2022
    gfc = ee.Image('UMD/hansen/global_forest_change_2024_v1_12').select('lossyear').reproject(crs=S2_projection, crsTransform = transform[change_map_grid]).clip(S2_tile[change_map_grid]).rename("gfc")
    gfc = gee2np(gfc,'gfc',change_map_grid,'')
    np2tif(gfc, 'gfc_2001to2024', change_map_grid, ['gfc'],{'gfc':0.5})
    del gfc

    # C2C forest change map for the years 1984 - 2020 (Canada only)
    c2c = ee.Image('projects/ee-ilya85parshakov/assets/C2C2020_harvest').reproject(crs=S2_projection, crsTransform = transform[change_map_grid]).clip(S2_tile[change_map_grid]).rename("c2c")
    c2c = gee2np(c2c,'c2c',change_map_grid)
    np2tif(c2c, 'c2c_1985to2020_harvest', change_map_grid, ['c2c_2020_harvest'],{'c2c_2020_harvest':0.5})
    del c2c

    c2c = ee.Image('projects/ee-ilya85parshakov/assets/C2C2020_fire').reproject(crs=S2_projection, crsTransform = transform[change_map_grid]).clip(S2_tile[change_map_grid]).rename("c2c")
    c2c = gee2np(c2c,'c2c',change_map_grid)
    np2tif(c2c, 'c2c_1985to2020_fire', change_map_grid, ['c2c_2020_fire'],{'c2c_2020_fire':0.5})
    del c2c

    ccdc_noel_prob = ee.ImageCollection("GOOGLE/GLOBAL_CCDC/V1").select('changeProb').filterBounds(S2_tile[change_map_grid]).mosaic().clip(S2_tile[change_map_grid]).arrayGet([0]) # .mosaic() same results as first(), except no mosaicing
    ccdc_noel = ee.ImageCollection("GOOGLE/GLOBAL_CCDC/V1").select('tBreak').filterBounds(S2_tile[change_map_grid]).mosaic().clip(S2_tile[change_map_grid]).arrayGet([0]) # .arrayGet([0]) same as .arrayGet(ee.Image([0]))
    ccdc_noel = ccdc_noel.updateMask(ccdc_noel_prob.gt(0.75)).reproject(crs=S2_projection, crsTransform = transform[change_map_grid]).rename("ccdc") #.first() produces same results??? .updateMask(ccdc_noel)
    ccdc = gee2np(ccdc_noel,'ccdc',change_map_grid,'',16)
    np2tif(ccdc, 'ccdc_2000to2019_Gorelick', change_map_grid, ['ccdc'],{'ccdc':0.5})
    del ccdc

    palsar = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH').filter(ee.Filter.date('2020-01-01', '2021-01-01')).first().reproject(crs=S2_projection, crsTransform = transform['S2']).clip(S2_tile['S2'])    
    
    sarHvHh = palsar.select('HV').divide(palsar.select('HH')).multiply(1000.0).toInt16().rename("HVHH")
    sarHvHh = gee2np(sarHvHh,'HVHH','S2')
    np2tif(sarHvHh, 'palsar_HvHh', 'S2', ['palsar_HvHh'],{'palsar_HvHh':0.5})

    sarHh = gee2np(palsar,'HH','S2')
    np2tif(sarHh, 'palsar_Hh', 'S2', ['palsar_Hh'],{'palsar_Hh':0.5})

    sarHv = gee2np(palsar,'HV','S2')
    np2tif(sarHv, 'palsar_Hv', 'S2', ['palsar_Hv'],{'palsar_Hv':0.5})
    del sarHvHh, sarHh, sarHv

    UCChange_old = None
    if UTM_zone == 9:
        UCChange_old = ee.ImageCollection('users/parshakov/UCBTS_BC_UTMz09_v2020-04-28')
    elif UTM_zone == 10:
        UCChange_old = ee.ImageCollection('users/parshakov/UCBTS_BC_2020-04-28')
    elif UTM_zone == 11:
        UCChange_old = ee.ImageCollection('users/parshakov/UCBTS_BC_UTMz11_v2020-04-28')

    if UCChange_old != None:
        year = 1985

        # Function to rename the band to the corresponding year
        def rename_band(image):
            year = image.date().get('year').format()
            return image.rename(year)

        # Map the rename function over the collection
        renamed_collection = UCChange_old.map(rename_band)

        # Create a list of images to stack
        image_list = renamed_collection.toList(renamed_collection.size())

        # Function to stack images
        def stack_images(image, previous):
            # Cast the previous value to an image
            previous = ee.Image(previous)
            # Combine the previous image with the current image
            stacked = previous.addBands(image)
            return stacked

        # Initialize with the first image
        initial = ee.Image(image_list.get(0))

        # Iterate over the collection to stack all images
        stacked_image = ee.Image(image_list.slice(1).iterate(stack_images, initial)).reproject(crs=S2_projection, crsTransform = transform['L']).clip(S2_tile['L']) # here! transform might be different

        band_names = stacked_image.bandNames().getInfo()
        print(band_names)

        fake_waves = {}
        for y in band_names:
            fake_waves[y] = int(y)

        UCChange_old = gee2np(stacked_image,band_names,'L')
        np2tif(UCChange_old, "UC-Change_old_"+MGRS_tile, 'L', band_names, fake_waves) # download a stack of yearly UC-Change maps
        del UCChange_old

## Get slope and aspect information for topographic correction from a DEM

In [ ]:
dem_elevation = ee.Image('NASA/NASADEM_HGT/001').select('elevation') # NASADEM_HGT is 30m - was the default before 2024-07-20 # consider using 'NRCan/CDEM' for Canada (resolution 0.75 arc-seconds = ~23m) or 'COPERNICUS/DEM/GLO30' (this one is a pure DSM and the collection consists of multiple images that need to be mosaiced together, which may result in GEE crashing)

# added 2025-06-21
use_ESADEM_instead_of_NASADEM = False # Here! ESADEM is reprojected onto the Landsat pixel grid! Not ideal for S2 and MSS. Got to reproject again from original when downloading S2 and MSS.
if use_ESADEM_instead_of_NASADEM:
    def resample_dem_30m(img):
        return img.resample('bicubic').reproject(S2_projection, crsTransform = transform['L']).clip(S2_tile['L'])
    dem_elevation = ee.ImageCollection('COPERNICUS/DEM/GLO30').filterBounds(S2_tile['GEE']).select('DEM').map(resample_dem_30m).mosaic().reproject(S2_projection, crsTransform = transform['L']).uint16()

degree2rad = ee.Number(3.14159265359/180.0)

slope_noreproj = ee.Terrain.slope(dem_elevation).multiply(degree2rad)
aspect_noreproj = ee.Terrain.aspect(dem_elevation).multiply(degree2rad)
cos_slope_noreproj = ee.Image(slope_noreproj.cos()).rename('cos_slope')
sin_slope_noreproj = ee.Image(slope_noreproj.sin()).rename('sin_slope')

if download_other_maps:
    gee2np(dem_elevation.reproject(crs=S2_projection, crsTransform = transform['L']).clip(S2_tile['L']),'elevation','L','GLO30_DEM_L' if use_ESADEM_instead_of_NASADEM else 'NASA_DEM_HGT') # 

## GEE functions for 30m Landsat data preprocessing

In [ ]:
collection_names = {}

collection_names_raw = {4: 'LANDSAT/LT04/C02/T1',
                        5: 'LANDSAT/LT05/C02/T1',
                        7: 'LANDSAT/LE07/C02/T1',
                        8: 'LANDSAT/LC08/C02/T1',
                        9: 'LANDSAT/LC09/C02/T1'}

collection_names_SR = {4: 'LANDSAT/LT04/C02/T1_L2',
                       5: 'LANDSAT/LT05/C02/T1_L2',
                       7: 'LANDSAT/LE07/C02/T1_L2',
                       8: 'LANDSAT/LC08/C02/T1_L2',
                       9: 'LANDSAT/LC09/C02/T1_L2'}

collection_names = collection_names_SR if sr else collection_names_raw

bands_original = bands['L'].copy()

band_names = {}
for s in [4,5,7]:
  band_names[s] = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7','QA_PIXEL','QA_RADSAT'] if sr else ['B1', 'B2', 'B3', 'B4', 'B5', 'B7','QA_PIXEL','QA_RADSAT']

for s in [8,9]:
  band_names[s] = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7','QA_PIXEL','QA_RADSAT'] if sr else ['B2',  'B3', 'B4', 'B5', 'B6', 'B7','QA_PIXEL','QA_RADSAT']

band_names_renamed = {}
for s in [4,5,7,8,9]:
  band_names_renamed[s] = ['B','G','R','NIR','SWIR1','SWIR2','QA_PIXEL','QA_RADSAT']

band_names_renamed_no_pan = band_names_renamed[9].copy()

if sr:
   download_L_pan = False
else:
  for s in [7,8,9]: # add panchromatic band for Landsat 7 - 9 in not SR
    band_names[s] = ['B8'] + band_names[s]
    band_names_renamed[s] = ['pan'] + band_names_renamed[s]

print(collection_names)
print(band_names)
print(band_names_renamed)

spacecraft_ID_number = {'LANDSAT_4': 4,'LANDSAT_5': 5,'LANDSAT_7': 7,'LANDSAT_8': 8,'LANDSAT_9': 9}
spacecraft_ID_short = {'LANDSAT_4': '_L4','LANDSAT_5': '_L5','LANDSAT_7': '_L7','LANDSAT_8': '_L8','LANDSAT_9': '_L9'}
spacecraft_ID_to_collection = {'LANDSAT_4': 'LANDSAT/LT04/C02/T1','LANDSAT_5': 'LANDSAT/LT05/C02/T1','LANDSAT_7': 'LANDSAT/LE07/C02/T1','LANDSAT_8': 'LANDSAT/LC08/C02/T1','LANDSAT_9': 'LANDSAT/LC09/C02/T1'} # surface reflectance data has no pan band

n_not_water_L = np.count_nonzero(water_and_agri_mask_np['L']) 
yearly_best_L = pd.DataFrame(columns = ['year','date_str','sensor','clearsky']) # e.g., sensor = '_L7'; clearsky is the percent of non-masked pixels

# cloudShadowBitMask = 1 << 4 # not 1,2,3,5,7,9,10 (snow); ok: 6 (some snow), 8 (some mountain shadow)
# cloudsBitMask = 1 << 8
# cirrusBitMask = 1 << 12 # L8 should have cirrus mask. This might not be required because cloudsBitMask = 1 << 8 likely already includes cirrus clouds

landsat_collection = ''
landsat_collection_gte1before = ''

def get_solar_info(img):
  zenith = ee.Number(90).subtract(ee.Number(img.get('SUN_ELEVATION'))).multiply(degree2rad)
  cosZ = zenith.cos() 
  return img.set('zenith',zenith).set('cosZ',cosZ)   

# convert user defined TOA smoke thresholds for Landsat 4 TM - 7 ETM+ sensors to raw DN values
def get_L5andL7_thresholds(img): # get wildfire smoke thresholds for raw L
  cosZ = ee.Number(img.get('cosZ'))
  smoke_thres_swir1_Lraw_DN = smoke_thres_swir1_Lraw.multiply(cosZ).subtract(ee.Number(img.get('REFLECTANCE_ADD_BAND_7'))).divide(ee.Number(img.get('REFLECTANCE_MULT_BAND_7')))
  smoke_thres_blue_Lraw_DN = smoke_thres_blue_Lraw.multiply(cosZ).subtract(ee.Number(img.get('REFLECTANCE_ADD_BAND_1'))).divide(ee.Number(img.get('REFLECTANCE_MULT_BAND_1')))
  return img.set('smoke_thres_swir1',smoke_thres_swir1_Lraw_DN).set('smoke_thres_blue',smoke_thres_blue_Lraw_DN) 

# convert user defined TOA smoke thresholds for Landsat 8 and newer to raw DN values
def get_L8_and_newer_thresholds(img): # get wildfire smoke thresholds for raw L
  cosZ = img.get('cosZ')
  smoke_thres_swir1_Lraw_DN = smoke_thres_swir1_Lraw.multiply(cosZ).subtract(ee.Number(img.get('REFLECTANCE_ADD_BAND_7'))).divide(ee.Number(img.get('REFLECTANCE_MULT_BAND_7')))
  b1_thres_Lraw_DN = smoke_thres_aerosol_Lraw.multiply(cosZ).subtract(ee.Number(img.get('REFLECTANCE_ADD_BAND_1'))).divide(ee.Number(img.get('REFLECTANCE_MULT_BAND_1')))
  return img.set('smoke_thres_swir1',smoke_thres_swir1_Lraw_DN).set('smoke_thres_blue',b1_thres_Lraw_DN) 

def smoke_raw(img):
  smoke_mask = img.select('SWIR1').lt(ee.Number(img.get('smoke_thres_swir1'))).And(img.select('B1').gt(ee.Number(img.get('smoke_thres_blue')))).unmask().eq(0).rename('smoke_mask') 
  return img.addBands(smoke_mask)

def smoke_sr(img):
  smoke_mask = img.select('SWIR1').lt(smoke_thres_swir1_L_sr).And(img.select('B1').gt(smoke_thres_blue_L_sr)).unmask().eq(0).rename('smoke_mask')
  return img.addBands(smoke_mask)

def smoke_sr_oli(img):
  smoke_mask = img.select('SWIR1').lt(smoke_thres_swir1_L_sr).And(img.select('B1').gt(smoke_thres_aerosol_L_sr)).unmask().eq(0).rename('smoke_mask')
  return img.addBands(smoke_mask)

# count images before/after to mark whether mosaic is needed
def mosaic_or_not(img):
  date_this = img.date()
  daily_collection_after = landsat_collection.filterDate(date_this, date_this.advance(1,'day'))
  daily_collection_before = landsat_collection.filterDate(date_this.advance(-1,'day'), date_this)
  return (img
          .set('n_before', daily_collection_before.size())
          .set('n_after', daily_collection_after.size())
          .set('date_str', date_this.format("YYYY-MM-dd"))
         )

props_to_copy = [
    'date_str', 'system:time_start', 'cosZ', 'zenith',
    'smoke_thres_blue', 'smoke_thres_swir1',
    'SUN_AZIMUTH', 'SPACECRAFT_ID', 'SCENE_CENTER_TIME'
]

# Mosaic two same-day same-path images that don't have a pan band
def mosaic_L_fast(img_wrapped):
    """Mosaic 2-image Landsat collection without pan band."""
    img = ee.Image(img_wrapped)
    second_img = ee.Image(img.get('second'))
    return ee.ImageCollection([img, second_img]).copyProperties(img, props_to_copy)

# Mosaic two same-day same-path images with pan band
def mosaic_L_pan_fast(img_wrapped):
    """Mosaic 2-image Landsat collection with pan band."""
    img = ee.Image(img_wrapped)
    second = ee.Image(img.get('second'))

    daily_collection = ee.ImageCollection([img, second])

    daily_mosaic30 = daily_collection.select(band_names_renamed_no_pan+['B1','T']).qualityMosaic('QA_PIXEL') # .reproject(crs=S2_projection, crsTransform = transform['L']) # here! would it be faster to reproject the two original images individually first?
    daily_mosaic = daily_collection.select('pan').mosaic() # .reproject(crs=S2_projection, crsTransform = transform['L_pan'])
    return daily_mosaic.addBands(daily_mosaic30).copyProperties(img, props_to_copy)

# create masks and count non-cloud pixels in Landsat images
def masker_L(img):
  BQA = img.select('QA_PIXEL') # BQA

  # Clouds = BQA.bitwiseAnd(8).unmask().eq(0)
  # Cloud_Shadows = BQA.bitwiseAnd(16).unmask().eq(0)
  # Snow_Ice = BQA.bitwiseAnd(32).unmask().eq(0) # here! check the quality of the snow mask and use NDSI if it's not good
  # Water = BQA.bitwiseAnd(64).unmask().eq(0)
  # Land_Water_Mask = BQA.bitwiseAnd(128).unmask().eq(0)
  # Cirrus_Clouds = BQA.bitwiseAnd(256).unmask().eq(0) # regular cloud mask (bit 8) already includes opaque clouds, probable clouds, and cirrus clouds, so this is not needed, but double check

  Snow_Ice = BQA.bitwiseAnd(32).unmask().eq(0) 

  cloud_mask = BQA.bitwiseAnd(int('11111', 2)).unmask().eq(0).updateMask(img.select('smoke_mask')).updateMask(Snow_Ice).rename('cloud_mask')

  # can we use .mask() here? .mask() = Get masks for all image bands. Valid pixels are value 1, invalid are 0.
  datapixels = img.select(bands[sensor]).unmask().gt(0).reduce('min') # not sure this is in the right order, but it works. Should be img.select(bands[sensor]).reduce('min').gt(0), but no. Unlike S2, there is no need to unmask() here for some reason
  saturationMask = img.select('QA_RADSAT').eq(0)
 
  mask = cloud_mask.updateMask(cloud_mask).updateMask(datapixels).updateMask(saturationMask).rename('mask') # .updateMask(phenologyStartMask).updateMask(phenologyEndMask)

  total_mask = mask.updateMask(water_and_agri_mask).rename('total_mask')

  n_clearsky = total_mask.reduceRegion(**{
      'reducer': ee.Reducer.count(),
      'geometry': S2_tile['L'],
      'crs': S2_projection,
      'scale': reducer_resolution,
      'bestEffort': True,
      'tileScale': 16,
      'maxPixels': 1e7
    }).get('total_mask')

  return img.addBands(cloud_mask).addBands(mask).set('clearsky', n_clearsky).set('date_str', img.get('date_str')) # raw Landsat 5 images also have attribute DATE_ACQUIRED "YYYY-MM-DD" (string)

# apply cloud, valid pixel, and saturation masks to Landsat 4 and 5 TM images - sensors that do not have panchromatic band
def apply_mask_L(img):
  mask = img.select('mask')
  cloud_mask = img.select('cloud_mask')  
  return img.select(bands[sensor]+['T']).updateMask(mask).addBands(cloud_mask)

# apply cloud, valid pixel, and saturation masks to Landsat 7 - 9 multispectral bands and only cloud mask to Landsat 7 - 9 panchromatic band
def apply_mask_L_pan(img):
  mask = img.select('mask') 
  cloud_mask = img.select('cloud_mask') 
  return img.select(bands[sensor]+['T']).updateMask(mask).addBands(img.select('pan').updateMask(cloud_mask)).addBands(cloud_mask)

 # this performs topographic correction on Landsat 4 - 7 images (8-bit sensors)
def illumination_L_8bit(img):
  img = img.float() # this is only required for raw L4 and L7 images, otherwise the output of topo correction will be 8bit (e.g., pixel value 11.88 becomes 11). SR data is already 16bit.
  azimuth = ee.Image.constant(ee.Number(img.get('SUN_AZIMUTH')).multiply(degree2rad)) #.clip(S2_tile) # degrees to radians
  zenith = ee.Number(img.get('zenith'))
  cosZ = ee.Number(img.get('cosZ'))
  cos_slope_x_cosZ = cos_slope_noreproj.multiply(cosZ)
  
  # illumination = cos(slope)*cos(zenith) + sin(slope)*sin(zenith)*cos(azimuth - aspect)
  illumination_azimuth = sin_slope_noreproj.multiply(zenith.sin()).multiply(azimuth.subtract(aspect_noreproj).cos()) # sin(slope)*sin(zenith)*cos(azimuth - aspect)
  illumination = cos_slope_x_cosZ.add(illumination_azimuth).rename('illumination')

  img_i = img.addBands(illumination).updateMask(img.select('R').gt(0)) # .gt(15000)

  illumination_gte0 = illumination.max(0.0) # clip cos(i) to a minimum of 0.0 because if illumination is negative, the denominator becomes unstable. 0.0 is 0% of the direct solar illumination compared to a flat surface under the sun (i.e., cos(i) = 1 when sun is directly overhead and terrain is flat).# added on 2025-06-21

  # https://mygeoblog.com/2018/10/17/terrain-correction-in-gee/
  for band in bands[sensor]:
    out = img_i.select('illumination', band).reduceRegion(**{
      'reducer': ee.Reducer.linearFit(), # Compute linear regression coefficients: a(slope), b(offset), c(b/a)
      'geometry': linearFitGeometry, 
      'scale': 120,
      'tileScale': 4,
      'maxPixels': 1000000000
    }) 

    out_a = ee.Number(out.get('scale'))
    out_b = ee.Number(out.get('offset'))
    out_c = out_b.divide(out_a)

    # Apply the SCSc correction
    SCSc_output = img.select(band).multiply(cos_slope_x_cosZ.add(out_c)).divide(illumination_gte0.add(out_c)).rename(band)
    
    img = img.addBands(**{
        'srcImg': SCSc_output.multiply(100.0), # multiply by 100.0 to preserve the precision of topo correction for 8-bit sensors. Otherwise will be converted to uint8
        'overwrite': True
      })
  return img.addBands(img_i.select('T')).uint16() # L9 raw DN is 16bit, L8 is 12bit, L5&7 are 8bit; surface reflectance is 16bit for all sensor

 # this performs topographic correction on Landsat 4 - 7 images (16-bit sensors)
def illumination_L_16bit(img): # this performs topographic correction
  azimuth = ee.Image.constant(ee.Number(img.get('SUN_AZIMUTH')).multiply(degree2rad)) # degrees to radians
  zenith = ee.Number(img.get('zenith'))
  cosZ = ee.Number(img.get('cosZ'))
  cos_slope_x_cosZ = cos_slope_noreproj.multiply(cosZ)
  
  # illumination = cos(slope)*cos(zenith) + sin(slope)*sin(zenith)*cos(azimuth - aspect)
  illumination_azimuth = sin_slope_noreproj.multiply(zenith.sin()).multiply(azimuth.subtract(aspect_noreproj).cos()) # sin(slope)*sin(zenith)*cos(azimuth - aspect)
  illumination = cos_slope_x_cosZ.add(illumination_azimuth).rename('illumination')

  img_i = img.addBands(illumination).updateMask(img.select('R').gt(0))

  illumination_gte0 = illumination.max(0.0) #.rename('illumination') # clip cos(i) to a minimum of 0.1 because if illumination is near zero or negative, the denominator becomes unstable. # added on 2025-06-21

  # https://mygeoblog.com/2018/10/17/terrain-correction-in-gee/
  for band in bands[sensor]:
    out = img_i.select('illumination', band).reduceRegion(**{
      'reducer': ee.Reducer.linearFit(), # Compute linear regression coefficients: a(slope), b(offset), c(b/a)
      'geometry': linearFitGeometry, # ee.Geometry(img.geometry().buffer(-5000)), # trim off the outer edges of the image for linear relationship 
      'scale': 120, # reduce resolution. This affects the speed but smaller topographic features might be lost at lower resolutions.
      'tileScale': 4, # this parameer only influences how much memory and parallelism the server uses to perform the operation.
      'maxPixels': 1000000000
    }) 

    out_a = ee.Number(out.get('scale'))
    out_b = ee.Number(out.get('offset'))
    out_c = out_b.divide(out_a)

    # Apply the SCSc correction
    # here! replace img with img_i if you want to apply cloud mask server side instead of building and applying it again on client side - this might make this slower
    SCSc_output = img.select(band).multiply(cos_slope_x_cosZ.add(out_c)).divide(illumination_gte0.add(out_c)).rename(band)
    
    img = img.addBands(**{
        'srcImg': SCSc_output,
        'overwrite': True
      })
  return img.addBands(img_i.select('T')).uint16() #.uint16() # here! L9 raw DN is 16bit, L8 is 12bit, L5&7 are 8bit; surface reflectance is 16bit for all sensor

## Connect & Login to NASA server to get access to HLS data

In [ ]:
# code copied from https://urs.earthdata.nasa.gov/documentation/for_users/data_access/python

# Create a password manager to deal with the 401 response that is returned from Earthdata Login
password_manager = request.HTTPPasswordMgrWithDefaultRealm()
password_manager.add_password(None, "https://urs.earthdata.nasa.gov", username_nasa, password_nasa)

# Create a cookie jar for storing cookies. This is used to store and return
# the session cookie given to use by the data server (otherwise it will just
# keep sending us back to Earthdata Login to authenticate). Ideally, we
# should use a file based cookie jar to preserve cookies between runs. This
# will make it much more efficient.
cookie_jar = cookiejar.CookieJar()
# Install all the handlers.
opener = request.build_opener(
    request.HTTPBasicAuthHandler(password_manager),
    request.HTTPCookieProcessor(cookie_jar))
request.install_opener(opener)

# HLS 2.0 bands:
# BAND_CROSSWALK = {
#     "HLSL30.v2.0": {
#         "B01": "coastal aerosol",
#         "B02": "blue",
#         "B03": "green",
#         "B04": "red",
#         "B05": "nir narrow",
#         "B06": "swir1",
#         "B07": "swir2",
#         "B09": "cirrus",
#         "B10": "thermal infrared 1",
#         "B11": "thermal",
#     },
#     "HLSS30.v2.0": {
#         "B01": "coastal aerosol",
#         "B02": "blue",
#         "B03": "green",
#         "B04": "red",
#         "B05": "red-edge 1",
#         "B06": "red-edge 2",
#         "B07": "red-edge 3",
#         "B08": "nir broad",
#         "B8A": "nir narrow",
#         "B09": "water vapor",
#         "B10": "cirrus",
#         "B11": "swir 1",
#         "B12": "swir 2",
#     },
# }

bands_S2_HLS = {"B": ".B02.tif", "G": ".B03.tif", "R": ".B04.tif", "RE1": ".B05.tif", "RE2": ".B06.tif", "RE3": ".B07.tif", "NIR": ".B8A.tif", "NIR_wide": ".B08.tif", "SWIR1": ".B11.tif", "SWIR2": ".B12.tif"} 
bands_L_HLS = {"B": ".B02.tif", "G": ".B03.tif", "R": ".B04.tif", "NIR": ".B05.tif", "SWIR1": ".B06.tif", "SWIR2": ".B07.tif", "T": ".B10.tif"} 

bands_HLS = {"L": bands_L_HLS, "S2": bands_S2_HLS}
HLS_prefix = {"L": 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020', "S2": 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020'}
hls_midfix_part1 = {"L": '/HLS.L30.T', "S2": '/HLS.S30.T'}

def download_HLS(datetime, short_name): # downloads all HLS S2 bands and created a multiband ENVI-compatible geotif image
  hls_midfix1 = hls_midfix_part1[sensor] + MGRS_tile + '.' + datetime + '.v2.0'

  hls_fmask_url = HLS_prefix[sensor] + hls_midfix1 + hls_midfix1 + ".Fmask.tif" # e.g., "https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSS30.020/HLS.S30.T57KVS.2023137T233821.v2.0/HLS.S30.T57KVS.2023137T233821.v2.0.Fmask.tif" 

  print('hls_fmask_url',hls_fmask_url)

  myrequest = request.Request(hls_fmask_url)
  response = ''
  try: response = request.urlopen(myrequest)
  except:
    print('Failed to download HLS')
    return False

  # Check if status is OK
  if response.code == 200:
    tif_filename = MGRS_tile+'_tif/'+short_name+"_HLS_fmask.tif"
    with open(tif_filename, "wb") as f: # here! should download into memory instead of disk!
        f.write(response.read())
    print("HLS Fmask image downloaded successfully:", short_name+"_HLS_fmask.tif")

    # Open the file using rasterio
    with rasterio.open(tif_filename) as src:
        # Read the raster data
        fmask = src.read()

    os.remove(MGRS_tile+'_tif/'+short_name+"_HLS_fmask.tif")
    fmask = fmask[0]

    cloud_mask = (fmask & 3) != 0  # Binary cloud mask
    cloud_adj_mask = (fmask & 4) != 0  # Adjacent to cloud/shadow
    shadow_mask = (fmask & 8) != 0  # Binary cloud shadow mask
    water_mask = (fmask & 16) != 0  # Binary water mask
    snow_mask = (fmask & 32) != 0  # Binary snow/ice mask

    # Combine the masks to create a final binary mask
    fmask = cloud_mask | cloud_adj_mask | shadow_mask | water_mask | snow_mask

    bands_to_download = bands[sensor]
    if sensor == 'L' and download_L_thermal:
       bands_to_download = bands_to_download + ['T']

    HLS_multiband = np.zeros((len(bands_to_download), fmask.shape[0], fmask.shape[1]), np.int16)

    for b_index, b in enumerate(bands_to_download):
        hls_url = HLS_prefix[sensor] + hls_midfix1 + hls_midfix1 + bands_HLS[sensor][b] # e.g., "https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T55GCQ.2023135T235740.v2.0/HLS.L30.T55GCQ.2023135T235740.v2.0.B03.tif" 
        print(hls_url)
        # Send a GET request to the URL to get the file content
        myrequest = request.Request(hls_url)
        response = request.urlopen(myrequest)

        # Check if status is OK
        if response.code == 200:
            band_filename = MGRS_tile+'_tif/'+short_name+"_HLS_"+b+".tif"
            with open(band_filename, "wb") as f:
                f.write(response.read())
            print("Download successful:", short_name+"_HLS_"+b+".tif")

            with rasterio.open(MGRS_tile+'_tif/'+short_name+"_HLS_"+b+".tif") as src:
                # Read the raster data
                HLS_multiband[b_index] = (src.read())[0]

            try:
              os.remove(band_filename)
            except FileNotFoundError:
              print(f"File '{band_filename}' not found.")

            continue

        print("Failed to download HLS image,:",short_name+"_HLS_"+b,"; retrying") # no retrying. Delete

    HLS_multiband[:,fmask] = 0
    HLS_multiband = HLS_multiband[:,:h['HLS'], 0:w['HLS']]

    if water_agri_mask_HLS:
      HLS_multiband[:,water_and_agri_mask_np['HLS']==False] = 0

    negative_pixels_mask = np.any(HLS_multiband <= 0, axis=0)
    HLS_multiband[:, negative_pixels_mask] = 0

    save_data_and_mask(HLS_multiband, short_name+"_HLS", "HLS", bands_to_download, wavelengths_all[short_name[10:]])

    return True

## Functions for downloading 30m Landsat from GEE to PC

In [ ]:
def downloader_L(collection_L,n_L,year_):
    start_ = time.time()
    t_previous = start_  

    sensors_L = collection_L.aggregate_array('SPACECRAFT_ID').getInfo() # e.g., 5, 7, 8, or 9
    sensors_L = [spacecraft_ID_short[s] for s in sensors_L] # e.g., '_L7'

    dates_L = collection_L.aggregate_array('date_str').getInfo() # e.g., '2022-07-12'

    print('sensors_L', sensors_L)
    print('dates_L',dates_L)

    # this code is for HLS downloading.
    hls_dates = []
    for d in dates_L:
        # Convert the date string to a datetime object
        date_obj = datetime.strptime(d, "%Y-%m-%d")

        # Extract the year and day of the year from the datetime object
        year = date_obj.strftime("%Y")
        day_of_year = date_obj.strftime("%j")

        # Concatenate the year and day of the year and convert to an integer
        hls_dates.append(year + day_of_year)

    # if the Landsat image is a mosaic, get the acquisition time of each original image (might be required for downloading HLS data, but time1 is probably enough)
    time1 = collection_L.aggregate_array('time1').getInfo()
    time2 = collection_L.aggregate_array('time2').getInfo()
    hls_time1 = [time[:2]+time[3:5]+time[6:8] for time in time1] # '19:13:58.6562020Z' → '191358'
    hls_time2 = [time[:2]+time[3:5]+time[6:8] for time in time2]

    collectionList = collection_L.toList(n_L)

    clearest_date_str = ''
    clearest_sensor = ''
    clearest_clearsky = 0

    short_names = []

    for i in range(n_L):
        start_i = time.time()
        short_names.append(dates_L[i] + sensors_L[i])

        if (
            sensors_L[i] >= '_L8'
            and download_HLS_L
            and (
                (not os.path.exists(MGRS_tile+'_tif/'+short_names[i]+'_HLS.tif') and save_tif)
                or (not os.path.exists(MGRS_tile+'/'+short_names[i]+'_HLS_data.npy') and save_np)
                or not skip_existing
            )
        ):
            downloaded_or_not = download_HLS(hls_dates[i]+'T'+hls_time1[i], short_names[i]) # 2023137T233821
            if not downloaded_or_not:
                downloaded_or_not = download_HLS(hls_dates[i]+'T'+hls_time2[i], short_names[i]) # here! only one attempt with each time (mosaic image 1 time and image 2 time) and without a wait?
                print('Using mosaic image 2 time for HLS download:', short_names[i])
                if not downloaded_or_not:
                    print('Failed to download HLS data for', short_names[i])

        if download_normal_L == False:
            continue

        if skip_existing:
            tif_ok = (not save_tif) or os.path.exists(MGRS_tile + '_tif/' + short_names[i] + '.tif')
            np_ok  = (not save_np)  or os.path.exists(MGRS_tile + '/' + short_names[i] + '_data.npy')
            if tif_ok and np_ok: # this is a duplicate of the code below in case the main image is already downloaded but the pan is not
                print(MGRS_tile+'/'+short_names[i]+' already downloaded') # but still need to check if pan is downloaded

                if download_L_pan and sensors_L[i] >= '_L7' and (
                    (not os.path.exists(MGRS_tile+'_tif/'+short_names[i]+'_pan.tif') and save_tif) or
                    (not os.path.exists(MGRS_tile+'/'+short_names[i]+'_pan.npy') and save_np)
                ):
                    mask_30m = gee2np(ee.Image(collectionList.get(i)), 'cloud_mask',sensor,'',4)
                    mask_30m = cloud_bufferer(mask_30m, round(cloud_buffer_m/resolution[sensor]))

                    mask_15m = shift_and_resize(mask_30m,resolution[sensor], resolution[hd_sensors[sensor]], shift_to_hd_y[sensor], shift_to_hd_x[sensor], 0) # nearest neighbour (method = 1) will shift the mask by 7.5 meters! 15m pan mask is shifted up and left relative to 30m. Method 0 (mixed2zero) will make masked areas bigger but it does not shift the mask, so it is better to use it here
                    mask_15m[water_and_agri_mask_np['L_pan'] == False] = False

                    pan = gee2np(ee.Image(collectionList.get(i)), 'pan','L_pan','',16)
                    pan[mask_15m == False] = 0
                    if save_np:
                        np.save(MGRS_tile+'/'+short_names[i]+'_pan',pan)
                    if save_tif:
                        np2tif(pan, short_names[i]+'_pan', "L_pan", 'pan', wavelengths_pan[sensors_L[i]])

                continue
        print('Processing image',i+1,'of',n_L,short_names[i])

        gee_image = ee.Image(collectionList.get(i)) # It probably uses NN by default when reprojecting from a different UTM zone. should compare with .resample('bicubic'). Tried using pixel aggregation but ".setDefaultProjection(S2_projection, crsTransform = transform['L_pan']).reduceResolution(**{'reducer': ee.Reducer.mean(), 'bestEffort': True})" makes no difference

        image_L = '' 

        for attempt in range(5):
          try:
            image_L = gee2np(gee_image.reproject(crs=S2_projection, crsTransform = transform['L']).clip(S2_tile['L']), bands[sensor]+(['T','cloud_mask'] if download_L_thermal else ['cloud_mask']),sensor,'',4) # do not set to 16! WIll crash 'EEException: Too Many Requests: Request was rejected because the request rate or concurrency limit was exceeded.'
          except:
            print('Ouch! Have to redownload all bands due to error.')
            image_L = ''
          if isinstance(image_L, np.ndarray): # is this required?
            break

        print('All bands downloaded:', round(time.time() - start_i), 'seconds')

        buffered_clouds = cloud_bufferer(image_L[-1], round(cloud_buffer_m/resolution[sensor])) # both clouds and snow are buffered in Landsat, but only snow in Sentinel-2 thanks to better cloud masks
        image_L = image_L[:-1]
        
        data_where = np.any(image_L == 0, axis=0) # this is duplicated inside save_data_and_mask, but it's required later here for pan masking (Landsat cloud mask may not cover entire pan area, so we have to mask pixels in pan that have no valid data in 30m bands to avoid unmasked clouds in such pixels)
        image_L[:,data_where] = 0

        image_L[:,~buffered_clouds] = 0
        image_L[:,water_and_agri_mask_np['L'] == False] = 0 # image_L[:,~water_and_agri_mask_np['L']] = 0 doesn't work for some reason

        save_data_and_mask(image_L, short_names[i], 'L', bands['L']+['T'] if download_L_thermal else bands['L'], wavelengths_all[sensors_L[i]])

        if download_L_pan and sensors_L[i] >= '_L7':
            mask_30m = buffered_clouds
            mask_15m = shift_and_resize(mask_30m,resolution[sensor], resolution[hd_sensors[sensor]], shift_to_hd_y[sensor], shift_to_hd_x[sensor], 0) # nearest neighbour (method = 1) will shift the mask by 7.5 meters!! Set to 0 instead (mixed2zero) to avoid shifting the mask, but it will make masked areas bigger
            
            data_where = binary_erosion(data_where, structure=np.ones((5, 5)))
            data_where = binary_dilation(data_where, structure=np.ones((3, 3)))
            data_where = shift_and_resize(data_where,resolution[sensor], resolution[hd_sensors[sensor]], shift_to_hd_y[sensor], shift_to_hd_x[sensor], 0)

            mask_15m[water_and_agri_mask_np['L_pan'] == False] = False
            mask_15m[data_where] = False

            pan = gee2np(gee_image.reproject(crs=S2_projection, crsTransform = transform['L_pan']), 'pan','L_pan','',16)
            pan[mask_15m == False] = 0
            if save_np:
                np.save(MGRS_tile+'/'+short_names[i]+'_pan',pan)
            if save_tif:
                np2tif(pan, short_names[i]+'_pan', 'L_pan', 'pan', wavelengths_pan[sensors_L[i]])

        t_now = time.time()
        print("Image",i+1,"of",n_L,"processed:",round(t_now - t_previous), "seconds; total:", round(t_now - start_), "seconds\n")

        # Parse the date string to a datetime object
        date_object = datetime.strptime(dates_L[i], "%Y-%m-%d")
        # Calculate the difference between the given date and the reference date
        julian_this = (date_object - datetime(date_object.year, 1, 1)).days
        if julian_this >= start_DOY_reference and julian_this <= end_DOY_reference:
            clearsky_ = np.count_nonzero(image_L[0])
            if clearsky_ > clearest_clearsky: 
                clearest_clearsky = round(100.0*clearsky_/n_not_water_L)
                clearest_date_str = dates_L[i]
                clearest_sensor = sensors_L[i]

        t_previous = t_now
                    
    if clearest_clearsky > 0:
        yearly_best_L.loc[len(yearly_best_L)] = {'year': year_,'date_str': clearest_date_str,'sensor': clearest_sensor,'clearsky': clearest_clearsky}

## Select 30m Landsat data

In [ ]:
# It takes 20m30s to process 1972 - 2022 # year at a time: 106 minutes to select and download 153 images at 300 Mbps (old version)
sensor = 'L'
n_images[sensor] = 0

if do_L and end_year >= 1982:
  n_not_water_max_sensor = np.count_nonzero(water_and_agri_mask_np['L'])*(resolution['L']/reducer_resolution_client)*(resolution['L']/reducer_resolution_client) # ugly

  start = time.time()

  start_year_L = start_year if start_year >= 1982 else 1982

  for year in range(start_year_L, end_year+1):
    print('_______________________\nprocessing year: ', year)

    landsat_collection_this_year = ee.ImageCollection([ee.Image(1), ee.Image(2)]) # ?

    available_sensors = []
    if year >= 1982 and year <= 1993:
      available_sensors.append(4)
    if year >= 1984 and year <= 2013:
      available_sensors.append(5)
    if year >= 1999 and year <= 2024: # "Landsat 7 captured one of its final images on May 28" 2024
      available_sensors.append(7)
    if year >= 2013:
      available_sensors.append(8)
    if year >= 2022:
      available_sensors.append(9)

    if pan_only:
      available_sensors = [7]

    for s in available_sensors:     
      landsat_collection = ee.ImageCollection(collection_names[s]).filterBounds(S2_tile['L']).filter(ee.Filter.calendarRange(year, year,'year')).filter(ee.Filter.calendarRange(start_DOY,end_DOY,'day_of_year')).filter(ee.Filter.lt('CLOUD_COVER', 85))

      addon_bands = ['SR_B1' if sr else 'B1'] # These are the bands used for smoke masking (in addition to SWIR1): B1 is blue in L4-7 and aerosol in B8-9
      addon_bands_renamed = ['B1']

      if sr:
        addon_bands = addon_bands+(['ST_B6' if s <= 7 else 'ST_B10']) 
      else:
        addon_bands = addon_bands+([('B6' if s < 7 else 'B6_VCID_2') if s <= 7 else 'B10']) # B11 is not very useful? In case of Landsat 7, High gain (VCID_2) is more sensitive to cooler surfaces like forests.
      addon_bands_renamed = addon_bands_renamed+['T']

      landsat_collection = landsat_collection.select(band_names[s]+addon_bands,band_names_renamed[s]+addon_bands_renamed) # no need for "if download_L_thermal"?
      landsat_collection = landsat_collection.map(get_solar_info)
      
      if not sr:
        landsat_collection = landsat_collection.map(get_L8_and_newer_thresholds) if s >= 8 else landsat_collection.map(get_L5andL7_thresholds)

      landsat_collection_mosiac_checked = landsat_collection.map(mosaic_or_not)
      landsat_collection_0before = landsat_collection_mosiac_checked.filter(ee.Filter.eq('n_before', 0))
      landsat_collection_gte1before = landsat_collection_mosiac_checked.filter(ee.Filter.gt('n_before', 0))

      landsat_collection_1img = landsat_collection_0before.filter(ee.Filter.eq('n_after', 1))
      landsat_collection_2img = landsat_collection_0before.filter(ee.Filter.gt('n_after', 1))

      join_filter = ee.Filter.equals(leftField='date_str', rightField='date_str')
      joined_collection = ee.Join.saveFirst('second').apply(
          primary=landsat_collection_2img,
          secondary=landsat_collection_gte1before,
          condition=join_filter
      )
      landsat_collection_2img_mosaiced = ee.ImageCollection(joined_collection.map(mosaic_L_pan_fast if (s >= 7 and not sr) else mosaic_L_fast))
      landsat_collection_1and2img_mosaiced = landsat_collection_2img_mosaiced.merge(landsat_collection_1img)
      landsat_collection_1and2img_mosaiced = (landsat_collection_1and2img_mosaiced.map(smoke_sr) if s < 8 else landsat_collection_1and2img_mosaiced.map(smoke_sr_oli)) if sr else landsat_collection_1and2img_mosaiced.map(smoke_raw)
      landsat_collection_1and2img_mosaiced = landsat_collection_1and2img_mosaiced.map(masker_L) # could do masker_L after combining collections from different L sensors, outside this loop, but the processing speed would probably be the same

      landsat_collection_this_year = landsat_collection_this_year.merge(landsat_collection_1and2img_mosaiced)

    landsat_collection_this_year = landsat_collection_this_year.filter(ee.Filter.gt('clearsky', 0))
    
    if landsat_collection_this_year.size().getInfo() == 0:
      print('no Landsat images for year ', year)
      continue 
    
    coverage = 0
    landsat_collection_clearsky = 0
    clearsky_threshold = clearsky_threshold_global - 0.1 if pan_only else clearsky_threshold_global # to consider more L7 images because we already downloaded L7 at clearsky_threshold_global during the normal run

    n_landsat_previous = 0

    n_landsat = 0

    while coverage < min_coverage and clearsky_threshold >= 0.2: # here! no need to do this for 2016-Present because there are more than enough images to provide full coverage of the study area
      clearsky_threshold_px = ee.Number(clearsky_threshold).multiply(n_not_water_max)

      landsat_collection_clearsky = landsat_collection_this_year.filter(ee.Filter.gt('clearsky', clearsky_threshold_px))

      n_landsat = landsat_collection_clearsky.size().getInfo()
      if n_landsat > n_landsat_previous:
        n_landsat_previous = n_landsat

        print('single-image clearsky threshold {:.1f}%: {} images'.format(clearsky_threshold*100, n_landsat))       
        yearly_mosaic = landsat_collection_clearsky.select('mask').mosaic().gt(0).updateMask(water_and_agri_mask).rename('coverage')
        coverage_stats = yearly_mosaic.reduceRegion(**{
            'reducer': ee.Reducer.count(),
            'geometry': S2_tile['L'],
            'crs': S2_projection,
            'scale': reducer_resolution,
            'bestEffort': True,
            'tileScale': 16,
            'maxPixels': 1e7
          }).get('coverage')     
        coverage_px = coverage_stats.getInfo() # number of land non-agri non-water pixels covered by all images this year
        coverage = coverage_px/n_not_water_max_sensor
        print("Coverage: {:.2f}%".format(coverage*100)) # note: this can exceed 100% becase n_not_water_max_client is calculated using reducer_resolution (120m by default) water_and_agri_mask (mixed to zero resampling) and coverage_px using the original 30m crop and water mask but at reducer_resolution
      else:
        print('single-image clearsky threshold {:.1f}%: same'.format(clearsky_threshold*100)) 
      
      clearsky_threshold -= 0.1

    if sr:
      landsat_collection_clearsky = landsat_collection_clearsky.map(apply_mask_L) # apply cloud mask and reproject data # do not move this outside this if-else # here! as of 2025-10-23 there is no reprojection!
      if do_topo:
        landsat_collection_clearsky = landsat_collection_clearsky.map(illumination_L_16bit)

    else:
      landsat_collection_clearsky_L4toL5 = landsat_collection_clearsky.filter(ee.Filter.lt('SPACECRAFT_ID', 'LANDSAT_7'))

      landsat_collection_clearsky_L4toL5 = landsat_collection_clearsky_L4toL5.map(apply_mask_L) # apply cloud mask and reproject data # here! as of 2025-10-23 there is no reprojection!
      if do_topo:
        landsat_collection_clearsky_L4toL5 = landsat_collection_clearsky_L4toL5.map(illumination_L_8bit) # apply topo correction and convert uint8 to uint16

      landsat_collection_clearsky_L7_and_newer = landsat_collection_clearsky.filter(ee.Filter.gte('SPACECRAFT_ID', 'LANDSAT_7')) # because L7 and newer have pan band
      
      landsat_collection_clearsky_L7_and_newer = landsat_collection_clearsky_L7_and_newer.map(apply_mask_L_pan) # apply cloud mask and reproject data # here! as of 2025-10-23 there is no reprojection!
      if do_topo:
        landsat_collection_clearsky_8bit = landsat_collection_clearsky_L7_and_newer.filter(ee.Filter.lte('SPACECRAFT_ID', 'LANDSAT_7'))
        landsat_collection_clearsky_8bit = landsat_collection_clearsky_8bit.map(illumination_L_8bit)
        landsat_collection_clearsky_16bit = landsat_collection_clearsky_L7_and_newer.filter(ee.Filter.gte('SPACECRAFT_ID', 'LANDSAT_8'))
        landsat_collection_clearsky_16bit = landsat_collection_clearsky_16bit.map(illumination_L_16bit)
        landsat_collection_clearsky_L7_and_newer = landsat_collection_clearsky_8bit.merge(landsat_collection_clearsky_16bit)

      landsat_collection_clearsky = landsat_collection_clearsky_L4toL5.merge(landsat_collection_clearsky_L7_and_newer) 
    
    print('\ndownloading year',year)  
    downloader_L(landsat_collection_clearsky,n_landsat,year)
    print('year', year,'processed:',round((time.time()-start)/60.0),'minutes since start\n')
    n_images[sensor] = n_images[sensor] + n_landsat

  print('======================\nTotal number of Landsat images downloaded:',n_images[sensor],'in',round((time.time()-start)/60.0),'minutes')
  print('best mid-summer image:\n',yearly_best_L)

In [ ]:
def bicubic_10m(img):
    cloud_mask = img.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).unmask().eq(0).rename('cloud_mask')
    return img.select('B8').updateMask(cloud_mask).resample('bicubic').reproject(crs=S2_projection, crsTransform = transform['S2_10m']).rename('pan') # reproject here might not be required but bicubic is

def bilinear_10m(img): # seems to produce blurrier / softer images than bicubic
    cloud_mask = img.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).unmask().eq(0).rename('cloud_mask')
    return img.select('B8').updateMask(cloud_mask).resample('bilinear').reproject(crs=S2_projection, crsTransform = transform['S2_10m']).rename('pan') # bilinear is default resample, can just use ".resample()"

In [ ]:
def find_best_image(s1, s2): # too many getInfo(), need to optimize
    sensor = 'L'
    print(f'Looking for a cloud-free L{s1} or L{s2} image')
    # LC09 is slightly better than LC08 (14bit vs 12bit), but only available since 2022. 
    # Do not filter by projection (i.e., same projection/UTM zone as S2), because pre-mosaiced images can be from different UTM zones

    start_year_ref = {4:1982,5:1984,8:2018,9:2022}
    end_year_ref = {4:1993,5:1993,8:datetime.now().year,9:datetime.now().year} # here! This assumes the user wants data up to the current year, but would need a bunch of if statements otherwise
    collection_ref = {4:'LANDSAT/LT04/C02/T1',5:'LANDSAT/LT05/C02/T1',8:'LANDSAT/LC08/C02/T1',9:'LANDSAT/LC09/C02/T1'}
    clearsky_ref = 0

    global landsat_collection,landsat_collection_gte1before
    landsat_collection = ee.ImageCollection(collection_ref[s1]).filterBounds(S2_tile['L']).filter(ee.Filter.calendarRange(start_year_ref[s1],end_year_ref[s1],'year')).filter(ee.Filter.calendarRange(start_DOY_reference,end_DOY_reference,'day_of_year')).filter(ee.Filter.lt('CLOUD_COVER', 30)).select(['B2',  'B3', 'B4', 'B5', 'B6', 'B7','QA_PIXEL','QA_RADSAT'], ['B','G','R','NIR','SWIR1','SWIR2','QA_PIXEL','QA_RADSAT'])       
    if s1 != 4 or landsat_collection.size().getInfo() > 0:
        landsat_collection = landsat_collection.sort('CLOUD_COVER').limit(20)
        landsat_collection = landsat_collection.map(get_solar_info)
        landsat_collection = landsat_collection.map(get_L8_and_newer_thresholds)
        landsat_collection_mosiac_checked = landsat_collection.map(mosaic_or_not)
        landsat_collection_0before = landsat_collection_mosiac_checked.filter(ee.Filter.eq('n_before', 0))
        landsat_collection_gte1before = landsat_collection_mosiac_checked.filter(ee.Filter.gt('n_before', 0))
        landsat_collection_1img = landsat_collection_0before.filter(ee.Filter.eq('n_after', 1))
        landsat_collection_2img = landsat_collection_0before.filter(ee.Filter.gt('n_after', 1))

        join_filter = ee.Filter.equals(leftField='date_str', rightField='date_str')
        joined_collection = ee.Join.saveFirst('second').apply(
            primary=landsat_collection_2img,
            secondary=landsat_collection_gte1before,
            condition=join_filter
        )
        
        landsat_collection_2img_mosaiced = ee.ImageCollection(joined_collection.map(mosaic_L_fast)) # no pan? # landsat_collection_2img.map(mosaic_L)
        landsat_collection_1and2img_mosaiced = landsat_collection_2img_mosaiced.merge(landsat_collection_1img)
        landsat_collection_1and2img_mosaiced = landsat_collection_1and2img_mosaiced.map(smoke_raw)
        landsat_collection = landsat_collection_1and2img_mosaiced.map(masker_L) 

        n_clearsky_pixels_list = landsat_collection.aggregate_array('clearsky').getInfo() # for some reason this is faster than doing .sort('clearsky', False).first()
        date_list = landsat_collection.aggregate_array('date_str').getInfo()
        most_clearsky_pixels = max(n_clearsky_pixels_list)
        most_clearsky_index = n_clearsky_pixels_list.index(most_clearsky_pixels)
        date_ref = date_list[most_clearsky_index]
        sensor_L = f'_L{s1}'
        clearsky_ref = round(100.0*most_clearsky_pixels/n_not_water_max_client)
        print(f'L{s1} clearsky: ', clearsky_ref,'%')

    # process L9 but don't merge with L8 collection or will get "EEException: User memory limit exceeded"
    landsat_collection = ee.ImageCollection(collection_ref[s2]).filterBounds(S2_tile['L']).filter(ee.Filter.calendarRange(start_year_ref[s2],end_year_ref[s2],'year')).filter(ee.Filter.calendarRange(start_DOY_reference,end_DOY_reference,'day_of_year')).filter(ee.Filter.lt('CLOUD_COVER', 30)).select(['B2',  'B3', 'B4', 'B5', 'B6', 'B7','QA_PIXEL','QA_RADSAT'], ['B','G','R','NIR','SWIR1','SWIR2','QA_PIXEL','QA_RADSAT'])
    landsat_collection = landsat_collection.sort('CLOUD_COVER').limit(20)
    landsat_collection = landsat_collection.map(get_solar_info)
    landsat_collection = landsat_collection.map(get_L8_and_newer_thresholds)
    landsat_collection_mosiac_checked = landsat_collection.map(mosaic_or_not)
    landsat_collection_0before = landsat_collection_mosiac_checked.filter(ee.Filter.eq('n_before', 0))
    landsat_collection_gte1before = landsat_collection_mosiac_checked.filter(ee.Filter.gt('n_before', 0))
    landsat_collection_1img = landsat_collection_0before.filter(ee.Filter.eq('n_after', 1))
    landsat_collection_2img = landsat_collection_0before.filter(ee.Filter.gt('n_after', 1))

    join_filter = ee.Filter.equals(leftField='date_str', rightField='date_str')
    joined_collection = ee.Join.saveFirst('second').apply(
        primary=landsat_collection_2img,
        secondary=landsat_collection_gte1before,
        condition=join_filter
    )
    
    landsat_collection_2img_mosaiced = ee.ImageCollection(joined_collection.map(mosaic_L_fast)) # no pan? # landsat_collection_2img.map(mosaic_L)
    landsat_collection_1and2img_mosaiced = landsat_collection_2img_mosaiced.merge(landsat_collection_1img)
    landsat_collection_1and2img_mosaiced = landsat_collection_1and2img_mosaiced.map(smoke_raw)
    landsat_collection = landsat_collection_1and2img_mosaiced.map(masker_L) 

    # landsat_collection = landsat_collection.map(masker_L)
    n_clearsky_pixels_list = landsat_collection.aggregate_array('clearsky').getInfo()
    most_clearsky_pixels_2 = max(n_clearsky_pixels_list) # number of reducer_resolution (120m) pixels 
    clearsky_ref2 = round(100.0*most_clearsky_pixels_2/n_not_water_max_client)
    print(f'L{s2} clearsky: ',clearsky_ref2,'%')

    if clearsky_ref2 >= clearsky_ref: # if the best L8 has only slightly more clearsky pixels (i.e., rounding error), choose Landsat 9 over Landsat 8
        clearsky_ref = clearsky_ref2
        most_clearsky_index = n_clearsky_pixels_list.index(most_clearsky_pixels_2)
        date_list = landsat_collection.aggregate_array('date_str').getInfo()
        date_ref = date_list[most_clearsky_index]
        sensor_L = f'_L{s2}'  

    print('Best image for coregistration:',date_ref+sensor_L) 
    return date_ref, sensor_L

## Find the best, most cloud-free Landsat 8 or 9 OLI panchromatic image to which Sentinel-2 data will be coregistered to

"As a shortcut, you can specify a scale parameter and Earth Engine will calculate a crsTransform parameter for you. However, simply setting the scale of an image does not specify the origin of the projection, and may result in an image that is shifted relative to another image with the same pixel size!
The reason for the potential shift is that the scale parameter is used to populate the xScale and yScale values of the crsTransform, but the xTranslation and yTranslation values are calculated so that if they are divided by the corresponding xScale and yScale values the remainder will be zero. These parameters specify a pixel grid where the projection's origin is at the corner of a pixel. This convention differs from the translation parameters used by some data providers, which use grids that are offset from the projection's origin. For example, Landsat images provided by USGS use translation parameters that are offset by a 1/2 pixel from the projection's origin (15m offset for the 30m bands) while Sentinel-2 images provided by ESA use translation parameters that are aligned with the projection's origin. If the crsTransform specified in an export do not match the crsTransform of the original image, the output pixels will be resampled (using nearest neighbor by default), which will make the resulting image be shifted relative to the original image.
To sum up, if you need to match the exported image's pixels to a specific image, make sure to use the crs and crsTransform parameters for full control of the grid."
https://developers.google.com/earth-engine/guides/exporting

In [ ]:
# simpleComposite required 2m 44s vs 55s for median without masking and 2m 27s for median with cloud and snow masking and 1m45s with cloud masking vs 40s when selecting one image
# https://developers.google.com/earth-engine/guides/landsat -- composite = ee.Algorithms.Landsat.simpleComposite(collection) -- or:
# customComposite = ee.Algorithms.Landsat.simpleComposite(**{
#   'collection': collection,
#   'percentile': 75,
#   'cloudScoreRange': 5
# })

landsat9_best_pan = ''
pan_date = ''

if do_S2 and end_year >= 2016 and (do_coregister_S2 or align_S2):
    if composite_pan: # use pan image composite for corigistering S2 images. This will crash GEE (too many operations)
        landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1').filterBounds(S2_tile['S2']).filter(ee.Filter.calendarRange(2018, datetime.now().year,'year')).filter(ee.Filter.calendarRange(start_DOY_reference,end_DOY_reference,'day_of_year')).filter(ee.Filter.lt('CLOUD_COVER', 30)).select('B8')   
        landsat_collection = landsat_collection.sort('CLOUD_COVER').limit(20)
        landsat9_best_pan = landsat_collection.select(['B8']).median().uint16()
    else: # use a single, most cloud-free Landsat pan image for corigistering S2 images. This may not cover the entire tile if the tile is located between two Landsat paths
        sensor_L = ''
        filtered_df = yearly_best_L[yearly_best_L['year'] >= 2018]

        if len(filtered_df) >= 5: # if at least 5 years of Landsat data (>= 2018) have been processed # filtered_df.loc[index_with_highest_clearsky, 'clearsky'] > 80:
            index_with_highest_clearsky = filtered_df['clearsky'].idxmax() # Find the index of the row with the highest 'clearsky' value
            pan_date = filtered_df.loc[index_with_highest_clearsky, 'date_str']
            sensor_L = filtered_df.loc[index_with_highest_clearsky, 'sensor']
        else:
            pan_date, sensor_L = find_best_image(8,9)

        # SR products don't have the pan band, so have to use radiance data
        landsat9_best_pan_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1' if sensor_L == '_L8' else 'LANDSAT/LC09/C02/T1').filter(ee.Filter.eq('DATE_ACQUIRED', pan_date)).filterBounds(S2_tile['S2'])

        # mask clouds and resample from 15m to 10m
        landsat9_best_pan_collection = landsat9_best_pan_collection.map(bicubic_10m) # make sure to resample using bilinear or bicubic convolution because the default is nearest neighbour (NN will simply shift pixels). Resample before mosaicing because bicubic and bilinear doesn't work on mosaics

        # if there are two images for this date, mosaic them together
        landsat9_best_pan = ee.Algorithms.If(ee.Algorithms.IsEqual(landsat9_best_pan_collection.size(), ee.Number(1)), landsat9_best_pan_collection.first().select('pan'), landsat9_best_pan_collection.select('pan').qualityMosaic('pan')) # output of this is a 'ComputedObject', so have to convert it to ee.Image
        landsat9_best_pan = ee.Image(landsat9_best_pan).uint16() # convert compute object to gee image

        sensor = 'S2'

    landsat9_best_pan = landsat9_best_pan.updateMask(water_and_agri_mask).reproject(crs=S2_projection, crsTransform = transform['S2_10m']).rename('pan') # reproject is required even though already resampled using bicubic_10m
    landsat9_best_pan_np = gee2np(landsat9_best_pan.clip(S2_tile['S2_10m']),'pan','S2_10m', 'L8_pan_composite_for_S2_coregistration_10m' if composite_pan else pan_date+sensor_L+'pan_best_for_S2_coregistration_10m',16)

    # use only pixels found in the diagonals of the 10m Landsat panchromatic image to make Sentinel-2 coregistration faster:
    pan_10m_trimmed = landsat9_best_pan_np[max_shift_s2_10m:h[10]-max_shift_s2_10m, max_shift_s2_10m:w[10]-max_shift_s2_10m]
    main_diagonal = pan_10m_trimmed.diagonal()
    anti_diagonal = np.fliplr(pan_10m_trimmed).diagonal()
    pan_10m_diagonal = np.concatenate((main_diagonal, anti_diagonal))
    pan_10m_diagonal_non0where = pan_10m_diagonal > 0
    pan_10m_diagonal_0where = pan_10m_diagonal <= 0

## Sentinel-2 data preprocessing GEE functions

In [ ]:
bands_S2_10mAnd20m = bands['S2_10m']+bands['S2_20m']
bands_S2_10mAnd20m_all = bands_S2_10mAnd20m + ['cloud_probability']

# here! consider using NDSI instead, but NDSI is not as stable for TOA data compared to SR data
def s2csPlus_cloudmasker_toa(img): # a separate function for TOA is required because TOA data lacks water and good (?) snow masks (TOA S2 data has no SCL band)
  # datapixels = img.unmask().reduce(ee.Reducer.min()).gt(0) # remove pixels that are zero in at least one of the bands. Is this required for S2?

  # This QA60 mask seems to be useless but it might get better with future S2 processing baselines.
  # qa = img.select('QA60')
  # snowMask1 = qa.bitwiseAnd(1 << 10).unmask().eq(0)
  # snowMask2 = qa.bitwiseAnd(1 << 11).unmask().eq(0)
  # snow_where = snowMask1.updateMask(snowMask2)

  # Create the mask
  snow_where = img.select('G').lt(snow_thres_green_s2).Or(img.select('SWIR1').gt(snow_thres_swir1_s2)).unmask(1).rename('snow')
  smoke_where = img.select('B1').lt(s2_toa_b1_threshold).Or(img.select('SWIR1').gt(s2_toa_swir1_threshold)).unmask(1).rename('smoke')
  snow_where = snow_where.updateMask(smoke_where).rename('snow')

  not_cloud = img.select(s2_cloud_mask_type).gte(s2_cloud_score_threshold)

  not_cloud_or_water = not_cloud.updateMask(snow_where).updateMask(water_and_agri_mask).rename('notCloudOrWater')
  
  n_not_cloud_or_water_obj = not_cloud_or_water.reduceRegion(**{
      'reducer': ee.Reducer.sum(),
      'geometry': S2_tile['S2'],
      'crs': S2_projection,
      'scale': reducer_resolution,
      'bestEffort': True,
      'maxPixels': 1e7
    }).get('notCloudOrWater')
  n_not_cloud_or_water = ee.Number(n_not_cloud_or_water_obj)

  return img.select(bands_S2_10mAnd20m).updateMask(not_cloud_or_water).addBands(snow_where).set('n_not_cloud_or_water',n_not_cloud_or_water).set('date', img.date().format("YYYY-MM-dd"))

def s2csPlus_cloudmasker_sr(img): # use existing snow mask, water mask and NDSI when using SR data
  # datapixels = img.unmask().reduce(ee.Reducer.min()).gt(0) # remove pixels that are zero in at least one of the bands. Is this required for S2?
  not_water = img.select('SCL').neq(6).rename('nonWater') # SR S2 data has a water mask

  # note: NDSI of water is also > 0, so a buffer is applied to water pixels that were not masked using ESA and AAFC crop and water masks as if they were snow if ndsi_thres_s2 is set to 0
  ndsi = img.normalizedDifference(['G', 'SWIR1']).unmask(-1).rename('NDSI') # https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-2-msi/level-2a/algorithm-overview 

  snow_where = (img.select('G').gt(1000).And(ndsi.gt(ndsi_thres_s2))).unmask().eq(0).rename('snow') # 1000 = 10% # here! hardcoded threshold
  smoke_where = (img.select('B1').gt(s2_sr_b1_threshold).And(img.select('SWIR1').lt(s2_sr_swir1_threshold))).unmask().eq(0).rename('smoke')
  snow_where = snow_where.updateMask(smoke_where).rename('snow')

  not_cloud_or_snow = img.select(s2_cloud_mask_type).gte(s2_cloud_score_threshold).updateMask(snow_where)

  not_cloud_or_water = not_water.updateMask(not_cloud_or_snow).updateMask(water_and_agri_mask).rename('notCloudOrWater') # water_and_agri_mask is required for areas with a lot of agriculture

  n_not_cloud_or_water_obj = not_cloud_or_water.reduceRegion(**{
      'reducer': ee.Reducer.count(),
      'geometry': S2_tile['S2'],
      'crs': S2_projection,
      'scale': reducer_resolution,
      'bestEffort': True,
      'maxPixels': 1e7
    }).get('notCloudOrWater')
  n_not_cloud_or_water = ee.Number(n_not_cloud_or_water_obj)

  return img.select(bands_S2_10mAnd20m).updateMask(not_cloud_or_water).addBands(snow_where).set('n_not_cloud_or_water',n_not_cloud_or_water).set('date', img.date().format("YYYY-MM-dd"))

 # GEE server-side topographic correction
def illumination_s2(img):
  azimuth = ee.Image.constant(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')).multiply(degree2rad)).clip(S2_tile['S2']) # degrees to radians # clip is probably not required
  zenith = ee.Number(img.get('MEAN_SOLAR_ZENITH_ANGLE')).multiply(degree2rad)
  cosZ = zenith.cos() 
  cos_slope_x_cosZ = cos_slope_noreproj.multiply(cosZ)
    
  # illumination = cos(slope)*cos(zenith) + sin(slope)*sin(zenith)*cos(azimuth - aspect)
  illumination_azimuth = sin_slope_noreproj.multiply(zenith.sin()).multiply(azimuth.subtract(aspect_noreproj).cos()) # sin(slope)*sin(zenith)*cos(azimuth - aspect)
  illumination = cos_slope_x_cosZ.add(illumination_azimuth).rename('illumination')

  img_i = img.addBands(illumination)

  illumination_gte0 = illumination.max(0.0)

  for band in bands_S2_10mAnd20m:
    out = img_i.select('illumination', band).reduceRegion(**{
      'reducer': ee.Reducer.linearFit(), # Compute linear regression coefficients: a(slope), b(offset), c(b/a)
      'geometry': linearFitGeometry, # ee.Geometry(img.geometry().buffer(-5000)), # trim off the outer edges of the image for linear relationship 
      'scale': 120,
      'maxPixels': 1000000000,
      'tileScale': 2 
    })
    
    out_a = ee.Number(out.get('scale'))
    out_b = ee.Number(out.get('offset'))
    out_c = out_b.divide(out_a)
    # Apply the SCSc correction
    # replace img with img_i if you want to apply cloud mask server side instead of building and applying it again on client side
    SCSc_output = img.select(band).multiply(cos_slope_x_cosZ.add(out_c)).divide(illumination_gte0.add(out_c)).rename(band)
    
    img = img.addBands(**{
        'srcImg': SCSc_output,
        'overwrite': True # replace original bands with topo corrected ones
      })
  
  return img.uint16()

# align S2 and Landsat images using the GEE displacement function: https://github.com/ndminhhus/geeguide/blob/master/08.image_registration.md and https://developers.google.com/earth-engine/guides/register
def coregister_S2(img): # blurs images unfortunately...
  displacement_s2 = img.select(['G','R']).reduce('mean').updateMask(img.select('snow')).displacement(**{ # use average of green and red band to match the spectral coverage of Landsat 8 - 9 pan band
      'referenceImage': landsat9_best_pan,
      'projection': S2_projection, # "The projection in which to output displacement values. The default is the projection of the first band of the reference image.""
      'maxOffset': 60.0, # "The maximum offset allowed when attempting to align the input images, in meters. Using a smaller value can reduce computation time significantly, but it must still be large enough to cover the greatest displacement within the entire image region.""
      # 'patchWidth': 360.0, # "Patch size for detecting image offsets, in meters. This should be set large enough to capture texture, as well as large enough that ignorable objects are small within the patch. Default is null. Patch size will be determined automatically if not provided.""
      'stiffness': 8 # little to no difference betwen stiffness=7 and stiffness=10. Will crash if set below 7 due to insafficient memory
  })
  return img.displace(displacement_s2)

In [ ]:
# client-side functions for Sentinel-2 images

def resize_10to20m(img): # client-side function for multi-band images that averages four adjacent pixels
  resized_S2 = np.zeros((img.shape[0], int(img.shape[1]/2), int(img.shape[2]/2)))
  resized_mask = np.full((int(img.shape[1]/2), int(img.shape[2]/2)), False)

  for sv in [0,1]:
    for sh in [0,1]:
      resized_mask[img[0,sv::2,sh::2] == 0] = True
      for b in range(img.shape[0]):
        resized_S2[b] += img[b,sv::2,sh::2] # sum the values of four adjacent pixel. This should work for uint16 because most values range between 0 and 10000 (10000 = 100% TOA)
    
  resized_S2 = resized_S2/4 # get the average value of four adjacent pixels
  resized_S2[:,resized_mask] = 0
  return resized_S2.astype(img.dtype)

def resize_custom(img, factor): # client-side function for multi-band images
  resized_S2 = np.zeros((img.shape[0], int(img.shape[1]/factor), int(img.shape[2]/factor))) # , dtype = img.dtype # changed from np.zeros((img.shape[0],h[20], w[20])) on 2024-10-01
  resized_mask = np.full((int(img.shape[1]/factor), int(img.shape[2]/factor)), False)

  for sv in range(factor):
    for sh in range(factor):
      resized_mask[img[0,sv::factor,sh::factor] == 0] = True

      for b in range(img.shape[0]):
        resized_S2[b] += img[b,sv::factor,sh::factor]
    
  resized_S2 = resized_S2/(factor*factor)
  resized_S2[:,resized_mask] = 0

  return resized_S2.astype(img.dtype)

def align_10m_S2_and_Lpan(img):
    highest_cor = 0
    h_shift_best = 0
    v_shift_best = 0  

    for v_shift in range(-1*max_shift_s2_10m,max_shift_s2_10m+1):
      for h_shift in range(-1*max_shift_s2_10m,max_shift_s2_10m+1):
        # shift image v_shift pixels up or down and h_shift left or right
        img_shifted = img[max_shift_s2_10m+v_shift:v_shift+h[10]-max_shift_s2_10m, max_shift_s2_10m+h_shift:h_shift+w[10]-max_shift_s2_10m].copy() # trim max_shift*2 pixels vertically and horizontally # no need for ".copy()" here

        # Extract main diagonal values
        main_diagonal = img_shifted.diagonal()
        # Extract anti-diagonal values
        anti_diagonal = np.fliplr(img_shifted).diagonal()
        img_shifted = np.concatenate((main_diagonal, anti_diagonal))

        img_shifted[pan_10m_diagonal_0where] = 0
        pan_10m_copy = pan_10m_diagonal.copy()
        pan_10m_copy = pan_10m_copy[img_shifted > 0]
        img_shifted = img_shifted[img_shifted > 0]

        # Calculate the correlation coefficient
        cor = np.corrcoef(img_shifted,pan_10m_copy)[0, 1] 

        if cor > highest_cor:
          highest_cor = cor
          h_shift_best = h_shift
          v_shift_best = v_shift    

    print('v_shift_best, h_shift_best,highest_cor:',-10*v_shift_best, "meters,", -10*h_shift_best, "meters,",highest_cor)
    return -1*v_shift_best, -1*h_shift_best

## Functions for downloading Sentinel-2 data from ESA

### Sun Canopy Sensor + C Correction (SCS + C) topographic correction of Sentinel-2 data on a local computer - this is required for ESA TOA Sentinel-2 data

In [ ]:
do_topo_s2 = do_topo # no need to .copy() because this is bool
if sr:
  do_topo_s2 = False
  print('Sentinel-2 Level-2A (surface reflectance) data do not require topographic correction (already applied)')

if do_S2 and do_topo_s2 and download_ESA_S2: 
    aspect_rad2d = {}
    cos_slope2d = {}
    sin_slope2d = {}
    
    # resample DEM to 10m using bicubic interpolation
    if DEM_forESA_S2 == 'ESA': # based on Tandem-X data. It is the most detailed and accurate, so it's the best for flattening small topographic features, BUT!! it contains trees (i.e., it's not a digital terrain model, but a digital surface model)! using it for topo will remove shadows from trees in clearcuts that occurred just before TandemX data was acquired. This is not good if clearcuts occured after that or long before that. Using this DEM may cause GEE crashes due to mosaicking
        dem_esa = ee.ImageCollection('COPERNICUS/DEM/GLO30').filterBounds(S2_tile['GEE']).select('DEM').map(resample_dem_10m).mosaic().reproject(S2_projection, crsTransform = transform['S2_10m']).uint16()
    elif DEM_forESA_S2 == 'NASA': 
        dem_esa = ee.Image('NASA/NASADEM_HGT/001').select('elevation').resample('bicubic').reproject(S2_projection, crsTransform = transform['S2_10m']).uint16().rename('DEM') # NASADEM_HGT is 30m - was used by TSDD before 2024-07-20
    elif DEM_forESA_S2 == 'CSA': # this is the worst. Makes things worse
        dem_esa = ee.ImageCollection('NRCan/CDEM').filterBounds(S2_tile['GEE']).select('elevation').map(resample_dem_10m).mosaic().reproject(S2_projection, crsTransform = transform['S2_10m']).uint16().rename('DEM') # CDEM is ~23m (0.75 arc-seconds) but available only for Canada
    elif DEM_forESA_S2 == 'ALOS':
        dem_esa = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2').filterBounds(S2_tile['GEE']).select('DSM').map(resample_dem_10m).mosaic().reproject(S2_projection, crsTransform = transform['S2_10m']).uint16().rename('DEM')

    dem_esa_np = gee2np(dem_esa,'DEM','S2_10m','DEM_ESA_10m', 16) # resampled from 30m to 10m
    # Calculate gradients (in meters)
    dy, dx = np.gradient(dem_esa_np, resolution['S2_10m'], resolution['S2_10m'])
    # Slope in radians
    slope = np.arctan(np.sqrt(dx**2 + dy**2))
    # Aspect in radians
    aspect = np.arctan2(-dx, dy) # not np.arctan2(-dy, dx) as suggested by ChatGPT!
    # Precomputed DEM-based values for topographic correction
    aspect_rad2d[resolution['S2_10m']] = np.mod(aspect, 2 * np.pi)
    cos_slope2d[resolution['S2_10m']] = np.cos(slope)
    sin_slope2d[resolution['S2_10m']] = np.sin(slope)

    # resample DEM to 20m using bicubic interpolation
    if DEM_forESA_S2 == 'ESA':
        dem_esa = ee.ImageCollection('COPERNICUS/DEM/GLO30').filterBounds(S2_tile['GEE']).select('DEM').map(resample_dem_20m).mosaic().reproject(S2_projection, crsTransform = transform['S2']).uint16()
    elif DEM_forESA_S2 == 'NASA':    
        dem_esa = ee.Image('NASA/NASADEM_HGT/001').select('elevation').resample('bicubic').reproject(S2_projection, crsTransform = transform['S2']).uint16().rename('DEM') # # ee.Image('NASA/NASADEM_HGT/001').select('elevation') NASADEM_HGT is 30m - was used by TSDD before 2024-07-20 # ee.ImageCollection('NRCan/CDEM') # CDEM is ~23m (0.75 arc-seconds) -- use CDEM for Canada? # ee.Image('COPERNICUS/DEM/GLO30').select('DEM').filterBounds(S2_tile['GEE']).first() might be more accurate global DEM according to ChatGPT but may cause crashes due to mosaicking
    elif DEM_forESA_S2 == 'CSA':
        dem_esa = ee.ImageCollection('NRCan/CDEM').filterBounds(S2_tile['GEE']).select('elevation').map(resample_dem_20m).mosaic().reproject(S2_projection, crsTransform = transform['S2']).uint16().rename('DEM')
    elif DEM_forESA_S2 == 'ALOS':
        dem_esa = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2').filterBounds(S2_tile['GEE']).select('DSM').map(resample_dem_20m).mosaic().reproject(S2_projection, crsTransform = transform['S2']).uint16().rename('DEM')

    dem_esa_np = gee2np(dem_esa,'DEM','S2','DEM_ESA_20m', 16) # resampled from 30m to 10m
    # Calculate gradients (in meters)
    dy, dx = np.gradient(dem_esa_np, resolution['S2'], resolution['S2'])
    # Slope in radians
    slope = np.arctan(np.sqrt(dx**2 + dy**2))
    # Aspect in radians
    aspect = np.arctan2(-dx, dy) # not np.arctan2(-dy, dx) as suggested by ChatGPT!
    # Precomputed DEM-based values for topographic correction
    aspect_rad2d[resolution['S2']] = np.mod(aspect, 2 * np.pi) #aspect_ENVI # 
    cos_slope2d[resolution['S2']] = np.cos(slope)
    sin_slope2d[resolution['S2']] = np.sin(slope)

In [ ]:
def topo_correction_np(img, res, solar_zenith_deg, solar_azimuth_deg):
    mask = img[0] != 0 # & (dem != 0)

    img_masked = img[:, mask]

    # Convert solar zenith and azimuth from degrees to radians
    solar_zenith_rad = np.deg2rad(solar_zenith_deg)
    solar_azimuth_rad = np.deg2rad(solar_azimuth_deg)

    # Compute illumination factors
    cos_zenith = np.cos(solar_zenith_rad)
    sin_zenith = np.sin(solar_zenith_rad)

    # Illumination calculation
    cos_slope_x_cosZ_2d = cos_slope2d[res][mask] * cos_zenith
    illumination_azimuth_2d = sin_slope2d[res][mask] * sin_zenith * np.cos(solar_azimuth_rad - aspect_rad2d[res][mask])
    illumination2d = cos_slope_x_cosZ_2d + illumination_azimuth_2d

    # Clip illumination (cos(i)) to a minimum of 0.0 to avoid instability
    illumination2d_gte0 = np.maximum(illumination2d, 0.0) # added on 2025-06-24

    # Add a column of ones to account for the intercept
    illumination_with_intercept = np.vstack([illumination2d, np.ones_like(illumination2d)]).T

    def topo1band(b):
        slope, intercept = np.linalg.lstsq(illumination_with_intercept, img_masked[b], rcond=None)[0]
        coef = intercept / slope if slope != 0 else 0
        img[b][mask] = (img_masked[b] * (cos_slope_x_cosZ_2d + coef)) / (illumination2d_gte0 + coef)

    # Apply topographic correction to all bands simultaneously
    with parallel_backend('threading'):
        Parallel(n_jobs=-1)(delayed(topo1band)(b) for b in range(img.shape[0]))

    return img

def topo_correction_np_120m(img, res, solar_zenith_deg, solar_azimuth_deg):
    mask = img[0] != 0 # & (dem != 0)
    img_data = img[:,mask]

    # reduce the resolution to 120m and then increase back to the original resolution to blur the image
    img_blurred = resize_custom(img, 12) if res == 10 else resize_custom(img, 6)
    # img_blurred = zoom(img_blurred,0.125 if res == 10 else 0.25, order = 0) 
    img_blurred_mask = img_blurred[0] != 0
    img_blurred_data = img_blurred[:, img_blurred_mask]

    # Convert solar zenith and azimuth from degrees to radians
    solar_zenith_rad = np.deg2rad(solar_zenith_deg)
    solar_azimuth_rad = np.deg2rad(solar_azimuth_deg)

    # Compute illumination factors
    cos_zenith = np.cos(solar_zenith_rad)
    sin_zenith = np.sin(solar_zenith_rad)

    # Illumination calculation # here! very inefficient! calculate at 80m!! or not??
    cos_slope_x_cosZ_2d = cos_slope2d[res][mask] * cos_zenith
    illumination_azimuth_2d = sin_slope2d[res][mask] * sin_zenith * np.cos(solar_azimuth_rad - aspect_rad2d[res][mask])
    illumination2d = cos_slope_x_cosZ_2d + illumination_azimuth_2d

    # Add a column of ones to account for the intercept
    illumination2d_blurred = np.zeros((1,img.shape[1],img.shape[2]), illumination2d.dtype)
    illumination2d_blurred[:,mask] = illumination2d

    illumination2d_blurred = resize_custom(illumination2d_blurred, 12) if res == 10 else resize_custom(illumination2d_blurred, 6)

    illumination2d_blurred = illumination2d_blurred[0,img_blurred_mask]
    illumination_with_intercept = np.vstack([illumination2d_blurred, np.ones_like(illumination2d_blurred)]).T

    # Clip illumination (cos(i)) to a minimum of 0.0 to avoid instability
    illumination2d_gte0 = np.maximum(illumination2d, 0.0) # added on 2025-06-24

    def topo1band(b):
        slope, intercept = np.linalg.lstsq(illumination_with_intercept, img_blurred_data[b], rcond=None)[0]
        coef = intercept / slope if slope != 0 else 0
        img[b][mask] = (img_data[b] * (cos_slope_x_cosZ_2d + coef)) / (illumination2d_gte0 + coef)

    # Apply topographic correction to all bands simultaneously
    with parallel_backend('threading'):
        Parallel(n_jobs=-1)(delayed(topo1band)(b) for b in range(img.shape[0]))

    return img

# Example usage:
# image: Sentinel-2 image as numpy array (n_bands, n_lines, n_rows)
# dem: DEM as numpy array (n_lines, n_rows)
# solar_zenith_deg: solar zenith angle in degrees
# solar_azimuth_deg: solar azimuth angle in degrees

In [ ]:
centroid_esa = "{:.2f} {:.2f}".format(S2_tile_Centroid[0], S2_tile_Centroid[1]) # this prints like "-125.27 48.95" (long and lat of the center of the tile)

def get_access_token(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
    }
    try:
        r = requests.post(
            "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
            data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Access token creation failed. Reponse from the server was: {r.json()}"
        )

    headers = {"Authorization": f"Bearer {r.json()["access_token"]}"}
    # global session_esa
    session_esa = requests.Session()
    session_esa.headers.update(headers)
    return session_esa

# Dictionary with band names and file patterns
bands_S2_esa = {
    "B": "B02",
    "G": "B03",
    "R": "B04",
    "RE1": "B05",
    "RE2": "B06",
    "RE3": "B07",
    "NIR": "B8A",
    "NIR_wide": "B08",
    "SWIR1": "B11",
    "SWIR2": "B12",
    "SCL": "SCL",
}

esa_10m_folder = ''
if sr: # for SR S2 data
    esa_10m_folder = 'Nodes(R10m)/' if download_S2_10m else 'Nodes(R20m)/' # SR images on the Copernicus Hub contain both the original 10m bands and 10m bands resampled to 20m, so if download_S2_10m is False, then the 20m bands will be downloaded instead of 10m bands

esa_20m_folder = 'Nodes(R20m)/' if sr else ''

bands_S2_esa_10m = bands['S2_10m'].copy() # do not remove .copy()
bands_S2_esa_20m = bands['S2_20m'].copy()

if not download_S2_10m and 'NIR' in bands_S2_esa_20m:
    bands_S2_esa_10m.remove('NIR_wide')
if sr:
    bands_S2_esa_20m.append('SCL')

session_esa = None

def download_S2_esa(s2_date):
    global session_esa

    if session_esa == None:
        session_esa = get_access_token(username_esa, password_esa)

    for attempt in range(5):
        json = requests.get(f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq 'SENTINEL-2' and OData.CSC.Intersects(area=geography'SRID=4326;POINT({centroid_esa})') and ContentDate/Start gt {s2_date}T00:00:00.000Z and ContentDate/Start lt {s2_date}T23:59:59.999Z").json()
        try:
            json_df = pd.DataFrame.from_dict(json['value'])
            break
        except:
            print('json not responding')
            session_esa = get_access_token(username_esa, password_esa) # required?
            time.sleep(5)  
              
    if json_df.empty:
        print("json request failed for", s2_date,"; skipping...")
        return 0, np.array([]), np.array([])

    json_df = json_df[json_df['Name'].str.contains(MGRS_tile) & json_df['Name'].str.contains("MSIL2A" if sr else "MSIL1C")] # select correct UTM zone and processing level (2A or 1C)
    # Get the record with the most recent ModificationDate (i.e., the best preprocessing - newest processing baseline)
    best_preprocessing = json_df.loc[json_df['ModificationDate'].idxmax()] # select the newest copy of the image (i.e., with the best preprocessing)
    # Extract the Id
    S2_with_best_preprocessing_id = best_preprocessing['Id']
    print(best_preprocessing['Name'])

    # Initialize lists to store the NumPy arrays
    array_10m = []
    array_20m = []

    # Step 1: Fetch intermediate_node_1
    response = session_esa.get(f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({S2_with_best_preprocessing_id})/Nodes", stream=True)

    if response.status_code == 200:
        intermediate_node_1 = response.json()['result'][0]['Name'] # assumes that there is only one folder inside the zip archive
        response = session_esa.get(f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({S2_with_best_preprocessing_id})/Nodes({intermediate_node_1})/Nodes(GRANULE)/Nodes", stream=True)
        intermediate_node_2 = response.json()['result'][0]['Name'] # assumes that there is only one folder inside the folder 'GRANULE'

        # Do 10m bands
        esa_s2image_url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({S2_with_best_preprocessing_id})/Nodes({intermediate_node_1})/Nodes(GRANULE)/Nodes({intermediate_node_2})/Nodes(IMG_DATA)/{esa_10m_folder}"
        response = session_esa.get(f"{esa_s2image_url}Nodes", stream=True)
        nodes = response.json()['result']

        def download_S2_band_esa(band):
            global session_esa
            for node in nodes:
                if bands_S2_esa[band] in node['Name']:
                    final_node = node['Name']
                    response = session_esa.get(f"{esa_s2image_url}Nodes({final_node})/$value", stream=True)

                    for attempt in range(10):
                        try:
                            with rasterio.open(BytesIO(response.content)) as src:
                                print(band, end=', ')
                                return src.read(1) # Read the first (and only) band
                        except:
                            print('error - redownloading',band)
                            session_esa = get_access_token(username_esa, password_esa) # reconnect to ESA
                            response = session_esa.get(f"{esa_s2image_url}Nodes({final_node})/$value", stream=True)
                            time.sleep(3*attempt) # wait up to 30 seconds before retrying

        with parallel_backend('threading'): # Parallel backend for threading # this doesn't seem to speed up downloading unfortunately :(
            array_10m = Parallel()(delayed(download_S2_band_esa)(band) for band in bands_S2_esa_10m)

        # Do 20m bands
        esa_s2image_url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({S2_with_best_preprocessing_id})/Nodes({intermediate_node_1})/Nodes(GRANULE)/Nodes({intermediate_node_2})/Nodes(IMG_DATA)/{esa_20m_folder}"
        response = session_esa.get(f"{esa_s2image_url}Nodes", stream=True)
        nodes = response.json()['result']
        with parallel_backend('threading'): # Parallel backend for threading
            array_20m = Parallel()(delayed(download_S2_band_esa)(band) for band in bands_S2_esa_20m) # here! parallelize array_10m and array_20m

        # Convert lists to NumPy arrays
        array_10m = np.array(array_10m)  # Shape: (3, height, width)
        array_20m = np.array(array_20m)  # Shape: (7, height, width)
    else:
        print(f"Failed to download. Status code: {response.status_code}, Response: {response.text}")
        session_esa = get_access_token(username_esa, password_esa)
        
    return response.status_code, array_10m, array_20m

## Select Sentinel-2 data and download from GEE to PC

In [ ]:
sr_geeS2 = sr
if download_ESA_S2:
  sr_geeS2 = False # because GEE does not have SR S2 for 2016 and much of 2017 but ESA does. GEE is used to find images and create Cloud Score+ masks, so should use TOA for GEE and SR for ESA (if sr = True)

if do_S2 and end_year >= 2016:
  # here! with SR data, try filtering by HIGH_PROBA_CLOUDS_PERCENTAGE, MEDIUM_PROBA_CLOUDS_PERCENTAGE, CLOUD_COVERAGE_ASSESSMENT, CLOUDY_SHADOW_PERCENTAGE, CLOUDY_PIXEL_PERCENTAGE
  if sr_geeS2 == True and not download_ESA_S2: # GEE does not have 2016 and 2017 SR S2 images, but ESA does!
    S2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filter(ee.Filter.calendarRange(start_year,end_year,'year')).filter(ee.Filter.calendarRange(start_DOY,end_DOY,'day_of_year')).filterMetadata('MGRS_TILE', 'equals', MGRS_tile).filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cloudiness_s2_max)).select(['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12','SCL','MSK_SNWPRB'],['B1','B','G','R','RE1','RE2','RE3','NIR_wide','NIR', 'SWIR1', 'SWIR2','SCL','MSK_SNWPRB']).linkCollection(ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED'), [s2_cloud_mask_type]) #.distinct('system:time_start')
  else:
    S2_collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filter(ee.Filter.calendarRange(start_year,end_year,'year')).filter(ee.Filter.calendarRange(start_DOY,end_DOY,'day_of_year')).filterMetadata('MGRS_TILE', 'equals', MGRS_tile).filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cloudiness_s2_max)).select(['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12'],['B1','B','G','R','RE1','RE2','RE3','NIR_wide','NIR', 'SWIR1', 'SWIR2']).linkCollection(ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED'), [s2_cloud_mask_type]) #.distinct('system:time_start')

  start_year_S2 = start_year if start_year > 2016 else 2016

  start = time.time()
  t_previous = start

  i_s2 = 0

  for year in range(start_year_S2, end_year + 1):  
    yearly_collection = S2_collection.filter(ee.Filter.calendarRange(year, year, 'year'))
    n_images_year = yearly_collection.size().getInfo()

    if n_images_year == 0:
      continue

    sensor = 'S2'
    cloudiness_threshold = cloudiness_s2_min # e.g. 30

    yearly_collection = yearly_collection.map(s2csPlus_cloudmasker_sr) if sr_geeS2 == True and not download_ESA_S2 else yearly_collection.map(s2csPlus_cloudmasker_toa)

    clear_area = 0
    col_size_previous = 0
    while cloudiness_threshold <= cloudiness_s2_max and clear_area/n_not_water_max_client < min_coverage:
      n_clear_px_threshold = (100-cloudiness_threshold)*0.01*n_not_water_max_client
      yearly_collection_filtered = yearly_collection.filter(ee.Filter.gt('n_not_cloud_or_water', n_clear_px_threshold))

      col_size = yearly_collection_filtered.size().getInfo()
      if col_size == col_size_previous: # if no images or no additional images found, increase (relax) the cloudiness threshold
        print(year, 'no images at cloudiness',cloudiness_threshold) if col_size == 0 else print('no additional images at cloudiness',cloudiness_threshold)
        cloudiness_threshold += 10
        continue

      col_size_previous = col_size
      notCloudOrWater_yearly = yearly_collection_filtered.select('SWIR2').max().gt(0) # no need to .updateMask(water_and_agri_mask) because water_and_agri_mask is applied per image (which might not be very efficient)
      notCloudOrWater_yearly = notCloudOrWater_yearly.updateMask(notCloudOrWater_yearly).rename('notCloudOrWater_yearly')

      clear_area = notCloudOrWater_yearly.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=S2_tile['S2'],
        crs = S2_projection,
        scale=reducer_resolution,
        maxPixels=1e9,
        bestEffort=True,
        tileScale=16
      ).getNumber('notCloudOrWater_yearly').getInfo()

      print(
          f"{year} | cloudiness threshold: {cloudiness_threshold} | "
          f"clear area: {100.0 * clear_area / n_not_water_max_client:.2f}% | "
          f"number of images: {col_size}"
      )

      if clear_area/n_not_water_max_client >= min_coverage:
        print('done')

      cloudiness_threshold += 10

    ## Note that GEE does not replace old S2 images when they are reprocessed based on a newer 'PROCESSING_BASELINE' by ESA, so the geometric accuracy of S2 on GEE is very inconsistent. Use https://browser.dataspace.copernicus.eu/ to download reprocessed images
    # print(S2_collection.aggregate_array('DATATAKE_IDENTIFIER').getInfo())
    # print(S2_collection.aggregate_array('PROCESSING_BASELINE').getInfo())
    ## prints ['02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14', '02.14'] for both SR and TOA whereas much newer preprocessing is available on the ESA Copernicus Hub, so download S2 from ESA by setting download_ESA_S2 to True

    # log in to ESA Copernicus website
    if download_ESA_S2:
      access_token = get_access_token(username_esa, password_esa)
      headers = {"Authorization": f"Bearer {access_token}"}
      session = requests.Session()
      session.headers.update(headers)

    s2_collection_filtered = yearly_collection_filtered.sort('cloudfree_proportion', False)

    if do_topo_s2 and download_GEE_S2:
      print('Topographic correction applied')
      s2_collection_filtered = s2_collection_filtered.map(illumination_s2)

    if do_coregister_S2: # here! before or after topo? Topo effects can make coregistration harder but topo correction maybe inaccurate if applied before coregistration because topocorrection uses DEM, though it's unlikely because geometric error in S2 is small compared to DEM resolution
      print('Co-registration with Landsat applied')
      s2_collection_filtered = s2_collection_filtered.map(coregister_S2)

    if download_HLS_S2:
      datetime_S2 = s2_collection_filtered.aggregate_array('DATATAKE_IDENTIFIER').getInfo()
      datetime_S2_HLS = []
      for d in datetime_S2:
        # Convert the date string to a datetime object
        date_obj = datetime.strptime(d[5:13], "%Y%m%d")
        # Extract the year and day of the year from the datetime object
        year = date_obj.strftime("%Y")
        day_of_year = date_obj.strftime("%j")
        # Concatenate the year and day of the year and convert to an integer
        datetime_S2_HLS.append(year + day_of_year + d[13:20]) 

    date_ids = s2_collection_filtered.aggregate_array("date").getInfo() # img.date().format("YYYY-MM-dd") # ('DATATAKE_IDENTIFIER')
    print('date_ids',date_ids)

    if do_topo_s2 and download_ESA_S2:
      sun_zeniths = s2_collection_filtered.aggregate_array("MEAN_SOLAR_ZENITH_ANGLE").getInfo()
      sun_azimuths = s2_collection_filtered.aggregate_array("MEAN_SOLAR_AZIMUTH_ANGLE").getInfo()
      print('zeniths:', sun_zeniths)
      print('azimuths:', sun_azimuths)

    short_names = [date + '_S2' for date in date_ids]

    n_images_ = len(short_names)

    collectionList = s2_collection_filtered.toList(n_images_)

    for i in range(n_images_):
      i_s2 += 1

      if download_HLS_S2 and (
        (not os.path.exists(f"{MGRS_tile}_tif/{short_names[i]}_HLS.tif") and save_tif) or
        (not os.path.exists(f"{MGRS_tile}/{short_names[i]}_HLS_data.npy") and save_np) or
        (not skip_existing)
      ):
        downloaded_or_not = download_HLS(datetime_S2_HLS[i], short_names[i]) # 2023137T233821
        if not downloaded_or_not:
          downloaded_or_not = download_HLS(datetime_S2_HLS[i], short_names[i]) # here! only two attempts and without wait?
          if not downloaded_or_not:
            print('Failed to download HLS data for', short_names[i])  

      if (not download_ESA_S2 and not download_GEE_S2):
        continue
      if skip_existing:
        tif_ok = (not save_tif) or os.path.exists(MGRS_tile + '_tif/' + short_names[i] + '.tif')
        np_ok  = (not save_np)  or os.path.exists(MGRS_tile + '/' + short_names[i] + '_data.npy')

        if (tif_ok and np_ok):
          print(f"{short_names[i]} already downloaded. Skipping...")
          continue

      t_previous = time.time()

      if download_ESA_S2:
        esa_download_code, s2_10m_esa, s2_20m_esa = download_S2_esa(date_ids[i])

        if esa_download_code == 0:
          continue

        if esa_download_code != 200: # if authorization token expired esa_download_code == 401
            print('esa_download_code', esa_download_code, '- retrying')
            esa_download_code, s2_10m_esa, s2_20m_esa = download_S2_esa(date_ids[i]) # only two attempts to download ESA S2 data?
        if esa_download_code == 200: # do not replace with elif!
            print(f"ESA data downloaded: {round(time.time() - t_previous)}s")

            if download_S2_10m or not sr:
              s2_image_np_10m = s2_10m_esa[:,:h['S2_10m'],:w['S2_10m']]
            else:
              s2_image_np_10m = s2_10m_esa[:,:h['S2'],:w['S2']]
            s2_image_np_20m = s2_20m_esa[:-1,:h['S2'],:w['S2']] if sr else s2_20m_esa[:,:h['S2'],:w['S2']]

            # here! create a snow mask directly from the ESA image instead of downloading from GEE!
            if mask_esa_s2_with_gee: # because Cloud Score+ masks are better than ESA cloud masks
              gee_image = ee.Image(collectionList.get(i))
              s2_gee_mask = gee2np(gee_image,['SWIR1','snow'],sensor,'',4) # here! why process image in GEE just for the Cloud Score+ mask? get it directly from the Cloud Score+ collection
              s2_snow_mask = cloud_bufferer(s2_gee_mask[-1], round(cloud_buffer_m/resolution[sensor]))
              s2_snow_mask[s2_gee_mask[0] == 0] = 0 # here! if sr = True, s2_gee_mask[0] contains GEE SCL water mask, which can have low geometric accuracy!
              s2_gee_mask = ~s2_snow_mask # clean this up
              s2_image_np_20m[:,s2_gee_mask] = 0
              
              if download_S2_10m or not sr: # need 'not sr_esa' because ESA does not have 20m copies of 10m L1C S2 bands
                s2_gee_mask = shift_and_resize(s2_gee_mask,resolution[sensor], resolution[hd_sensors[sensor]], shift_to_hd_y[sensor], shift_to_hd_x[sensor], 0)
                s2_image_np_10m[:,s2_gee_mask] = 0
              
              del s2_gee_mask # s2_snow_mask

            if sr:
              s2_mask = s2_20m_esa[-1,:h['S2'],:w['S2']]
              mask_values = [1,3,6,8,9,10,11] # here! should add 2=dark pixels? # 1: Saturated or defective pixel; 2: Dark features (e.g., shadows); 3: Cloud shadows; 4: Vegetation; 5: Not-vegetated; 6: Water; 7: Unclassified; 8: Cloud medium probability; 9: Cloud high probability; 10: Thin cirrus; 11: Snow or ice
              s2_mask = np.isin(s2_mask, mask_values)
              s2_image_np_20m[:,s2_mask] = 0

              if download_S2_10m:
                s2_mask = shift_and_resize(s2_mask,resolution[sensor], resolution[hd_sensors[sensor]], shift_to_hd_y[sensor], shift_to_hd_x[sensor], 0)
                s2_image_np_10m[:,s2_mask] = 0

            s2_image_np_20m[:,water_and_agri_mask_np['S2'] == False] = 0 

            if download_S2_10m or not sr:
              s2_image_np_10m[:,water_and_agri_mask_np['S2_10m'] == False] = 0 

            if do_topo_s2:
              s2_image_np_10m = topo_correction_np_120m(s2_image_np_10m, 10, sun_zeniths[i], sun_azimuths[i]) # here! need to optimize for download_S2_10m = False
              s2_image_np_20m = topo_correction_np_120m(s2_image_np_20m, 20, sun_zeniths[i], sun_azimuths[i]) # here! try topo_correction_np() instead of topo_correction_np_120m()

            if download_S2_10m or not sr: # resample 10m bands to 20m resolution and stack with 20m bands # Copernicus S2 SR data already includes 10m bands resampled to 20m # optimize
              if 'NIR_wide' in bands['S2_10m'] and 'NIR' in bands['S2_20m']:
                s2_image_np_10m_noNIR = s2_image_np_10m[[band != 'NIR_wide' for band in bands['S2_10m']]]
                s2_image_np = np.concatenate((resize_10to20m(s2_image_np_10m_noNIR), s2_image_np_20m)) # resample 10m bands to 20m resolution and stack with 20m bands
              else:
                s2_image_np = np.concatenate((resize_10to20m(s2_image_np_10m), s2_image_np_20m))
            else:
              s2_image_np = np.concatenate(s2_image_np_10m, s2_image_np_20m)

            s2_image_np[:,np.any(s2_image_np == 0, axis=0)] = 0

            save_data_and_mask(s2_image_np, short_names[i],'S2',bands[sensor],wavelengths_all['_S2']) # save 10m and 20m bands together as a 20m image

            if download_S2_10m: # save 10m bands separately as a 10m image
              s2_image_np_10m[:,np.any(s2_image_np_10m == 0, axis=0)] = 0

              save_data_and_mask(s2_image_np_10m, short_names[i]+'_10m','S2_10m',bands['S2_10m'],wavelengths_all['_S2'])

        t_now = time.time()
        print(f"image {i+1} of {n_images_} processed: {round(t_now - t_previous)}s; total: {round(t_now - start)}s\n")
        continue

      ###### if download_ESA_S2 = False, then download S2 from GEE instead of the Copernicus Hub ######
      print('Downloading from GEE:',short_names[i])

      gee_image = ee.Image(collectionList.get(i))

      s2_image_np_10m = ''
      s2_image_np = ''

      if download_S2_10m: # download 10m and 20m bands separately
        # Downloading 10m data may cause "EEException: Too Many Requests: Request was rejected because the request rate or concurrency limit was exceeded." 16 tiles x 4 bands = 64 concurrent requests
        # or "ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))"
        # so might have to retry multiple times
        # these error are caught inside gee2np, but if gee2np still fails, we also retry to download here
        s2_image_np_10m = ''
        for attempt in range(10):
          try:
            s2_image_np_10m = gee2np(gee_image,bands['S2_10m'],'S2_10m','',16)
          except:
            print('Ouch! Have to redownload 10m bands due to error.')
            s2_image_np_10m = ''
          if isinstance(s2_image_np_10m, np.ndarray): # is this required?
            break

        for attempt in range(5):
          try:
            s2_image_np_20m = gee2np(gee_image,bands['S2_20m']+['snow'],sensor,'',4) # do not set to 16! Will crash 'EEException: Too Many Requests: Request was rejected because the request rate or concurrency limit was exceeded.'
          except:
            print('Ouch! Have to redownload 20m bands due to error.')
            s2_image_np_20m = ''
          if isinstance(s2_image_np_20m, np.ndarray): # is this required?
            break

        if 'NIR_wide' in bands['S2_10m'] and 'NIR' in bands['S2_20m']:
          s2_image_np_10m_noNIR = s2_image_np_10m[[band != 'NIR_wide' for band in bands['S2_10m']]]
          s2_image_np = np.concatenate((resize_10to20m(s2_image_np_10m_noNIR), s2_image_np_20m))
        else:
          s2_image_np = np.concatenate((resize_10to20m(s2_image_np_10m), s2_image_np_20m))
      else: # download 10m and 20m together at 20m resolution if download_S2_10m = False
        s2_image_np = gee2np(gee_image,bands[sensor]+['snow'],sensor,'',4) # ['G', 'R', 'RE1', 'RE2','RE3', 'NIR', 'SWIR1', 'SWIR2','cloud_probability']

      # add a buffer to snow masks
      snow_mask = s2_image_np[-1]
      s2_image_np = s2_image_np[:-1]
      snow_mask = cloud_bufferer(snow_mask, round(cloud_buffer_m/resolution[sensor]))
      s2_image_np[:,snow_mask==0] = 0

      print(date_ids[i],'downloaded (',i_s2,"of",n_images_,")", time.time() - t_previous)

      if download_S2_10m:
        snow_mask = shift_and_resize(snow_mask,resolution[sensor], resolution[hd_sensors[sensor]], shift_to_hd_y[sensor], shift_to_hd_x[sensor], 0)
        s2_image_np_10m[:,snow_mask==0] = 0

        if align_S2:
          s2_pan = (s2_image_np_10m[1]+s2_image_np_10m[2])/2.0 # combine green and red bands to match the relative spectral response of the Landsat 8 - 9 panchromatic band # no need to apply water_and_agri_mask_np[10] because water and agri can halp with alignment, so do not mask them out

          # can just use the green band, almost as good as green+red and better than red, see "2023-05-30 Correlation between Landsat 8 pan and green red bands of Sentinel-2.txt" for comparison results
          y_shift_10m, x_shift_10m = align_10m_S2_and_Lpan(s2_pan)
          print('y_shift_10m, x_shift_10m',y_shift_10m, x_shift_10m)
          y_shift_20m = round(y_shift_10m/2)
          x_shift_20m = round(x_shift_10m/2)
          print('y_shift_20m, x_shift_20m',y_shift_20m, x_shift_20m)

          # shift the image
          if y_shift_10m != 0 or x_shift_10m != 0:
            s2_image_np_10m = np.pad(s2_image_np_10m, ((0, 0), (max_shift_s2_10m, max_shift_s2_10m), (max_shift_s2_10m, max_shift_s2_10m)), mode='constant', constant_values=0) # add some zero-valued pixels on each side of the image to prevent rollover when doing np.roll
            s2_image_np_10m = np.roll(s2_image_np_10m, (y_shift_10m, x_shift_10m), axis=(1, 2))
            s2_image_np_10m = s2_image_np_10m[:,max_shift_s2_10m:max_shift_s2_10m+h[10], max_shift_s2_10m:max_shift_s2_10m+w[10]]  # Crop back to original shape

          if y_shift_20m != 0 or x_shift_20m != 0:
            s2_image_np = np.pad(s2_image_np, ((0, 0), (max_shift_s2_10m, max_shift_s2_10m), (max_shift_s2_10m, max_shift_s2_10m)), mode='constant', constant_values=0) # add some zero-valued pixels on each side of the image to prevent rollover when doing np.roll
            s2_image_np = np.roll(s2_image_np, (y_shift_20m, x_shift_20m), axis=(1, 2))
            s2_image_np = s2_image_np[:,max_shift_s2_10m:max_shift_s2_10m+h[20], max_shift_s2_10m:max_shift_s2_10m+w[20]]  # Crop back to original shape

        s2_image_np_10m[:,water_and_agri_mask_np['S2_10m'] == False] = 0 # here! invert the mask at the beginning, when creating it

      s2_image_np[:,water_and_agri_mask_np['S2'] == False] = 0 
      s2_image_np[:,np.any(s2_image_np == 0, axis=0)] = 0
      save_data_and_mask(s2_image_np, short_names[i],'S2',bands[sensor],wavelengths_all['_S2'])

      if download_S2_10m:
        s2_image_np_10m[:,np.any(s2_image_np_10m == 0, axis=0)] = 0
        save_data_and_mask(s2_image_np_10m, short_names[i]+'_10m','S2_10m',bands['S2_10m'],wavelengths_all['_S2'])

      t_now = time.time()
      print(f"image {i+1} of {n_images_} processed: {round(t_now - t_previous)}s; total: {round(t_now - start)}s\n")

In [ ]:
def bilinear_60m(img):
    return img.resample('bilinear').reproject(crs=S2_projection, crsTransform = transform['M'])

def bicubic_60m(img):
    return img.resample('bicubic').reproject(crs=S2_projection, crsTransform = transform['M'])

def bicubic_30m(img):
    return img.resample('bicubic').reproject(crs=S2_projection, crsTransform = transform['M_30m'])

## Find the best, most cloud-free Landsat 5 TM image to which MSS data will be coregistered

In [ ]:
landsat5_best = ''
landsat5_best_np = ''
landsat5_best_watermask = ''
landsat5_best_trimmed = ''
landsat5_best_trimmed_0where = ''
landsat5_best_sensor = ''
landsat5_best_date = ''
landsat5_best_clearsky = 0
sensor = 'L'

def L_masking_simple(img): # mask non-clear pixels in the reference TM image to be used for MSS coregistration
    img = img.clip(S2_tile['GEE'])
    cloud_mask = img.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).unmask().eq(0)
    return img.select('B4').updateMask(cloud_mask).resample('bicubic')

def set_bicubic(img):
    return img.resample('bicubic') # cloud and snow masking may produce slightly better composites but causes GEE memory crash 

if do_MSS and start_year <= 1993: # reference L5 image is required regardless coregistration settings
    if use_TM_composite: # this may crash GEE (too many operations)
        landsat_collection = ee.ImageCollection('LANDSAT/LT05/C02/T1').filterBounds(S2_tile['M']).filter(ee.Filter.calendarRange(1984, 1993,'year')).filter(ee.Filter.calendarRange(start_DOY_reference,end_DOY_reference,'day_of_year')).filter(ee.Filter.lt('CLOUD_COVER', 30)).select(['B4','QA_PIXEL'])   
        landsat_collection = landsat_collection.sort('CLOUD_COVER').limit(20)
        # landsat9_best_pan = ee.Algorithms.Landsat.simpleComposite(landsat_collection) # 2m 44s vs 55s for median without masking and 2m 27s for median with cloud and snow masking and 1m45s with cloud masking
        landsat_collection = landsat_collection.map(set_bicubic) # clipping and cloud masking don't help
        landsat5_best = landsat_collection.select(['B4']).median().uint16().rename('NIR')
    else: # this may not cover the entire tile if the tile is located between two Landsat paths
        # Step 1: Filter the DataFrame to include only records where 'year' is less than 1990
        filtered_df = yearly_best_L[yearly_best_L['year'] <= 1993]
        landsat_collection = ''
        # Step 3: Retrieve the 'date_str' and 'sensor' values for the most cloud-free pre-1994 Landsat TM image
        if len(filtered_df) >= 5: # if at least 5 years of Landsat data (1982-1993) have been processed when downloading Landsat 30m data:
            index_with_highest_clearsky = filtered_df['clearsky'].idxmax() # Find the index of the row with the highest 'clearsky' value
            landsat30m_best_date = filtered_df.loc[index_with_highest_clearsky, 'date_str']
            landsat30m_best_sensor = filtered_df.loc[index_with_highest_clearsky, 'sensor']
            landsat5_best_clearsky = filtered_df.loc[index_with_highest_clearsky, 'clearsky']
            print('Best 30m Landsat:', landsat30m_best_date+landsat30m_best_sensor,landsat5_best_clearsky,'%')
        else: # if do_L = False # Here! Sometimes this cell triggers this error: "EEException: User memory limit exceeded". Might need to process 2 years at a time or free up GEE memory somehow
            # Do not filter by projection (i.e., same UTM zone as S2), because pre-mosaiced images can be from different UTM zones
            landsat30m_best_date, landsat30m_best_sensor = find_best_image(4,5)

        landsat_collection = ee.ImageCollection(collection_names_raw[5 if landsat30m_best_sensor == '_L5' else 4]).filter(ee.Filter.eq('DATE_ACQUIRED', landsat30m_best_date)).filterBounds(S2_tile['M'])
        landsat_collection = landsat_collection.map(L_masking_simple) # make sure to resample using bilinear or bicubic convolution in L_masking_simple because the default is nearest neighbour (NN will simply shift pixels, reducing positional accuracy). Resample before mosaicing because bicubic and bilinear doesn't work on mosaics

        # if there are two images for this data, mosaic them together
        landsat5_best = ee.Algorithms.If(ee.Algorithms.IsEqual(landsat_collection.size(), ee.Number(1)), landsat_collection.first().select('B4'), landsat_collection.select('B4').qualityMosaic('B4')) # output of this is a 'ComputedObject', so have to convert it to ee.Image
        landsat5_best = ee.Image(landsat5_best).uint8().rename('NIR') # convert compute object to gee image

    landsat5_best = landsat5_best.reproject(crs=S2_projection, crsTransform = transform['M_30m']).clip(S2_tile['GEE'])
    
    # here! DN = 15 is too high a threshold for water masking for rivers with lots of sediment!
    landsat5_best_watermask = landsat5_best.select('NIR').lt(15).rename('landsat5_best_watermask') # This is not robust as these are DN. Using DN = 15 results in some shadowed areas labelled as water, but it misses water with lots of sediment. So it might make sense to use topo-corrected (and SR?) reference image instead
    landsat5_best_watermask_np = gee2np(landsat5_best_watermask.reproject(crs=S2_projection, crsTransform = transform['M']),'landsat5_best_watermask',"M",'Landsat5_composite_watermask' if use_TM_composite else landsat30m_best_date+landsat30m_best_sensor+'_best_watermask',16) # here! much better mask without bilinear interpolation! # here! get this mask client side intead!
    
    # Check if water > 1%
    if np.count_nonzero(landsat5_best_watermask_np) > 0.01 * landsat5_best_watermask_np.size:
        check_MSStoTM_water_alignment = True
        print("Water > 1%")
    else:
        check_MSStoTM_water_alignment = False
        print("Water <= 1%. TSD won't be able to use water bodies to filter out incorrectly georeferenced MSS images.")

    # landsat5_best = landsat5_best.updateMask(water_and_agri_mask) # here! should only mask out agriculture (due to crop rotation those areas will have low correlation with MSS images). Keeping water would probably improve MSS-to-TM alignment
    landsat5_best_np = gee2np(landsat5_best.reproject(crs=S2_projection, crsTransform = transform['M_30m']),'NIR',"M_30m", 'Landsat5_composite_for_MSS_coregistration' if use_TM_composite else landsat30m_best_date+landsat30m_best_sensor+'_best_for_MSS_30m',16) 

    landsat5_best_trimmed = landsat5_best_np[max_shift_s2_10m:h[60]-max_shift_s2_10m, max_shift_s2_10m:w[60]-max_shift_s2_10m] # can delete landsat5_best_np after use
    landsat5_best_trimmed_0where = landsat5_best_trimmed <= 0 

## MSS GEE functions

In [ ]:
sensor = 'M'

cloud_buffer = round(cloud_buffer_m/resolution[sensor]) # convert cloud buffer from meters to pixels
cloud_buffer_server = ee.Number(cloud_buffer) # conversion not required. Delete
cloud_height_mss = ee.Number(int(cloud_height_m/60))

NIR2_MSS_dark_threshold_mss_server = ee.Number(NIR2_MSS_dark_threshold) # NIR surface reflectance of dense forest is ~12%
min_nondark = ee.Number(0.8)

def rename_coef_MSS_4to5(img):
  return img.set('date_str', img.date().format("YYYY-MM-dd")).set('REFLECTANCE_MULT_BAND_NIR1',ee.Number(img.get('REFLECTANCE_MULT_BAND_3'))).set('REFLECTANCE_MULT_BAND_NIR2',ee.Number(img.get('REFLECTANCE_MULT_BAND_4'))).set('REFLECTANCE_MULT_BAND_G',ee.Number(img.get('REFLECTANCE_MULT_BAND_1'))).set('REFLECTANCE_ADD_BAND_G',ee.Number(img.get('REFLECTANCE_ADD_BAND_1'))).set('REFLECTANCE_ADD_BAND_NIR1',ee.Number(img.get('REFLECTANCE_ADD_BAND_3'))).set('REFLECTANCE_ADD_BAND_NIR2',ee.Number(img.get('REFLECTANCE_ADD_BAND_4')))

def rename_coef_MSS_1to3(img):
  return img.set('date_str', img.date().format("YYYY-MM-dd")).set('REFLECTANCE_MULT_BAND_NIR1',ee.Number(img.get('REFLECTANCE_MULT_BAND_6'))).set('REFLECTANCE_MULT_BAND_NIR2',ee.Number(img.get('REFLECTANCE_MULT_BAND_7'))).set('REFLECTANCE_MULT_BAND_G',ee.Number(img.get('REFLECTANCE_MULT_BAND_4'))).set('REFLECTANCE_ADD_BAND_G',ee.Number(img.get('REFLECTANCE_ADD_BAND_4'))).set('REFLECTANCE_ADD_BAND_NIR1',ee.Number(img.get('REFLECTANCE_ADD_BAND_6'))).set('REFLECTANCE_ADD_BAND_NIR2',ee.Number(img.get('REFLECTANCE_ADD_BAND_7')))

def rename_coef_MSS_4to5_2012(img): # ugly first solution. 2012 MSS data lack NIR2, so replace invalid NIR2 band and the corresponding coefficients with NIR1 band and its coefficients for masking purposes
  # clone Band 3 data into Band 4's position
  nir1 = img.select('NIR1')
  nir2_replaced = nir1.rename('NIR2') # Rename to match Band 4 name
  return img.addBands(nir2_replaced, overwrite=True).set('REFLECTANCE_MULT_BAND_NIR2',ee.Number(img.get('REFLECTANCE_MULT_BAND_NIR1'))).set('REFLECTANCE_ADD_BAND_NIR2',ee.Number(img.get('REFLECTANCE_ADD_BAND_NIR1')))

def get_MSS_thresholds(img):
  sun_elevation = ee.Number(img.get('SUN_ELEVATION'))
  zenith = ee.Number(90).subtract(sun_elevation).multiply(degree2rad)
  cosZ = zenith.cos() 

  dark_threshold = NIR2_MSS_dark_threshold_mss_server.multiply(cosZ).subtract(ee.Number(img.get('REFLECTANCE_ADD_BAND_NIR2'))).divide(ee.Number(img.get('REFLECTANCE_MULT_BAND_NIR2')))
  green_threshold = ee.Number(green_MSS_bright_threshold).multiply(cosZ).subtract(ee.Number(img.get('REFLECTANCE_ADD_BAND_G'))).divide(ee.Number(img.get('REFLECTANCE_MULT_BAND_G')))

  return img.set('dark_threshold',dark_threshold).set('green_threshold',green_threshold).set('zenith',zenith).set('cosZ',cosZ)

props_to_copy_MSS = [
      'date_str', 'system:time_start', 'REFLECTANCE_MULT_BAND_G',
      'REFLECTANCE_MULT_BAND_NIR2', 'REFLECTANCE_ADD_BAND_NIR2',
      'REFLECTANCE_ADD_BAND_G', 'SUN_ELEVATION', 'SUN_AZIMUTH',
      'SPACECRAFT_ID', 'SCENE_CENTER_TIME'
    ]

def mosaic_MSS_img1coef(img):
    second_img = ee.Image(img.get('second'))
    return ee.ImageCollection([img, second_img]).qualityMosaic('QA_PIXEL').copyProperties(img, props_to_copy_MSS) # use image 1 metadata for the mosaic

def masker_MSS(img):
  clearsky = img.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).unmask().eq(0).rename('cloud_mask')

  green_band_mask = img.select('G').lt(ee.Number(img.get('green_threshold'))).rename('green_mask') # mask out green reflectance > 15%; should work for snow too (though would be nice if there was a SWIR band for a more accurate snow detection), but what about clearcuts? It might mask out some fresh clearcuts too :(
  dark_pixels = img.select('NIR2').gt(ee.Number(img.get('dark_threshold'))).rename('dark_pixels')

  clearsky = clearsky.updateMask(clearsky).updateMask(green_band_mask).rename('cloud_mask')
 
  datapixels = img.select(bands[sensor]).unmask().gt(0).reduce('min') # here! not sure it's in the right order; should be img.select(bands[sensor]).reduce('min').gt(0), no? Unlike S2, there is no need to unmask() here for some reason
  saturationMask = img.select('QA_RADSAT').eq(0)

  mask_pre_AAFC = clearsky.updateMask(datapixels).updateMask(saturationMask).rename('mask_pre_AAFC')

  mask_without_shadows = mask_pre_AAFC.updateMask(water_and_agri_mask).updateMask(clearsky).rename('mask_noshadow') # this is required to check if the image is improperly georeferenced
  n_clearsky_and_shadows = mask_without_shadows.reduceRegion(**{
      'reducer': ee.Reducer.count(),
      'geometry': S2_tile['M'],
      'crs': S2_projection,
      'scale': reducer_resolution,
      'bestEffort': True,
      'maxPixels': 1e7
    }).get('mask_noshadow')

  n_clearsky_and_shadows = ee.Number(n_clearsky_and_shadows).multiply(min_nondark)

  mask = mask_without_shadows.updateMask(dark_pixels).rename('mask') # note: this mask is not good because it's not coregistered to AAFC, but it's good for the purpose of n_clearsky

  n_clearsky = mask.reduceRegion(**{
      'reducer': ee.Reducer.count(),
      'geometry': S2_tile['M'],
      'crs': S2_projection,
      'scale': reducer_resolution,
      'bestEffort': True,
      'maxPixels': 1e7
    }).get('mask')

  return img.select(bands[sensor]).addBands(mask).addBands(clearsky).addBands(mask_pre_AAFC).set('clearsky', n_clearsky).addBands(green_band_mask).set('clearsky_with_shadows', n_clearsky_and_shadows)

# this is not robust - especially when there are large water bodies (sea, very large lakes); it doesn't work when there are no water bodies in the tile. Example: 37TGH 2012-09-05_M5
def coregister_or_not(img): # check if water bodies in MSS align with the reference Landsat TM image 
  nir2_in_water =  img.select('NIR2').updateMask(landsat5_best_watermask).updateMask(img.select('mask_pre_AAFC')).rename('inWater') # mask everything but water bodies based on their location in the reference TM image, and clear pixels
  nir2_mean_in_water = nir2_in_water.reduceRegion(**{
      'reducer': ee.Reducer.mean(),
      'geometry': S2_tile['M'],
      'scale': reducer_resolution, # 60
      'bestEffort': True,
      'maxPixels': 1e9
  }).get('inWater') # get average NIR2 reflectance in pixels that are supposed to represent water bodies
  
  return img.set('inWater',nir2_mean_in_water)

# low quality images have lots of noise and striping issues which can be detected by calculating green-red and NIR1-NIR2 interband correlation
region_fc = ee.FeatureCollection([ee.Feature(S2_tile['M'])])
def green_red_corr(img):
    corr_fc = img.select(['G', 'R']).reduceRegions( # reduceRegion replaced with reduceRegions on 2025-11-15
        collection=region_fc,
        reducer=ee.Reducer.pearsonsCorrelation(),
        scale=60
    )
    corr = corr_fc.first().get('correlation')
    return img.set('green_red_correlation', corr)

def nir_corr(img):
    corr_fc = img.select(['NIR1', 'NIR2']).reduceRegions(
        collection=region_fc,
        reducer=ee.Reducer.pearsonsCorrelation(),
        scale=60
    )
    corr = corr_fc.first().get('correlation')
    return img.set('NIR_correlation', corr)

# this performs cloud shadow masking, cloud buffer, topographic correction, and co-registration with L5 # here! split this function into three
def coreg_and_illumination_MSS(img): # this performs cloud shadow masking, cloud buffer, co-registration with L5, and topographic correction
  # img = img.clip(S2_tile['GEE'])
  azimuth_image = ee.Image.constant(ee.Number(img.get('SUN_AZIMUTH')).multiply(degree2rad)) #.clip(S2_tile) # degrees to radians

  is_cloud = img.select('cloud_mask').unmask().eq(0).rename('cloud_mask_Not')
  # Try this formula: Cloud test = (B1 > 0.175 and NDGR > 0.0) or B1 > 0.39, where NDGR = (B1 − B2)/(B1 + B2) # (c) 2015 Automated cloud and cloud shadow identification in Landsat MSS imagery for temperate ecosystems.pdf
  
  dark_pixels = img.select('NIR2').lt(ee.Number(img.get('dark_threshold'))).rename('dark_pixels') # .multiply(WorldCover10m_water)

  # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
  shadow_azimuth = ee.Number(img.get('SUN_AZIMUTH')).subtract(ee.Number(90)) # here! not sure if this is correct! should be ee.Number(90).subtract(ee.Number(img.get('SUN_AZIMUTH'))); according to https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

  shadow_distance_px = cloud_height_mss.multiply(ee.Number(img.get('zenith')).tan()).ceil() # 

  # Project shadows from clouds for the distance specified by the cloud_shadow_distance input. cloud_shadow_distance = maximum distance (km) to search for cloud shadows from cloud edges
  cld_proj = (is_cloud.directionalDistanceTransform(shadow_azimuth, shadow_distance_px) # This functions takes degrees, not radians!
      .reproject(**{'crs': S2_projection, 'crsTransform': transform[sensor]}) # (crs=S2_projection, crsTransform = transform[sensor]) # here! use proper transform! # removed 2024-07-17 reproject(**{'crs': S2_projection, 'scale': reducer_resolution})
      .select('distance')
      .mask()).rename('cloud_transform')

  # Identify the intersection of dark pixels with cloud shadow projection.
  shadows = cld_proj.multiply(dark_pixels).unmask().rename('shadows') # this is positive

  is_cloud_and_shadow = is_cloud.add(shadows).eq(0).rename('cloud_mask_no_buffer')

  # here! should remove small cloud patches first and only then dilate remaining pixels by BUFFER input.
  not_cloud_and_shadow = is_cloud_and_shadow.unmask().focalMin(**{'radius': cloud_buffer_server, 'kernelType': 'square'}).rename('final_cloud') # 39-75 # focalMin dialates in this case, so .focalMax(**{'radius': cloud_buffer_server}) should go first if you want to remove small patches of masked pixels

  img = img.select(bands[sensor]).updateMask(img.select('mask_pre_AAFC')).updateMask(not_cloud_and_shadow) #.reproject(crs=S2_projection, scale=resolution[sensor])

  img = img.float() # adding .resample('bicubic') here doesn't make any difference

  # Co-register to Landsat 5
  displacement = img.select('NIR2').displacement(**{
      'referenceImage': landsat5_best,
      # 'projection': S2_projection, # The projection in which to output displacement values. The default is the projection of the first band of the reference image.
      'maxOffset': 180.0, # The maximum offset allowed when attempting to align the input images, in meters. Using a smaller value can reduce computation time significantly, but it must still be large enough to cover the greatest displacement within the entire image region.
      # 'patchWidth': 360.0, # Patch size for detecting image offsets, in meters. This should be set large enough to capture texture, as well as large enough that ignorable objects are small within the patch. Default is null. Patch size will be determined automatically if not provided.
      'stiffness': 7 # little to no difference betwen stiffness=7 and stiffness=10. Will crash if set below 7 due to insafficient memory
  })
  img = img.displace(displacement)

   # Topographic correction: https://mygeoblog.com/2018/10/17/terrain-correction-in-gee/
   # illumination = cos(slope)*cos(zenith) + sin(slope)*sin(zenith)*cos(azimuth - aspect) # this illumination image is not affected by co-registration errors. Therefore, topographic correction should go after co-registration
  cos_slope_x_cosZ = cos_slope_noreproj.multiply(ee.Number(img.get('cosZ'))) 
  illumination_azimuth = sin_slope_noreproj.multiply(ee.Number(img.get('zenith')).sin()).multiply(azimuth_image.subtract(aspect_noreproj).cos()) # sin(slope)*sin(zenith)*cos(azimuth - aspect)
  illumination = cos_slope_x_cosZ.add(illumination_azimuth).rename('illumination') 
  
  img = img.addBands(illumination).updateMask(water_and_agri_mask) # add the illumination image and apply AAFC mask only after co-registration
  
  for band in bands[sensor]:
    out = img.select('illumination', band).reduceRegion(**{
      'reducer': ee.Reducer.linearFit(), # Compute coefficients: a(slope), b(offset), c(b/a)
      'geometry': linearFitGeometry,
      'scale': 120,
      'bestEffort': True,
      'maxPixels': 1000000000
    }) 
  
    out_a = ee.Number(out.get('scale'))
    out_b = ee.Number(out.get('offset'))
    out_c = out_b.divide(out_a)
    # Apply the SCSc correction
    # replace img with img_i if you want to apply cloud mask server side instead of building and applying it again on client side
    SCSc_output = img.select(band).multiply(cos_slope_x_cosZ.add(out_c)).divide(illumination.add(out_c)).rename(band) # .round() here! convert to 10 bit or 16bit after topo correction so no info is lost! (but 6bit MSS is already converted to 8bit by USGS)
    
    img = img.addBands(**{
        'srcImg': SCSc_output,
        'overwrite': True
      })

  # convert to 16bit after topo correction so no info is lost! (6bit → 8bit (USGS) → float (topo correction) * 10 → 16bit)
  return img.select(bands[sensor]).multiply(100.0).uint16().set('date_str', img.get('date_str')).addBands(is_cloud_and_shadow.displace(displacement)).set('shadow_distance_px', shadow_distance_px)

def illumination_MSS(img): # this performs cloud shadow masking, cloud buffer, and topographic correction, but no co-registration with L5
  azimuth_image = ee.Image.constant(ee.Number(img.get('SUN_AZIMUTH')).multiply(degree2rad)) # degrees to radians

  is_cloud = img.select('cloud_mask').unmask().eq(0).rename('cloud_mask_Not') # Try this formula: Cloud test = (B1 > 0.175 and NDGR > 0.0) or B1 > 0.39, where NDGR = (B1 − B2)/(B1 + B2) # (c) 2015 Automated cloud and cloud shadow identification in Landsat MSS imagery for temperate ecosystems.pdf
  dark_pixels = img.select('NIR2').lt(ee.Number(img.get('dark_threshold'))).rename('dark_pixels')

  # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
  shadow_azimuth = ee.Number(img.get('SUN_AZIMUTH')).subtract(ee.Number(90)) # this works, though it should be ee.Number(90).subtract(ee.Number(img.get('SUN_AZIMUTH'))); according to https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless

  # Project shadows from clouds for the distance specified by the cloud_shadow_distance input. cloud_shadow_distance = maximum distance (km) to search for cloud shadows from cloud edges
  cld_proj = (is_cloud.directionalDistanceTransform(shadow_azimuth, cloud_shadow_distance) # This functions takes degrees, not radians!
      .select('distance')
      .mask()).rename('cloud_transform')

  # Identify the intersection of dark pixels with cloud shadow projection.
  shadows = cld_proj.multiply(dark_pixels).unmask().rename('shadows') # this is positive

  is_cloud_and_shadow = is_cloud.add(shadows).eq(0).rename('cloud_mask_no_buffer')

  # here! should remove small cloud patches first and only then dilate remaining pixels by BUFFER input.
  # not_cloud_and_shadow = is_cloud_and_shadow.unmask().focal_min(cloud_buffer_server).rename('final_cloud') #.Not() # same speed as .focal_max(**{'radius': 240, 'units': 'meters'}) # this produces great results but is very very slow # 1m8s vs 52 s/image # is_cloud_and_shadow.Not().focal_min(2).focal_max(8) = 2m20s
  not_cloud_and_shadow = is_cloud_and_shadow.unmask().focalMin(**{'radius': cloud_buffer_server, 'kernelType': 'square'}).rename('final_cloud') # 39-75 # focalMin dialates in this case, so .focalMax(**{'radius': cloud_buffer_server}) should go first if you want to remove small patches of masked pixels

  img = img.select(bands[sensor]).updateMask(img.select('mask_pre_AAFC')).updateMask(not_cloud_and_shadow)

  img = img.float()

   # Topographic correction: https://mygeoblog.com/2018/10/17/terrain-correction-in-gee/
   # illumination = cos(slope)*cos(zenith) + sin(slope)*sin(zenith)*cos(azimuth - aspect) # this illumination image is not affected by co-registration errors. It should be added after co-registration
  cos_slope_x_cosZ = cos_slope_noreproj.multiply(ee.Number(img.get('cosZ'))) 
  illumination_azimuth = sin_slope_noreproj.multiply(ee.Number(img.get('zenith')).sin()).multiply(azimuth_image.subtract(aspect_noreproj).cos()) # sin(slope)*sin(zenith)*cos(azimuth - aspect)
  illumination = cos_slope_x_cosZ.add(illumination_azimuth).rename('illumination') 
  
  img = img.addBands(illumination).updateMask(water_and_agri_mask) # add the illumination image and apply AAFC mask only after co-registration
  
  for band in bands[sensor]:
    out = img.select('illumination', band).reduceRegion(**{
      'reducer': ee.Reducer.linearFit(), # Compute coefficients: a(slope), b(offset), c(b/a)
      'geometry': linearFitGeometry,
      'scale': 120,
      'bestEffort': True, # here! check how this affects the results
      'maxPixels': 1000000000
    }) 
  
    out_a = ee.Number(out.get('scale'))
    out_b = ee.Number(out.get('offset'))
    out_c = out_b.divide(out_a)
    # Apply the SCSc correction
    # replace img with img_i if you want to apply cloud mask server side instead of building and applying it again on client side
    SCSc_output = img.select(band).multiply(cos_slope_x_cosZ.add(out_c)).divide(illumination.add(out_c)).rename(band) # .round() here! convert to 10 bit or 16bit after topo correction so no info is lost!
    
    img = img.addBands(**{
        'srcImg': SCSc_output,
        'overwrite': True
      })

  return img.select(bands[sensor]).multiply(100.0).uint16().set('date_str', img.get('date_str')).addBands(is_cloud_and_shadow) #.uint16() 

def coreg_MSS(img): # this performs cloud shadow masking, cloud buffer, co-registration with L5, but no topographic correction
  is_cloud = img.select('cloud_mask').unmask().eq(0).rename('cloud_mask_Not')
  # Try this formula: Cloud test = (B1 > 0.175 and NDGR > 0.0) or B1 > 0.39, where NDGR = (B1 − B2)/(B1 + B2) # (c) 2015 Automated cloud and cloud shadow identification in Landsat MSS imagery for temperate ecosystems.pdf
  
  dark_pixels = img.select('NIR2').lt(ee.Number(img.get('dark_threshold'))).rename('dark_pixels')

  # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
  shadow_azimuth = ee.Number(img.get('SUN_AZIMUTH')).subtract(ee.Number(90))

  shadow_distance_px = cloud_height_mss.multiply(ee.Number(img.get('zenith')).tan()).ceil() # 

  # Project shadows from clouds for the distance specified by the cloud_shadow_distance input. cloud_shadow_distance = maximum distance (km) to search for cloud shadows from cloud edges
  cld_proj = (is_cloud.directionalDistanceTransform(shadow_azimuth, shadow_distance_px) # This functions takes degrees, not radians!
      .reproject(**{'crs': S2_projection, 'crsTransform': transform[sensor]})
      .select('distance')
      .mask()).rename('cloud_transform')

  # Identify the intersection of dark pixels with cloud shadow projection.
  shadows = cld_proj.multiply(dark_pixels).unmask().rename('shadows') # this is positive
  is_cloud_and_shadow = is_cloud.add(shadows).eq(0).rename('cloud_mask_no_buffer')
  not_cloud_and_shadow = is_cloud_and_shadow.unmask().focalMin(**{'radius': cloud_buffer_server, 'kernelType': 'square'}).rename('final_cloud') # focalMin dialates in this case, so .focalMax(**{'radius': cloud_buffer_server}) should go first if you want to remove small patches of masked pixels

  img = img.select(bands[sensor]).updateMask(img.select('mask_pre_AAFC')).updateMask(not_cloud_and_shadow)
  img = img.float() # adding .resample('bicubic') here doesn't make any difference

  # Co-register to Landsat 5
  displacement = img.select('NIR2').displacement(**{
      'referenceImage': landsat5_best,
      'maxOffset': 180.0, # The maximum offset allowed when attempting to align the input images, in meters. Using a smaller value can reduce computation time significantly, but it must still be large enough to cover the greatest displacement within the entire image region.
      # 'patchWidth': 360.0, # Patch size for detecting image offsets, in meters. This should be set large enough to capture texture, as well as large enough that ignorable objects are small within the patch. Default is null. Patch size will be determined automatically if not provided.
      'stiffness': 7 # little to no difference betwen stiffness=7 and stiffness=10. Will crash if set below 7 due to insufficient memory
  })
  img = img.displace(displacement).updateMask(water_and_agri_mask)
  
  return img.select(bands[sensor]).multiply(100.0).uint16().set('date_str', img.get('date_str')).addBands(is_cloud_and_shadow.displace(displacement)).set('shadow_distance_px', shadow_distance_px)

def maskonly_MSS(img): # this performs cloud shadow masking, cloud buffer, but no co-registration with Landsat TM and no topographic correction
  is_cloud = img.select('cloud_mask').unmask().eq(0).rename('cloud_mask_Not')
  # Try this formula: Cloud test = (B1 > 0.175 and NDGR > 0.0) or B1 > 0.39, where NDGR = (B1 − B2)/(B1 + B2) # (c) 2015 Automated cloud and cloud shadow identification in Landsat MSS imagery for temperate ecosystems.pdf
  
  dark_pixels = img.select('NIR2').lt(ee.Number(img.get('dark_threshold'))).rename('dark_pixels')

  # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
  shadow_azimuth = ee.Number(img.get('SUN_AZIMUTH')).subtract(ee.Number(90)) # this works, though it should be ee.Number(90).subtract(ee.Number(img.get('SUN_AZIMUTH'))); according to https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless
  shadow_distance_px = cloud_height_mss.multiply(ee.Number(img.get('zenith')).tan()).ceil()

  # Project shadows from clouds for the distance specified by the cloud_shadow_distance input. cloud_shadow_distance = maximum distance (km) to search for cloud shadows from cloud edges
  cld_proj = (is_cloud.directionalDistanceTransform(shadow_azimuth, shadow_distance_px) # This functions takes degrees, not radians!
      .reproject(**{'crs': S2_projection, 'crsTransform': transform[sensor]})
      .select('distance')
      .mask()).rename('cloud_transform')

  # Identify the intersection of dark pixels with cloud shadow projection.
  shadows = cld_proj.multiply(dark_pixels).unmask().rename('shadows') # this is positive
  is_cloud_and_shadow = is_cloud.add(shadows).eq(0).rename('cloud_mask_no_buffer')
  not_cloud_and_shadow = is_cloud_and_shadow.unmask().focalMin(**{'radius': cloud_buffer_server, 'kernelType': 'square'}).rename('final_cloud') # focalMin dialates in this case, so .focalMax(**{'radius': cloud_buffer_server}) should go first if you want to remove small patches of masked pixels

  # need to optimize this 
  img = img.select(bands[sensor]).updateMask(img.select('mask_pre_AAFC')).updateMask(not_cloud_and_shadow).updateMask(water_and_agri_mask)
  
  return img.select(bands[sensor]).set('date_str', img.get('date_str')).addBands(is_cloud_and_shadow).set('shadow_distance_px', shadow_distance_px)

def reprojecter(img):
  return img.reproject(crs=S2_projection, crsTransform = transform[sensor]).clip(S2_tile['GEE']).set('time1', img.get('SCENE_CENTER_TIME')).set('time2', img.get('SCENE_CENTER_TIME'))

def align_60m(img): # client-side global "coregistration"
    highest_cor = 0
    h_shift_best = 0
    v_shift_best = 0

    for v_shift in range(-1*max_shift_s2_10m,max_shift_s2_10m+1):
      for h_shift in range(-1*max_shift_s2_10m,max_shift_s2_10m+1):
        # shift image v_shift pixels up or down and h_shift left or right
        img_shifted = img[max_shift_s2_10m+v_shift:v_shift+h[60]-max_shift_s2_10m, max_shift_s2_10m+h_shift:h_shift+w[60]-max_shift_s2_10m].copy() # trim max_shift*2 pixels vertically and horizontally # do not remove ".copy()"!!

        img_shifted[landsat5_best_trimmed_0where] = 0
        landsat5_best_trimmed_copy = landsat5_best_trimmed.copy()
        landsat5_best_trimmed_copy = landsat5_best_trimmed_copy[img_shifted > 0]
        img_shifted = img_shifted[img_shifted > 0]

        cor = np.corrcoef(img_shifted,landsat5_best_trimmed_copy)[0, 1] # Calculate the correlation coefficient

        if cor > highest_cor:
          highest_cor = cor
          h_shift_best = h_shift
          v_shift_best = v_shift

    print('v_shift_best, h_shift_best,highest correlation:',-60*v_shift_best, "meters,", -60*h_shift_best, "meters,",highest_cor)

    return -1*v_shift_best, -1*h_shift_best

## Select and download MSS data

In [ ]:
# future work: use Landsat 5 TM cloud masks to mask Landsat 5 MSS

sensor = 'M'
bands_MSS_2012 = bands[sensor].copy()
bands_MSS_2012.remove('NIR2')

n_images[sensor] = 0

if do_MSS:
  band_names_MSS = {}
  for s in [1,2,3]:
    band_names_MSS[s] = ['B4', 'B5', 'B6', 'B7','QA_PIXEL','QA_RADSAT']
  for s in [4,5]:
    band_names_MSS[s] = ['B1', 'B2', 'B3', 'B4','QA_PIXEL','QA_RADSAT']

  band_names_renamed_MMS = ['G', 'R', 'NIR1','NIR2','QA_PIXEL','QA_RADSAT']
  spacecraft_ID_short = {'LANDSAT_1': '_M1','LANDSAT_2': '_M2','LANDSAT_3': '_M3','LANDSAT_4': '_M4','LANDSAT_5': '_M5'}

  MSS_y_shift,MSS_x_shift = np.load(MGRS_tile+'/M_shift.npy') # shift in meters, if positive = shift in the SE direction
  landsat_y_shift,landsat_x_shift = np.load(MGRS_tile+'/L_shift.npy') # shift in meters, if positive = shift in the SE direction
  print('Landsat misalignment y', landsat_y_shift,'; x', landsat_x_shift) # L shift and MSS shift are for the same UTM projections, so this is correct

  start_time = time.time()
  constant1 = ee.Image(1)
  constant2 = ee.Image(2)

  for year in range(start_year, end_year+1):
    if (year > 1988 and year < 2012) or (year > 2013) or (year == 2013 and start_DOY > 7): # Skip years with no MSS acquisitions; Last M5 image was acquired on 2013-01-07, so no MSS data after that
      continue
    
    print('_____________________\nprocessing year: ', year)

    landsat_collection_this_year = ee.ImageCollection([constant1, constant2])

    available_sensors = []
    if year >= 1972 and year <= 1977:
      available_sensors.append(1)
    if year >= 1975 and year <= 1981:
      available_sensors.append(2)
    if year >= 1978 and year <= 1982:
      available_sensors.append(3)
    if year >= 1982 and year <= 1993:
      available_sensors.append(4)
    if (year >= 1984 and year <= 1993) or year >= 2012:
      available_sensors.append(5)
      if year >= 2012: # find a new 30m Landsat reference image for MSS co-registration (L8). here! when restarting this cell after reaching 2012, TSD will use 2012 reference image for 1972 - 1988 MSS data! this is not robust
        print('New reference image for coregistration - Landsat 8')
        # here! should use pan band instead, but then would need to make changes to the coregistration function because it uses NIR bands.
        landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1').filterBounds(S2_tile['M']).filter(ee.Filter.calendarRange(2013, 2015,'year')).filter(ee.Filter.calendarRange(start_DOY_reference,end_DOY_reference,'day_of_year')).filter(ee.Filter.lt('CLOUD_COVER', 30)).select(['B5','QA_PIXEL'])   
        landsat_collection = landsat_collection.sort('CLOUD_COVER').limit(20)
        landsat_collection = landsat_collection.map(set_bicubic)
        landsat5_best = landsat_collection.select(['B5']).median().uint16().rename('NIR')
        landsat5_best = landsat5_best.reproject(crs=S2_projection, crsTransform = transform['M_30m']).clip(S2_tile['GEE'])
        landsat5_best_watermask = landsat5_best.select('NIR').lt(8000).rename('landsat5_best_watermask') # here! hard-coded thershold! This is not robust as these are DN. 
        landsat5_best_watermask_np = gee2np(landsat5_best_watermask.reproject(crs=S2_projection, crsTransform = transform['M']),'landsat5_best_watermask',"M",'Landsat8_composite_watermask',16)

        landsat5_best = landsat5_best.updateMask(water_and_agri_mask)
        landsat5_best_np = gee2np(landsat5_best.reproject(crs=S2_projection, crsTransform = transform['M_30m']),'NIR',"M_30m", 'Landsat8_composite_for_MSS_coregistration',16) 

        # Check if water > 1%
        if np.count_nonzero(landsat5_best_watermask_np) > 0.01 * landsat5_best_watermask_np.size:
            check_MSStoTM_water_alignment = True
            print("Water > 1%")
        else:
            check_MSStoTM_water_alignment = False
            print("Water <= 1%. TSD won't be able to use water bodies to filter out incorrectly georeferenced MSS images.")

        landsat5_best_trimmed = landsat5_best_np[max_shift_s2_10m:h[60]-max_shift_s2_10m, max_shift_s2_10m:w[60]-max_shift_s2_10m]
        landsat5_best_trimmed_0where = landsat5_best_trimmed <= 0

    for s in available_sensors:    
      landsat_collection = ee.ImageCollection('LANDSAT/LM0'+str(s)+'/C02/T2').filterBounds(S2_tile['M']).filter(ee.Filter.calendarRange(year, year,'year')).filter(ee.Filter.calendarRange(start_DOY,end_DOY,'day_of_year')).select(band_names_MSS[s],band_names_renamed_MMS)
      landsat_collection_T1 = ee.ImageCollection('LANDSAT/LM0'+str(s)+'/C02/T1').filterBounds(S2_tile['M']).filter(ee.Filter.calendarRange(year, year,'year')).filter(ee.Filter.calendarRange(start_DOY,end_DOY,'day_of_year')).select(band_names_MSS[s],band_names_renamed_MMS)
      landsat_collection = landsat_collection.merge(landsat_collection_T1)

      # mosaic images
      landsat_collection = landsat_collection.map(rename_coef_MSS_4to5) if s > 3 else landsat_collection.map(rename_coef_MSS_1to3)
      landsat_collection = landsat_collection.filter(ee.Filter.neq('REFLECTANCE_ADD_BAND_G', None)) # here! wasting images just because there are no DN-to-TOA coefficients!

      if year >= 2012: # remove this if because pre-2012 can also have NIR2 band missing
        # print('2012 lol')

        MSS_collection_4bands = landsat_collection.filter(ee.Filter.neq('REFLECTANCE_ADD_BAND_NIR2', None))
        MSS_collection_3bands = landsat_collection.filter(ee.Filter.eq('REFLECTANCE_ADD_BAND_NIR2', None)).map(rename_coef_MSS_4to5_2012) # replace NIR2 with NIR1 and REFLECTANCE_MULT_BAND_NIR2 with REFLECTANCE_MULT_BAND_NIR1 in images that lack the NIR2 band
        landsat_collection = MSS_collection_4bands.merge(MSS_collection_3bands)

      landsat_collection_mosiac_checked = landsat_collection.map(mosaic_or_not)
      landsat_collection_0before = landsat_collection_mosiac_checked.filter(ee.Filter.eq('n_before', 0))
      landsat_collection_gte1before = landsat_collection_mosiac_checked.filter(ee.Filter.gt('n_before', 0))
      landsat_collection_1img = landsat_collection_0before.filter(ee.Filter.eq('n_after', 1))
      landsat_collection_2img = landsat_collection_0before.filter(ee.Filter.gt('n_after', 1))

      #  PRE-JOIN instead of slow filtering inside map
      join_filter = ee.Filter.equals(leftField='date_str', rightField='date_str')
      joined = ee.Join.saveFirst('second').apply(landsat_collection_2img, landsat_collection_gte1before, join_filter)

      landsat_collection_2img_mosaiced = ee.ImageCollection(joined.map(mosaic_MSS_img1coef))
      landsat_collection = landsat_collection_2img_mosaiced.merge(landsat_collection_1img).map(get_MSS_thresholds)
      landsat_collection_this_year = landsat_collection_this_year.merge(landsat_collection)

    # Apply various filters to remove bad images
    landsat_collection_this_year = landsat_collection_this_year.filter(ee.Filter.gt('green_threshold', 0)).map(masker_MSS)
    landsat_collection_this_year = landsat_collection_this_year.filter(ee.Filter.greaterThan(**{'leftField': 'clearsky', 'rightField': 'clearsky_with_shadows'}))

    if check_MSStoTM_water_alignment:
      landsat_collection_this_year = landsat_collection_this_year.map(coregister_or_not) # check if water bodies in MSS and 30m Landsat align. It might be better to check this before mosaicking.
      landsat_collection_this_year = landsat_collection_this_year.filter(ee.Filter.lessThan(**{'leftField': 'inWater', 'rightField': 'dark_threshold'})) # remove MSS images where water bodies don't align with the reference Landsat 5 TM image

    # filter out images that have an interband (greed-red and NIR1-NIR2) correlation coefficient < 0.8
    landsat_collection_this_year = landsat_collection_this_year.map(green_red_corr).filter(ee.Filter.gt('green_red_correlation', 0.8))
    landsat_collection_this_year = landsat_collection_this_year.map(nir_corr).filter(ee.Filter.gt('NIR_correlation', 0.8))

    if landsat_collection_this_year.size().getInfo() == 0:
      print('no good Landsat MSS images for year ', year)
      continue 

    coverage = 0
    landsat_collection_clearsky = 0
    clearsky_threshold = clearsky_threshold_global

    n_landsat = 0
    n_landsat_previous = 0

    while (coverage < min_coverage and clearsky_threshold >= 0.2) or (n_landsat < 2 and coverage > min_coverage): # reduce the clearsky percentage requirement for individual images 10% at a time until you get >90% coverage of the study area for this year # added "or n_landsat < 2" on 2025-10-27
      clearsky_threshold_px = ee.Number(clearsky_threshold).multiply(n_not_water_max) 
      landsat_collection_clearsky = landsat_collection_this_year.filter(ee.Filter.gt('clearsky', clearsky_threshold_px))
            
      n_landsat = landsat_collection_clearsky.size().getInfo()
      if n_landsat > n_landsat_previous:
        n_landsat_previous = n_landsat

        print('single-image clearsky threshold {}%: {} images'.format(round(clearsky_threshold*100), n_landsat))     
        yearly_mosaic = landsat_collection_clearsky.select('mask').mosaic().gt(0).rename('coverage')

        coverage_stats = yearly_mosaic.reduceRegion(**{
            'reducer': ee.Reducer.count(),
            'geometry': S2_tile['M'],
            'crs': S2_projection,
            'scale': reducer_resolution,
            'bestEffort': True,
            'maxPixels': 1e7
          }).get('coverage')     
        coverage_px = coverage_stats.getInfo() # number of land pixels covered by all images this year
        coverage = coverage_px/n_not_water_max_client
        print('coverage: {:.1f}%'.format(coverage*100))
      elif clearsky_threshold != clearsky_threshold_global:
        print('single-image clearsky threshold {}%: same'.format(round(clearsky_threshold*100))) 
      else:
        print('No images matching clearsky critirion')
      
      clearsky_threshold -= 0.1
    
    n_images[sensor] += n_landsat

    landsat_collection_clearsky = landsat_collection_clearsky.map(reprojecter) # DO NOT remove reprojecter, it's required for coregistration
    if do_topo: # perform topographic correction
      landsat_collection_clearsky = landsat_collection_clearsky.map(coreg_and_illumination_MSS) if coregister_MSS else landsat_collection_clearsky.map(illumination_MSS) # topographic correction with or without co-registration with Landsat 5 TM
    elif coregister_MSS: # option not to do topographic correction but still coregister MSS images to Landsat 5 TM
      landsat_collection_clearsky = landsat_collection_clearsky.map(coreg_MSS) 
    else: # no topo, no co-registration - only masking
      landsat_collection_clearsky = landsat_collection_clearsky.map(maskonly_MSS)

    if n_landsat == 0:
      continue

    # =============== Download MSS data =====================
    sensors_L = landsat_collection_clearsky.aggregate_array('system:index').getInfo()
    sensors_L = ['_M'+i[-17:-16] for i in sensors_L] # extract sensor name from image id, e.g. '1_1_2_2_2_LM01_048025_19720727' → '_M1' # is this robust?

    dates_MSS = landsat_collection_clearsky.aggregate_array('date_str').getInfo() 
    print(dates_MSS)
    print(sensors_L)

    collectionList = landsat_collection_clearsky.toList(n_landsat)

    start = time.time()
    t_previous = start

    short_names = []

    for i in range(n_landsat):
      start_i = time.time()
      short_names.append(dates_MSS[i] + sensors_L[i])

      if skip_existing:
        tif_ok = (not save_tif) or os.path.exists(MGRS_tile + '_tif/' + short_names[i] + '.tif')
        np_ok  = (not save_np)  or os.path.exists(MGRS_tile + '/' + short_names[i] + '_data.npy')
        if tif_ok and np_ok:
          print(short_names[i] + ' already downloaded, skipping...') 
          continue

      print('processing image',i+1,'of',n_landsat,short_names[i])
      
      gee_image = ee.Image(collectionList.get(i))
      image_L = gee2np(gee_image, (bands_MSS_2012 if year >= 2012 else bands[sensor])+['cloud_mask_no_buffer'],sensor)
      buffered_clouds = cloud_bufferer(image_L[-1], round(cloud_buffer_m/resolution[sensor])) # this includes snow, ice, and smoke
      image_L = image_L[:-1]
      image_L[:,buffered_clouds == 0] = 0

      if align_MSS: # shift the MSS image vertically and horizontally until it align with the reference Landsat TM image
        y_shift_60m, x_shift_60m = align_60m(image_L[-1].copy())
        print('y_shift_60m, x_shift_60m',y_shift_60m, x_shift_60m)
        
        if y_shift_60m != 0 or x_shift_60m != 0:
          image_L = np.pad(image_L, ((0, 0), (max_shift_s2_10m, max_shift_s2_10m), (max_shift_s2_10m, max_shift_s2_10m)), mode='constant', constant_values=0) # add some zero-valued pixels on each side of the image to prevent rollover when doing np.roll
          image_L = np.roll(image_L, (y_shift_60m, x_shift_60m), axis=(1, 2))
          image_L = image_L[:,max_shift_s2_10m:max_shift_s2_10m+h[60], max_shift_s2_10m:max_shift_s2_10m+w[60]] # Crop back to original shape
      
      data_where = np.any(image_L == 0, axis=0)
      image_L[:,data_where] = 0

      image_L[:,water_and_agri_mask_np['M']==0] = 0 # here! water_and_AAFC is applied twice (before shift (server-side) and here)
      print('all bands precessed', time.time() - start_i)
    
      save_data_and_mask(image_L, short_names[i], sensor, (bands_MSS_2012 if year >= 2012 else bands[sensor]), wavelengths_all[sensors_L[i]])

      t_now = time.time()
      print("image",i+1,"of",n_landsat,"processed: ", round(t_now - t_previous), "s; total:", round(t_now - start), "s; from beginning:", round((t_now - start_time)/60.0), "minutes\n")
      t_previous = t_now

  print(n_images[sensor],'Landsat MSS images downloaded:',round((time.time() - start_time)/60.0), "minutes\n")

## Download monthly GEDI data
as two files:
1. rh50, rh70, rh98, and biomass (uint16)
2. day-of-month of acquisition (uint8)

rh50, rh70, rh98 are in cm; biomass in Mg/ha

In [ ]:
# 5 seconds / month on a 100 Mbps connection
dt0gedi = datetime(2018, 1, 1, 0, 0, 0) # do not modify! # GEDI dates starts from 2018-01-01
dt0gee = datetime(1970, 1, 1, 0, 0, 0) # do not modify! # GEE epoch
gee2gedi = ee.Number((dt0gedi - dt0gee).total_seconds())

def gedi2date_height(img):
  # convert float32 (?) data to uint16 to save space. Unit meter → cm
  return img.select('rh50','rh70','rh95').multiply(100.0).clip(S2_tile['GEE']).toInt16().set('date',ee.Date(img.get('system:time_start')).format("yyyy-MM-dd")) # here! check if any pixels fall outside the int16 range

def gedi2date_biomass(img):
  dtime = img.select('delta_time')
  month_start = ee.Date(img.get('system:time_start')).millis().divide(1000).subtract(gee2gedi)
  dtime_int = dtime.subtract(month_start).divide(86400).rename('dayofmonth_biomass') # convert number of seconds since 2018-01-01 to Day of the Month so that data can be converted to uint8
  
  # biomass is already integer, unlike canopy height
  return img.addBands(dtime_int).updateMask(img.select('l4_quality_flag').eq(1)).updateMask(img.select('degrade_flag').eq(0)).updateMask(img.select('leaf_off_flag').eq(0)).select('agbd','dayofmonth_biomass').clip(S2_tile['GEE']).toInt16()

southmost_tile16_centroid = tiles['S2'][16][0].centroid(**{'maxError': 1}).coordinates().getInfo()
print('GEDI data available between 51.6°N and 51.6°S. Southmost_tile16_centroid:',southmost_tile16_centroid)
if southmost_tile16_centroid[1] > 51.6:
  print('Tile '+ MGRS_tile + ' is outside the GEDI coverage')
elif do_GEDI:
  GEDI_height_dataset= ee.ImageCollection('LARSE/GEDI/GEDI02_A_002_MONTHLY').filterBounds(S2_tile['GEE']).filter(ee.Filter.calendarRange(start_year,end_year,'year')).map(gedi2date_height)
  GEDI_biomass_dataset = ee.ImageCollection('LARSE/GEDI/GEDI04_A_002_MONTHLY').filterBounds(S2_tile['GEE']).filter(ee.Filter.calendarRange(start_year,end_year,'year')).map(gedi2date_biomass) # note: the biomass data has fewer points than the canopy height data! TSD only removes points that don't have both biomass and height
  GEDI_collection = GEDI_height_dataset.combine(GEDI_biomass_dataset)

  start = time.time()

  n_gedi_server = GEDI_collection.size()
  n_gedi = n_gedi_server.getInfo()

  gedi_dates = GEDI_collection.aggregate_array('date').getInfo()

  print('gedi_dates',gedi_dates)

  for s in GEDI_resolutions:
    GEDI_collection_r = GEDI_collection.map(lambda img: img.reproject(S2_projection, crsTransform=transform[s])) # nearest neighbour resampling affects positional accuracy, but how else can we preserve the values?
    GEDI_collection_list = GEDI_collection_r.toList(n_gedi_server)
    print("\nDownloading GEDI data at the spatial resolution of", s)

    meta_copy = metadata[s].copy()
    metadata[s]['nodata'] = -1000 # change no-data value for geotif from 0 to -1000 because GEDI RH metrics can be zero or negative

    for i in range(n_gedi):
      start_i = time.time()
      short_name = gedi_dates[i] + '_GD'

      if skip_existing:
        tif_ok = (not save_tif) or os.path.exists(MGRS_tile+'_tif/'+short_name+'_'+str(resolution[s])+'m.tif')
        np_ok = (not save_np) or os.path.exists(MGRS_tile+'/'+short_name+'_data_'+str(resolution[s])+'m.npy')
        if tif_ok and np_ok:
          print(short_name+' ('+str(resolution[s])+'m) already downloaded, skipping...')
          continue

      print('\nProcessing',short_name)
      gee_image = ee.Image(GEDI_collection_list.get(i))

      # Earth Engine performs nearest neighbor resampling by default during reprojection
      gedi_temp = gee2np(gee_image,['rh50', 'rh70', 'rh95', 'agbd', 'dayofmonth_biomass'], s)

      gedi_data = gedi_temp[:-1] # ['rh50', 'rh70', 'rh98', 'agbd']
      gedi_date = gedi_temp[-1] # ['dayofmonth_biomass']
      gedi_date[gedi_date < 1] = 0

      # Condition: biomass < -10 Mg/ha and height < 10 meters (1000 cm)
      mask = (gedi_data[-1] < -10) | (np.all(gedi_data[:-1] < -1000, axis=0))

      gedi_date[mask] = 0 # remove points where biomass < -10 tons (i.e., no-data pixels) and conapy height < -1000cm # here! this will remove a lot of points because biomass data is available only for some GEDI points
      gedi_date[water_and_agri_mask_np[s]==0] = 0

      if save_np:
        np.save(MGRS_tile+'/'+short_name+'_data_'+str(resolution[s])+'m', gedi_data[:,gedi_date>0]) # ~0.5 MB # uint16 (data only)
        np.save(MGRS_tile+'/'+short_name+'_date_'+str(resolution[s])+'m', gedi_date.astype('uint8')) # ~12 MB # uint8, pixel value = 1-31 days of the month, serves as a mask (pixel locations)

      if save_tif:
        gedi_temp[:,gedi_date<1] = -1000
        np2tif(gedi_temp, short_name+'_'+str(resolution[s])+'m', s, ['rh50', 'rh70', 'rh95', 'agbd', 'dayofmonth'],{'rh50':0.48, 'rh70':0.56, 'rh95':0.66, 'agbd':0.865, 'dayofmonth':1.65}) # use fake wavelengths so it's easier to display GEDI data in ENVI

      print("image",i+1,"of",n_gedi,"downloaded: ", time.time() - start_i, time.time() - start)
      
    metadata[s] = meta_copy

## Download Google Satellite Embedding yearly composites
Produced at 10 m in UTM projections for the years 2017 onward, these images can be resampled to the pixel grid of any sensor (paramerer 'embedding_resolutions').
TSD converts these embeddings from double (-1.0 to 1.0) to uint16 (1 to 65535), with 0 reserved for masked pixels (e.g., water and agricultural areas).

In [ ]:
# Satellite Embedding images are gridded into tiles of up to 163,840 m x 163,840 m each. The Satellite Embedding dataset is an image collection containing annual images from the years 2017 onward (e.g., 2017, 2018, 2019, etc.).
# There are 64 bands in this dataset, so expect very large files. When downloaded at 30m, the file size for one year is ~1.5 GB.
if do_GEE_embedding:
    start = time.time()
    
    # All 64 band names
    band_names = [f"A{i:02d}" for i in range(64)]
    # Subset size (4 or 6 depending on GEE limit). Haven't tried 6
    subset_size = 4
    # Split band names into chunks
    band_subsets = [band_names[i:i+subset_size] for i in range(0, len(band_names), subset_size)]
    # 64 bands between 0.4 and 2.4
    wavelengths_emb = np.linspace(0.4, 2.4, 64)
    # create dictionary like {'A01': 0.4, ..., 'A63': 2.4}
    band_dict = {f"A{i:02d}": round(float(w), 2) for i, w in enumerate(wavelengths_emb, start=0)} 
    gee_embedding_all = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL').filterMetadata('UTM_ZONE', 'equals', str(utm_zone)+('N' if is_northern else 'S')).filterBounds(S2_tile['GEE']) # .filterBounds(S2_tile_center) # this will not work properly if more than one version (DATASET_VERSION) of the Embedding become available

    # Download GEE embedding data
    for year in range(start_year, end_year+1):
        
        gee_embedding = gee_embedding_all.filter(ee.Filter.calendarRange(year, year,'year'))
        if (gee_embedding.size().getInfo() != 0):
            if skip_existing:
                tif_ok = (not save_tif) or os.path.exists(MGRS_tile+'_tif/GEE_embedding_'+s+'_'+str(year)+'.tif')
                np_ok = (not save_np) or os.path.exists(MGRS_tile+'/GEE_embedding_'+s+'_'+str(year)+'_data.npy')
                if tif_ok and np_ok:
                    print('GEE_embedding_'+s+'_' + str(year)+' already downloaded, skipping...')
                    continue
            
            print('Downloading GEE embedding (624 bands) for the year', year, 'at the spatial resolution of', s)

            gee_embedding = gee_embedding.mosaic()

            for s in embedding_resolutions:
                # "reduceResolution(**{'reducer': ee.Reducer.mean(), 'bestEffort': True})" = set spatial resampling to pixel aggregation (the most accurate resampling method). Remove this code to use nearest neighbour
                gee_embedding_s = gee_embedding.setDefaultProjection(S2_projection, crsTransform = transform[s]).reduceResolution(**{'reducer': ee.Reducer.mean(), 'bestEffort': True}).reproject(S2_projection, crsTransform = transform[s]).clip(S2_tile[s]) # the reduceResolution above ensures that reproject uses pixel aggregation instead of nearest neighbor

                # convert double (from -1.0 to 1.0) to uint16 (1 to 65535, with 0 reserved for masked pixels - e.g., water and agri)
                gee_embedding_s = gee_embedding_s.multiply(32767).add(32768).round().uint16()

                # Download and stack results
                embedding_arrays = []
                for subset in band_subsets:
                    print(f"Downloading bands: {subset}")
                    arr = gee2np(gee_embedding_s, subset, s,'', 4)
                    embedding_arrays.append(arr)

                # Concatenate along the last dimension (bands)
                embedding_full = np.concatenate(embedding_arrays, axis=0)
                print("Final embedding array shape:", embedding_full.shape)
                embedding_full[:,water_and_agri_mask_np[s] == 0] = 0 # apply water and AAFC mask
                save_data_and_mask(embedding_full, 'GEE_embedding_'+s+'_'+str(year), s, band_names, band_dict)
                print('GEE embedding data downloaded in ', round(time.time() - start), "seconds\n")